<a href="https://colab.research.google.com/github/dkalenov/backtesting/blob/main/hyperopt_backtesting_by_month.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Backtest

In [ ]:
auto_mode=True

symbol = 'SUIUSDT'
interval = '15m'
start_date = '2023-10'
end_date = '2024-11'
max_evals = 70


symbol_1 = 'SUIUSDT'
interval_1 = '15m'
start_date_1 = '2023-10'
end_date_1 = '2024-11'
max_evals_1 = 70


symbol_2 = 'SUIUSDT'
interval_2 = '15m'
start_date_2 = '2023-10'
end_date_2 = '2024-11'
max_evals_2 = 70

## Connections and imports

### Drive Connection & DB

In [ ]:
import pandas as pd
from google.colab import drive
import os

# Подключение к Google Drive
drive.mount('/content/drive')

# Указать путь к папке с файлами на Google Drive
shared_folder_path = '/content/drive/MyDrive/Sloping bot'

# Указать пути к файлам параметров и результатов бэктеста
params_file_path = f'{shared_folder_path}/optimized_params.csv'
results_file_path = f'{shared_folder_path}/backtest_results.csv'

# Проверка существования файла с оптимизированными параметрами
if os.path.exists(params_file_path):
    optimized_params = pd.read_csv(params_file_path)
else:
    params_columns = [
        'Symbol', 'Interval', 'Start Date', 'End Date', 'Test Return [%]',
        'window_length', 'min_space', 'sloping_atr_length', 'stop_loss_multiplier',
        'take_profit_multiplier'
    ]
    optimized_params = pd.DataFrame(columns=params_columns)

# Проверка существования файла с результатами бэктеста
if os.path.exists(results_file_path):
    backtest_results = pd.read_csv(results_file_path)
else:
    backtest_results_columns = [
        'Symbol', 'Interval', 'Start Date', 'End Date', 'Total Trades', 'Total PNL', 'Percentage PNL (%)',
        'Win Rate (%)', 'Profit Factor', 'Risk Ratio',
        'Num Long Trades', 'Num Short Trades', 'Num Profitable Trades', 'Num Unprofitable Trades',
        'Avg Profit', 'Avg Loss', 'Max Profit', 'Min Profit', 'Max Loss', 'Min Loss',
        'Avg Long Duration (days)', 'Avg Long Duration (hours)', 'Avg Long Duration (minutes)', 'Avg Long Duration (seconds)',
        'Avg Short Duration (days)', 'Avg Short Duration (hours)', 'Avg Short Duration (minutes)', 'Avg Short Duration (seconds)',
        'window_length', 'min_space', 'sloping_atr_length', 'stop_loss_multiplier', 'take_profit_multiplier'
    ]
    backtest_results = pd.DataFrame(columns=backtest_results_columns)




Mounted at /content/drive


In [ ]:
# import pandas as pd
# from google.colab import drive
# import os

# # Connect to Google Drive
# drive.mount('/content/drive')

# # Specify the path to the shared folder on Google Drive
# shared_folder_path = '/content/drive/MyDrive/Sloping bot'

# # Load existing DataFrames or create new ones
# params_file_path = f'{shared_folder_path}/optimized_params.xlsx'
# results_file_path = f'{shared_folder_path}/backtest_results.xlsx'

# # Check if the optimized parameters file exists
# if os.path.exists(params_file_path):
#     optimized_params = pd.read_excel(params_file_path)
# else:
#     params_columns = [
#         'Symbol', 'Interval', 'Start Date', 'End Date', 'Test Result',
#         'min_space', 'sloping_atr_length', 'stop_loss_multiplier',
#         'take_profit_multiplier', 'window_length'
#     ]
#     optimized_params = pd.DataFrame(columns=params_columns)

# # Check if the backtest results file exists
# if os.path.exists(results_file_path):
#     backtest_results = pd.read_excel(results_file_path)
# else:
#     backtest_results_columns = [
#         'Symbol', 'Interval', 'Total Trades', 'Total PNL', 'Percentage PNL (%)',
#         'Win Rate (%)', 'Profit Factor', 'Risk Ratio',
#         'Num Long Trades', 'Num Short Trades', 'Num Profitable Trades', 'Num Unprofitable Trades',
#         'Avg Profit', 'Avg Loss', 'Max Profit', 'Min Profit', 'Max Loss', 'Min Loss',
#         'Avg Long Duration (days)', 'Avg Long Duration (hours)', 'Avg Long Duration (minutes)', 'Avg Long Duration (seconds)',
#         'Avg Short Duration (days)', 'Avg Short Duration (hours)', 'Avg Short Duration (minutes)', 'Avg Short Duration (seconds)'
#     ]
#     backtest_results = pd.DataFrame(columns=backtest_results_columns)


### Imports

In [ ]:
!pip install pandas_ta
import pandas_ta as ta
!pip install backtesting
!pip install hyperopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218909 sha256=8f3fdc670500c2afe224c13a97273ff07625f73fe889be761628707da50ffba2
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.5/175.5 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173912 sha256=80392c048c0f6979304f0010df3afb998278db4e85b6b8fe3bab59f421a7a985
  Stored in directory: /root/.cache/pip/wheels/e2/30/7f/19cbe31987c6ebdb47f1f510343249066711609e3da2d57176
Successfully built backtesting


In [ ]:
# УСТАНОВКА TA-LIB В COLAB
# Устанавливаем предварительно скомпилированные файлы библиотеки TA-Lib
!pip install pandas==1.5.3
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib
!./configure --prefix=/usr
!make
!make install
%cd ..
!pip install ta-lib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 51.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pandas<2.2.3dev0,>=2.0, but you have pandas 1.5.3 which is incompatible.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 1.5.3 which is incompatible.
mizani 0.13.1 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
plotnine 0.14.4 requires pandas>=2.2.0, but you have pandas 1.5.3 which is incompatible.
xarray 2024.11.0 requires pandas>=2.1, but you have pandas 1.5.3 which is incompatible.


--2024-12-20 06:31:52--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 104.18.13.149, 104.18.12.149, 2606:4700::6812:d95, ...
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|104.18.13.149|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2024-12-20 06:31:52--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 104.18.12.149, 104.18.13.149, 2606:4700::6812:c95, ...
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://versaweb.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz?viasf=1 [following]
--2024-12-20 06:31:52--  http://versaweb.dl.sour

In [ ]:
!pip install pandas==2.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 93.0 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3


In [ ]:
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import os
import requests
import zipfile
import csv
import io
import backtesting
from backtesting.lib import crossover

/usr/local/lib/python3.10/dist-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


In [ ]:
# import pandas as pd

# # Создаем пустой DataFrame с для хранения данных
# columns = [
#     'Symbol', 'Interval', 'Total Trades', 'Total PNL', 'Percentage PNL (%)',
#     'Win Rate (%)', 'Profit Factor', 'Risk Ratio',
#     'Num Long Trades', 'Num Short Trades', 'Num Profitable Trades', 'Num Unprofitable Trades',
#     'Avg Profit', 'Avg Loss', 'Max Profit', 'Min Profit', 'Max Loss', 'Min Loss',
#     'Avg Long Duration (days)', 'Avg Long Duration (hours)', 'Avg Long Duration (minutes)', 'Avg Long Duration (seconds)',
#     'Avg Short Duration (days)', 'Avg Short Duration (hours)', 'Avg Short Duration (minutes)', 'Avg Short Duration (seconds)'
# ]


# sloping_backtest_results = pd.DataFrame(columns=columns)

### Strategy

#### Main

In [ ]:
# Сама стратегия

from collections import deque
import numpy as np
from dataclasses import dataclass
import talib

# датакласс для сигналов
@dataclass
class Signal:
    side: bool
    price: float
    atr: float
    line: tuple

# класс для поиска наклонок
class Sloping:
    length: int
    ts: deque
    open: deque
    high: deque
    low: deque
    close: deque
    body_up: deque
    body_down: deque

    def __init__(self, length, atr_length=14, min_space=5, max_len=500, debug=False):
        # инициализируем переменные
        self.length = length
        self.atr_length = atr_length
        self.min_space = min_space
        self.max_len = max_len
        self.debug = debug
        # счетчики сигналов
        self._last_signal_resistance = min_space + 1
        self._last_signal_support = min_space + 1
        # создаем очереди
        for key, value in self.__annotations__.items():
            if issubclass(value, deque):
                setattr(self, key, deque([.0], maxlen=self.max_len))

    # функция для добавления свечи
    def add_kline(self, ts, open, high, low, close):
        ts = int(ts)
        # проверяем не дублируется ли свеча
        if not self.ts or ts > self.ts[-1]:
            # заполняем очереди
            self.ts.append(ts)
            self.open.append(float(open))
            self.high.append(float(high))
            self.low.append(float(low))
            self.close.append(float(close))
            # определяем верх и низ тела свечи
            self.body_up.append(max(float(open), float(close)))
            self.body_down.append(min(float(open), float(close)))
            # возвращаем True если добавили свечу
            return True
        return False

    # функция для получения сигнала
    def get_value(self, support=True, resistance=True):
        # если не хватает количества свеч, то пропускаем этот сигнал
        if len(self.ts) < self.length + 1:
            return None
        for is_resistance, side in enumerate([support, resistance]):
            # если нам не нужно вычислять поддержу или сопротивление, то пропускаем
            if not side:
                continue
            # берем окно для поиска наклонки (необходимую нам часть тела свечи)
            window = np.array(list(self.body_up if is_resistance else self.body_down)[-self.length:])
            # генерируем индексы свечей
            x = np.arange(len(window))
            # вычисляем точки для поиска наклонки
            slope, intercept = np.polyfit(x, window, 1)
            points = slope * x + intercept
            # ищем минимум или максимум в пределах окна, в зависимости от того, что мы ищем
            value = (window - points).argmax() if is_resistance else (window - points).argmin()
            # получаем наклонную линию
            line = self._get_line(window, slope, value, is_resistance)
            # если произошел пробой сопротивления
            if is_resistance and (self.close[-1] > line[1] + self.length * line[0] or self.debug):
                # если сигнал был более min_space количества свечей назад
                if self._last_signal_resistance > self.min_space:
                    # обнуляем счетчик
                    self._last_signal_resistance = 0
                    # возвращаем сигнал
                    return Signal(True, self.close[-1], self.get_atr(), line)
                # обнуляем счетчик в любом случае
                self._last_signal_resistance = 0
            else:
                # если не было сигнала, то увеличиваем счетчик
                self._last_signal_resistance += 1
            # если произошел пробой поддержки
            if not is_resistance and (self.close[-1] < line[1] + self.length * line[0] or self.debug):
                # если сигнал был более min_space количества свечей назад
                if self._last_signal_support > self.min_space:
                    # обнуляем счетчик
                    self._last_signal_support = 0
                    # возвращаем сигнал
                    return Signal(False, self.close[-1], self.get_atr(), line)
                # обнуляем счетчик в любом случае
                self._last_signal_support = 0
            else:
                # если не было сигнала, то увеличиваем счетчик
                self._last_signal_support += 1

    # функция для поиска наклонной линии
    def _get_line(self, window, slope, value_index, is_resistance):
        # вычисляем шаг изменения наклона
        slope_step = (window.max() - window.min()) / len(window)
        # инициализируем начальные значения
        step = 1
        min_step = 0.0001
        best_slope = slope
        # вычисляем первоначальное значение тренда
        best_value = self._check_trend(window, slope, value_index, is_resistance)
        # флаг для вычисления производной
        get_der = True
        # переменная для хранения производной
        der = None
        # пока шаг не меньше min_step
        while step > min_step:
            # если нам нужно вычислять производную
            if get_der:
                # изменяем наклон на минимальный шаг
                slope_change = best_slope + slope_step * min_step
                # вычисляем значение еще раз
                test_value = self._check_trend(window, slope_change, value_index, is_resistance)
                # если значение меньше нуля, то меняем наклон
                if test_value < 0:
                    slope_change = best_slope - slope_step * min_step
                    # вычисляем значение еще раз
                    test_value = self._check_trend(window, slope_change, value_index, is_resistance)
                # запоминаем производную
                der = test_value - best_value
                get_der = False
            # в зависимости от знака производной меняем наклон
            if der > 0:
                test_slope = best_slope - slope_step * step
            else:
                test_slope = best_slope + slope_step * step
            # вычисляем значение
            test_value = self._check_trend(window, test_slope, value_index, is_resistance)
            # если значение меньше нуля или не улучшает лучшее значение, то уменьшаем шаг в 2 раза
            if test_value < 0 or test_value >= best_value:
                step *= 0.5
            else:
                # если значение улучшилось, то запоминаем наклон и значение
                best_slope = test_slope
                best_value = test_value
                get_der = True
        # возвращаем лучшее значение и пересечение с линей
        return best_slope, -best_slope * value_index + window[value_index]

    # функция для проверки тренда
    def _check_trend(self, window, slope, value_index, is_resistance):
        # вычисляем пересечение с линей
        intecept = -slope * value_index + window[value_index]
        # получаем все точки наклонки
        line_vals = slope * np.arange(len(window)) + intecept
        # вычисляем разницу между точками наклонки и телом свечей
        diffs = line_vals - window
        # проверяем линию на корректность
        if (not is_resistance and diffs.max() > 0) or (is_resistance and diffs.min() < 0):
            # если линия некорректна, то возвращаем -1
            return -1
        # возвращаем сумму квадратов разниц, которая является мерой отклонения линии тренда
        return (diffs ** 2).sum()

    # функция для получения ATR
    def get_atr(self):
        # вычисляем ATR
        return talib.ATR(np.array(self.high), np.array(self.low), np.array(self.close), self.atr_length)[-1]

    # вычисляем ATR с использованием pandas_ta
    # # Преобразуем deque в pandas.Series
    #   high_series = pd.Series(self.high)
    #   low_series = pd.Series(self.low)
    #   close_series = pd.Series(self.close)

    #   # Вычисляем ATR с использованием pandas_ta
    #   atr = ta.atr(high_series, low_series, close_series, length=self.atr_length)

    #   # Проверяем, если atr пуст или данных недостаточно
    #   if atr is None or len(atr) < self.atr_length:
    #       return 0  # или любое значение по умолчанию

    #   return atr.iloc[-1]




In [ ]:
# # Инициализируем индикатор
# sloping = Sloping(length=10, atr_length=14, min_space=5)  # Измените параметры по мере необходимости

# # Генерируем сигналы
# signals = []
# for index, row in df.iterrows():
#     ts = int(row.name.timestamp() * 1000)  # Время в миллисекундах
#     open = row['Open']
#     high = row['High']
#     low = row['Low']
#     close = row['Close']

#     if sloping.add_kline(ts, open, high, low, close):
#         signal = sloping.get_value()
#         if signal:
#             signals.append({
#                 'timestamp': index,
#                 'side': signal.side,
#                 'price': signal.price,
#                 'atr': signal.atr,
#                 'line': signal.line
#             })

# count = 0
# # Выводим полученные сигналы
# for signal in signals:

#     print(signal)
#     count += 1
# print(count)

In [ ]:
# # Инициализируем индикатор
# sloping = Sloping(length=10, atr_length=14, min_space=5)  # Измените параметры по мере необходимости

# # Генерируем сигналы
# signals_lib = []
# for index, row in df.iterrows():
#     ts = int(row.name.timestamp() * 1000)  # Время в миллисекундах
#     open = row['Open']
#     high = row['High']
#     low = row['Low']
#     close = row['Close']

#     if sloping.add_kline(ts, open, high, low, close):
#         signal = sloping.get_value()
#         if signal:
#             signals_lib.append({
#                 'timestamp': index,
#                 'side': signal.side,
#                 'price': signal.price,
#                 'atr': signal.atr,
#                 'line': signal.line
#             })

# count = 0
# # Выводим полученные сигналы
# for signal in signals_lib:

#     print(signal)
#     count += 1
# print(count)

#### Base Backtest

In [ ]:
from backtesting import Backtest, Strategy
import pandas as pd

# Define the strategy
class SlopingStrategyBasic(Strategy):
    window_length = 50
    min_space = 5
    sloping_atr_length = 14

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        # Добавить текущие данные к индикатору и получить сигнал
        self.slope_indicator.add_kline(
            self.data.index[-1].timestamp(),
            self.data.Open[-1],
            self.data.High[-1],
            self.data.Low[-1],
            self.data.Close[-1]
        )
        signal = self.slope_indicator.get_value()

        # Получаем время сигнала
        signal_time = self.data.index[-1]

        # Проверяем наличие сигнала и открытой позиции, исполняем ордер немедленно
        if signal and signal.side and not self.position.is_long:
            self.buy()

        elif signal and not signal.side and not self.position.is_short:
            self.sell()

                # Если сигнал на продажу, а позиция лонг
        elif signal and not signal.side and self.position.is_long:
            self.position.close()

        # Если сигнал на покупку, а позиция шорт
        elif signal and signal.side and self.position.is_short:
            self.position.close()

# # запускаем бэктест и выводим результат
# test = backtesting.Backtest(data=df, strategy=SlopingStrategyBasic, cash=1_000_000, commission=.0005)
# res = test.run()
# print(res)
# print(res._strategy)



# from hyperopt import fmin, tpe, hp, Trials
# from backtesting import Backtest

# # Целевая функция для оптимизации
# def objective(params):
#     window_length = int(params['window_length'])
#     min_space = int(params['min_space'])
#     sloping_atr_length = int(params['sloping_atr_length'])


#     # Создаем объект Backtest с настройками стратегии
#     bt = Backtest(data=df,
#                   strategy=SlopingStrategyBasic,
#                   cash=1_000_000,
#                   commission=0.0005)

#     # Запускаем бэктест
#     stats = bt.run(window_length=window_length,
#                    min_space=min_space)

#     # Возвращаем отрицательную прибыль для минимизации
#     return -stats['Return [%]']

# # Определяем пространство поиска
# space = {
#     'window_length': hp.quniform('window_length', 10, 100, 1),
#     'min_space': hp.quniform('min_space', 5, 20, 1),
#     'sloping_atr_length': hp.quniform('sloping_atr_length', 5, 30, 1)
# }

# # Используем TPE для оптимизации
# trials = Trials()
# best = fmin(objective,
#             space,
#             algo=tpe.suggest,
#             max_evals=5, # количество итераций
#             trials=trials
#             )

# print("Лучшие параметры:", best)



# # Можно продолжить поиск
# best = fmin(objective,
#             space,
#             algo=tpe.suggest,
#             max_evals=10, # количество иттераций для продолжения поиска
#             trials=trials
#             )

# print("Лучшие параметры:", best)


# # import time

# # start_time = time.time()

# # res = test.optimize(
# #     maximize='Return [%]',
# #     window_length=range(10, 100, 5),
# #     min_space=range(1, 20, 1),
# #     atr_length=range(1, 30, 1)
# # )

# # end_time = time.time()
# # print(f"Результаты оптимизации:\n{res}")
# # print(res._strategy)
# # print(f"Время выполнения: {end_time - start_time} секунд")



# # Применяем лучшие параметры к стратегии и запускаем окончательный бэктест

# optimized_length = int(best['window_length'])
# optimized_min_space = int(best['min_space'])
# optimized_sloping_atr_length = int(best['sloping_atr_length'])

# SlopingStrategyBasic.length = optimized_length
# SlopingStrategyBasic.min_space = optimized_min_space
# SlopingStrategyBasic.sloping_atr_length = optimized_sloping_atr_length

# # Запускаем бэктест с оптимизированными параметрами
# test = backtesting.Backtest(data=df, strategy=SlopingStrategyBasic, cash=1_000_000, commission=0.0005)
# res = test.run()
# print("Результаты с оптимизированными параметрами:", res)
# print("Лучшие параметры:", best)

#### Base+Take+Loss Backtest

In [ ]:
import backtesting
import pandas as pd


# Класс стратегии для backtesting
class SlopingStrategy(backtesting.Strategy):

    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5  # Мультиплеер для стоп-лосса
    take_profit_multiplier = 1.5  # Мультиплеер для тейк-профита
    trade_on_close = True # Add trade_on_close as a class variable
    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        # Добавляем текущие данные свечей в индикатор
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])

        # Получаем сигнал от индикатора
        signal = self.slope_indicator.get_value()


        # Если сигнал на покупку и нет позиции
        if signal and signal.side and not self.position:
          # Текущее значение ATR для текущей свечи
          current_atr = signal.atr
          try:
            stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
            take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
            self.buy(sl=stop_loss, tp=take_profit)
          except:
            pass

        # Если сигнал на продажу и нет позиции
        elif signal and not signal.side and not self.position:
          current_atr = signal.atr
          try:
            stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
            take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
            self.sell(sl=stop_loss, tp=take_profit)
          except:
            pass



# # Запускаем бэктест и выводим результат
# test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=.0005)
# res = test.run()
# print(res)

#### Base overturn

In [ ]:
from backtesting import Backtest, Strategy
import pandas as pd

class SlopingStrategyBasicOverturn(Strategy):
    window_length = 50
    min_space = 5
    sloping_atr_length = 14

    def init(self):
        # Инициализация индикатора наклона
        self.slope_indicator = Sloping(
            length=self.window_length,
            atr_length=self.sloping_atr_length,
            min_space=self.min_space
        )

    def next(self):
        # Обновляем данные индикатора
        self.slope_indicator.add_kline(
            self.data.index[-1].timestamp(),
            self.data.Open[-1],
            self.data.High[-1],
            self.data.Low[-1],
            self.data.Close[-1]
        )
        signal = self.slope_indicator.get_value()

        # Если есть сигнал, проверяем текущую позицию и принимаем действия
        if signal:
            if signal.side:  # Сигнал на покупку
                if self.position.is_short:
                    self.position.close()  # Закрыть шорт
                    self.buy()             # Открыть лонг
                elif not self.position.is_long:
                    self.buy()             # Если нет позиции, открыть лонг

            else:  # Сигнал на продажу
                if self.position.is_long:
                    self.position.close()  # Закрыть лонг
                    self.sell()            # Открыть шорт
                elif not self.position.is_short:
                    self.sell()            # Если нет позиции, открыть шорт


#### Base+Take+Loss+Overturn

In [ ]:
import backtesting
import pandas as pd


# Класс стратегии для backtesting
class SlopingStrategyOverturn(backtesting.Strategy):

    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5  # Мультиплеер для стоп-лосса
    take_profit_multiplier = 1.5  # Мультиплеер для тейк-профита

    def init(self):
        self.slope_indicator = Sloping(
            length=self.window_length,
            atr_length=self.sloping_atr_length,
            min_space=self.min_space
        )

    def next(self):
        # Обновляем индикатор с новыми данными свечей
        self.slope_indicator.add_kline(
            self.data.index[-1].timestamp(),
            self.data.Open[-1],
            self.data.High[-1],
            self.data.Low[-1],
            self.data.Close[-1]
        )

        # Получаем текущий сигнал
        signal = self.slope_indicator.get_value()

        if signal:
            current_atr = signal.atr
            close_price = self.data.Close[-1]

            if signal.side:  # Сигнал на покупку
                # Если открыта шортовая позиция, закрыть её и открыть лонг
                if self.position.is_short:
                    self.position.close()
                    stop_loss = close_price - (self.stop_loss_multiplier * current_atr)
                    take_profit = close_price + (self.take_profit_multiplier * current_atr)
                    self.buy(sl=stop_loss, tp=take_profit)
                elif not self.position:  # Если позиции нет, открыть лонг
                    stop_loss = close_price - (self.stop_loss_multiplier * current_atr)
                    take_profit = close_price + (self.take_profit_multiplier * current_atr)
                    self.buy(sl=stop_loss, tp=take_profit)

            else:  # Сигнал на продажу
                # Если открыта лонговая позиция, закрыть её и открыть шорт
                if self.position.is_long:
                    self.position.close()
                    stop_loss = close_price + (self.stop_loss_multiplier * current_atr)
                    take_profit = close_price - (self.take_profit_multiplier * current_atr)
                    self.sell(sl=stop_loss, tp=take_profit)
                elif not self.position:  # Если позиции нет, открыть шорт
                    stop_loss = close_price + (self.stop_loss_multiplier * current_atr)
                    take_profit = close_price - (self.take_profit_multiplier * current_atr)
                    self.sell(sl=stop_loss, tp=take_profit)



#### SlopingStrategyCloseOnOppositeSignal

In [ ]:
import backtesting
import pandas as pd


# Класс стратегии для backtesting
class SlopingStrategyCloseOnOppositeSignal(backtesting.Strategy):

    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5  # Мультиплеер для стоп-лосса
    take_profit_multiplier = 1.5  # Мультиплеер для тейк-профита

    def init(self):
        self.slope_indicator = Sloping(
            length=self.window_length,
            atr_length=self.sloping_atr_length,
            min_space=self.min_space
        )

    def next(self):
        # Обновляем индикатор с новыми данными свечей
        self.slope_indicator.add_kline(
            self.data.index[-1].timestamp(),
            self.data.Open[-1],
            self.data.High[-1],
            self.data.Low[-1],
            self.data.Close[-1]
        )

        # Получаем текущий сигнал
        signal = self.slope_indicator.get_value()

        if signal:
            if signal.side:  # Сигнал на покупку
                # Закрыть текущую шортовую позицию, если есть
                if self.position.is_short:
                    self.position.close()

            else:  # Сигнал на продажу
                # Закрыть текущую лонговую позицию, если есть
                if self.position.is_long:
                    self.position.close()

            # Если позиции нет, ничего не делаем
            if not self.position:
                current_atr = signal.atr
                close_price = self.data.Close[-1]

                if signal.side:  # Открываем лонг по новому сигналу
                    stop_loss = close_price - (self.stop_loss_multiplier * current_atr)
                    take_profit = close_price + (self.take_profit_multiplier * current_atr)
                    self.buy(sl=stop_loss, tp=take_profit)
                else:  # Открываем шорт по новому сигналу
                    stop_loss = close_price + (self.stop_loss_multiplier * current_atr)
                    take_profit = close_price - (self.take_profit_multiplier * current_atr)
                    self.sell(sl=stop_loss, tp=take_profit)


### Functions

#### Get data and gnerate klines

In [ ]:
# def generate_months(start_date, end_date):
#     """Генерирует список месяцев в заданном интервале."""
#     start_date = datetime.strptime(start_date, '%Y-%m')
#     end_date = datetime.strptime(end_date, '%Y-%m')
#     months = []

#     while start_date <= end_date:
#         months.append(start_date.strftime('%Y-%m'))
#         start_date = start_date + relativedelta(months=1)

#     return months

# def download_klines(symbol, interval, start_date, end_date):
#     """Скачивает исторические данные свечей с Binance и возвращает DataFrame."""
#     # Генерируем месяцы
#     months = generate_months(start_date, end_date)

#     # Создаем папки, если их нет
#     if not os.path.exists('klines'):
#         os.mkdir('klines')

#     klines = {
#         'Date': [],
#         'Open': [],
#         'High': [],
#         'Low': [],
#         'Close': [],
#         'Volume': []
#     }

#     for month in months:
#         filename = f"{symbol}-{interval}-{month}.zip"

#         # Если нет klines, то скачиваем
#         if not os.path.exists(f"klines/{filename}"):
#             url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
#             r = requests.get(url, allow_redirects=True)
#             open(f"klines/{filename}", 'wb').write(r.content)

#         # Открываем zip
#         with zipfile.ZipFile(f"klines/{filename}", 'r') as zip_file:
#             # Читаем csv внутри zip
#             with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
#                 csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
#                 for row in csv_reader:
#                     # Если строка содержит число (отбрасываем первую строку)
#                     if row[0].isdigit():
#                         # Заполняем списки с данными свечи
#                         klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
#                         klines['Open'].append(float(row[1]))
#                         klines['High'].append(float(row[2]))
#                         klines['Low'].append(float(row[3]))
#                         klines['Close'].append(float(row[4]))
#                         klines['Volume'].append(float(row[5]))

#     # Создаем DataFrame
#     df = pd.DataFrame(klines)
#     df['Date'] = pd.to_datetime(df['Date'])  # Конвертируем столбец 'Date' в datetime
#     df.set_index('Date', inplace=True)

#     return df

In [ ]:
# import os
# import requests
# import zipfile
# import csv
# import io
# import pandas as pd
# from datetime import datetime, timezone
# from dateutil.relativedelta import relativedelta
# from backtesting import Backtest

# # Генерация списка месяцев
# def generate_months(start_date, end_date):
#     start_date = datetime.strptime(start_date, '%Y-%m')
#     end_date = datetime.strptime(end_date, '%Y-%m')
#     months = []
#     while start_date <= end_date:
#         months.append(start_date.strftime('%Y-%m'))
#         start_date = start_date + relativedelta(months=1)
#     return months

# # Загрузка исторических данных с Binance
# def download_klines(symbol, interval, start_date, end_date):
#     months = generate_months(start_date, end_date)

#     if not os.path.exists('klines'):
#         os.mkdir('klines')

#     klines = {
#         'Date': [],
#         'Open': [],
#         'High': [],
#         'Low': [],
#         'Close': [],
#         'Volume': []
#     }

#     for month in months:
#         filename = f"{symbol}-{interval}-{month}.zip"
#         if not os.path.exists(f"klines/{filename}"):
#             url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
#             r = requests.get(url, allow_redirects=True)
#             open(f"klines/{filename}", 'wb').write(r.content)

#         with zipfile.ZipFile(f"klines/{filename}", 'r') as zip_file:
#             with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
#                 csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
#                 for row in csv_reader:
#                     if row[0].isdigit():
#                         klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
#                         klines['Open'].append(float(row[1]))
#                         klines['High'].append(float(row[2]))
#                         klines['Low'].append(float(row[3]))
#                         klines['Close'].append(float(row[4]))
#                         klines['Volume'].append(float(row[5]))

#     df = pd.DataFrame(klines)
#     df['Date'] = pd.to_datetime(df['Date'])
#     df.set_index('Date', inplace=True)
#     return df

# # Автоматический подбор стартовой даты
# def get_available_data(symbol, interval, start_date, end_date):
#     current_start_date = start_date
#     while True:
#         try:
#             data = download_klines(symbol, interval, current_start_date, end_date)
#             if not data.empty:
#                 return data
#         except Exception:
#             pass  # Пропускаем месяцы, если данные отсутствуют
#         current_start_date = (datetime.strptime(current_start_date, '%Y-%m') + relativedelta(months=1)).strftime('%Y-%m')
#         if datetime.strptime(current_start_date, '%Y-%m') > datetime.strptime(end_date, '%Y-%m'):
#             break
#     return pd.DataFrame()  # Возвращаем пустой DataFrame, если данные отсутствуют

# # Бэктестинг
# def run_backtest(symbols, strategy_class, parameters, start_date, end_date, interval='1d', initial_cash=1_000_000, commission=0.0005):
#     results = {}
#     for symbol in symbols:
#         print(f"Обработка {symbol}...")
#         data = get_available_data(symbol, interval, start_date, end_date)
#         if data.empty:
#             print(f"Нет доступных данных для {symbol}. Пропуск.")
#             continue

#         strategy_class.window_length = parameters[symbol]['window_length']
#         strategy_class.min_space = parameters[symbol]['min_space']
#         strategy_class.sloping_atr_length = parameters[symbol]['sloping_atr_length']
#         strategy_class.take_profit_multiplier = parameters[symbol]['take_profit_multiplier']
#         strategy_class.stop_loss_multiplier = parameters[symbol]['stop_loss_multiplier']

#         print(f"Запуск стратегии для {symbol}...")
#         test = Backtest(data, strategy_class, cash=initial_cash, commission=commission)
#         res = test.run()
#         results[symbol] = res

#         print(f"Результаты для {symbol}:")
#         print(res)
#     return results


In [ ]:
# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    return df

#### Save data

In [ ]:
import pandas as pd
import os
import shutil

def save_results_to_csv(df, shared_folder_path, file_name):
    csv_file_path = os.path.join(shared_folder_path, file_name)
    temp_file_path = f"{csv_file_path}.temp"

    # Проверка существования файла
    if os.path.exists(csv_file_path):
        # Загружаем текущие данные из файла
        existing_df = pd.read_csv(csv_file_path)

        # Конкатенируем текущие данные с новыми
        combined_df = pd.concat([existing_df, df], ignore_index=True)

        # Удаляем полностью дублирующиеся строки
        combined_df = combined_df.drop_duplicates(keep='first').reset_index(drop=True)
    else:
        print(f"Файл не существует, будет создан новый файл: {csv_file_path}")
        combined_df = df

    # Сохраняем обновленные данные во временный файл
    combined_df.to_csv(temp_file_path, index=False)

    # Резервное копирование старого файла перед заменой
    if os.path.exists(csv_file_path):
        backup_file_path = f"{csv_file_path}.backup"
        shutil.copy(csv_file_path, backup_file_path)
        print(f"Резервная копия создана: {backup_file_path}")

    # Перемещаем временный файл, заменяя старый файл
    os.replace(temp_file_path, csv_file_path)
    print(f"Обновленные данные сохранены в {csv_file_path}")



In [ ]:
import os
import csv
import pandas as pd

# # Функция для сохранения результатов в CSV
# def save_optimized_results_to_csv(optimized_params, path):
#     optimized_params.to_csv(f'{path}/optimized_params.csv', index=False)
#     print("Results saved to CSV.")

# Из-за того, что приходится откатывать pandas для установки ta-lib, может возникать ошибка при сохранении DataFrame в CSV, для решения можно использовать данную функцию

# def save_optimized_results_to_csv(optimized_params, path, file_name):
#     csv_file_path = f'{path}/{file_name}'
#     # Use the built-in CSV library to write to the CSV file
#     with open(csv_file_path, mode='w', newline='') as file:
#         writer = csv.writer(file)
#         # Write header
#         writer.writerow(optimized_params.columns)
#         # Write data
#         for index, row in optimized_params.iterrows():
#             writer.writerow(row)
#     print("Results saved to CSV.")


# import csv

# # Функция для сохранения параметров в CSV с использованием модуля csv
# def save_optimized_results_to_csv(optimized_params, path, file_name):
#     csv_file_path = f'{path}/{file_name}'

#     # Проверяем, существует ли файл
#     file_exists = os.path.isfile(csv_file_path)

#     # Открываем файл в режиме добавления (append), если он существует, иначе в режиме записи (write)
#     with open(csv_file_path, mode='a' if file_exists else 'w', newline='') as file:
#         writer = csv.writer(file)

#         # Записываем заголовки, если файл только что создан
#         if not file_exists:
#             writer.writerow(optimized_params.columns)

#         # Записываем строки из DataFrame
#         for index, row in optimized_params.iterrows():
#             writer.writerow(row)
#     print("Results saved to CSV.")

# Теперь вызов этой функции сохранит данные независимо от версии pandas


In [ ]:
import pandas as pd
import os
import shutil

def update_csv_with_temp_backup(df, csv_file_path):
    temp_file_path = f"{csv_file_path}.temp"

    # Проверка существования файла
    if os.path.exists(csv_file_path):
        # Загружаем текущие данные из файла
        existing_df = pd.read_csv(csv_file_path)

        # Конкатенируем текущие данные с новыми
        combined_df = pd.concat([existing_df, df], ignore_index=True)

        # Удаляем полностью дублирующиеся строки
        combined_df = combined_df.drop_duplicates(keep='first').reset_index(drop=True)
    else:
        print(f"Файл не существует, будет создан новый файл: {csv_file_path}")
        combined_df = df

    # Сохраняем обновленные данные во временный файл
    combined_df.to_csv(temp_file_path, index=False)

    # Резервное копирование старого файла перед заменой
    if os.path.exists(csv_file_path):
        backup_file_path = f"{csv_file_path}.backup"
        shutil.copy(csv_file_path, backup_file_path)
        print(f"Резервная копия создана: {backup_file_path}")

    # Перемещаем временный файл, заменяя старый файл
    os.replace(temp_file_path, csv_file_path)
    print(f"Обновленные данные сохранены в {csv_file_path}")

# Пример использования функции
# Подготовка новых данных для добавления в optimized_params.csv или backtest_results.csv


In [ ]:
# Функция для создания записи параметров
def create_optimized_params_row(best_params, best_loss, symbol, interval, start_date, end_date):
    return {
        'Symbol': symbol,
        'Interval': interval,
        'Start Date': start_date,
        'End Date': end_date,
        'Test Return [%]': best_loss,
        'window_length': best_params['window_length'],
        # 'min_space': best_params['min_space'],
        'sloping_atr_length': best_params['sloping_atr_length'],
        'stop_loss_multiplier': best_params['stop_loss_multiplier'],
        'take_profit_multiplier': best_params['take_profit_multiplier']
    }

#### Optimization

In [ ]:
from hyperopt import fmin, tpe, hp, Trials

def hyperopt_optimizer(data, symbol, interval, start_date, end_date, auto_mode=True, max_evals=5, next_evals=3, shared_folder_path=shared_folder_path):
    global optimized_params
    auto_mode = auto_mode
    attempts = 0
    best_loss = float('inf')
    file_name = 'optimized_params.csv'


    def objective(params):
        window_length = int(params['window_length'])
        min_space = int(params['min_space'])
        sloping_atr_length = int(params['sloping_atr_length'])
        take_profit_multiplier = params['take_profit_multiplier']
        stop_loss_multiplier = params['stop_loss_multiplier']

        bt = backtesting.Backtest(data=data,
                      strategy=SlopingStrategy,
                      cash=1_000_000,
                      commission=0.0005)

        stats = bt.run(
            window_length=window_length,
            # min_space=min_space,
            sloping_atr_length=sloping_atr_length,
            take_profit_multiplier=take_profit_multiplier,
            stop_loss_multiplier=stop_loss_multiplier)

        return -stats['Return [%]']

    space = {
        'window_length': hp.quniform('window_length', 10, 100, 2),
        'min_space': hp.quniform('min_space', 3, 20, 1),
        'sloping_atr_length': hp.quniform('sloping_atr_length', 5, 50, 1),
        'take_profit_multiplier': hp.uniform('take_profit_multiplier', 1.0, 2.5),
        'stop_loss_multiplier': hp.uniform('stop_loss_multiplier', 1.0, 2.5)
    }

    trials = Trials()

    if auto_mode:
      while attempts < 3:
          # Запуск оптимизации
          best_params = fmin(objective,
                      space,
                      algo=tpe.suggest,
                      max_evals=max_evals,
                      trials=trials
                      )

          # Обновление текущей наименьшей убыточности
          current_loss = min(trial['result']['loss'] for trial in trials.trials)
          print("Лучшие параметры:", best_params)


          # Сохраняем данные
          new_row = create_optimized_params_row(best_params, -current_loss, symbol, interval, start_date, end_date)
          # Ensure new_row is not empty before concatenation
          if any(new_row.values()):
              optimized_params = pd.concat([optimized_params, pd.DataFrame([new_row])], ignore_index=True)
          save_results_to_csv(optimized_params, shared_folder_path, file_name)


          # Проверка, нужно ли повторить попытку или завершить оптимизацию
          if current_loss > 50: # если более 50 (50% убыток), то скорее всего мы так и не преблизимся к профиту, лучше заново инициализировать поиск
              attempts += 1
              print(f'Текущий Return [%] {-current_loss}. Сликшом далеко до профита')
              trials = Trials()  # Очищаем trials для новой попытки
              continue

          else:
              # Успешная попытка, продолжаем оптимизацию
              while current_loss < best_loss:
                  best_loss = current_loss

                  best_params = fmin(objective,
                              space,
                              algo=tpe.suggest,
                              max_evals=max_evals + next_evals,
                              trials=trials
                              )

                  max_evals += next_evals
                  current_loss = min(trial['result']['loss'] for trial in trials.trials)
                  print("Лучшие параметры:", best_params)

                  # Сохраняем данные
                  if current_loss < best_loss:
                    new_row = create_optimized_params_row(best_params, -current_loss, symbol, interval, start_date, end_date)
                    # Ensure new_row is not empty before concatenation
                    if any(new_row.values()):
                        optimized_params = pd.concat([optimized_params, pd.DataFrame([new_row])], ignore_index=True)
                    save_results_to_csv(optimized_params, shared_folder_path, file_name)

              # Завершаем цикл, если не достигнуты минимальные потери
              attempts = 3

    # Вручную задаём количество итераций
    else:
        while True:
            best_params = fmin(objective,
                        space,
                        algo=tpe.suggest,
                        max_evals=max_evals,
                        trials=trials)

            current_loss = min(trial['result']['loss'] for trial in trials.trials)
            print("Лучшие параметры:", best_params)

            new_row = create_optimized_params_row(best_params, -current_loss, symbol, interval, start_date, end_date)
            if any(new_row.values()):
                optimized_params = pd.concat([optimized_params, pd.DataFrame([new_row])], ignore_index=True)
            save_results_to_csv(optimized_params, shared_folder_path, file_name)


            user_input = input("Продолжить поиск? Введите количество дополнительных итераций или 'no', 0 для завершения: ").strip()
            if user_input.lower() in ['no', 'n', '0']:
                print("Manual optimization ended.")
                break

            try:
                continuation_rounds = int(user_input)
                if continuation_rounds <= 0:
                    print("Завершаем подбор параметров.")
                    break

                # Продолжаем оптимизацию
                else:
                  best_loss = current_loss
                  best_params = fmin(objective,
                              space,
                              algo=tpe.suggest,
                              max_evals=max_evals + continuation_rounds,
                              trials=trials)
                  max_evals += continuation_rounds
                  current_loss = min(trial['result']['loss'] for trial in trials.trials)
                  print("Лучшие параметры:", best_params)
                  # Сохраняем параметры
                  if current_loss < best_loss:
                    new_row = create_optimized_params_row(best_params, -current_loss, symbol, interval, start_date, end_date)
                    if any(new_row.values()):
                        optimized_params = pd.concat([optimized_params, pd.DataFrame([new_row])], ignore_index=True)
                    save_results_to_csv(optimized_params, shared_folder_path, file_name)


            except ValueError:
                print("Вы неверно ввели количество итераций. Попробуйте сново")

    return best_params

#### Get Backtest Data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

initial_capital = 1_000_000

def get_backtest_data(trades, initial_capital=initial_capital):
    # Разделение сделок на лонг и шорт
    long_trades = trades[trades['Size'] > 0].copy()
    short_trades = trades[trades['Size'] < 0].copy()
    num_long_trades = len(long_trades)
    num_short_trades = len(short_trades)

    # Подсчет прибыльных и убыточных сделок
    profitable_trades = trades[trades['PnL'] > 0]
    unprofitable_trades = trades[trades['PnL'] <= 0]
    num_profitable_trades = len(profitable_trades)
    num_unprofitable_trades = len(unprofitable_trades)

    # Средняя прибыль и убыток
    avg_profit = profitable_trades['PnL'].mean() if num_profitable_trades > 0 else 0
    avg_loss = unprofitable_trades['PnL'].mean() if num_unprofitable_trades > 0 else 0

    # Средняя продолжительность сделок в лонг и шорт
    avg_long_duration = (long_trades['ExitTime'] - long_trades['EntryTime']).mean() if num_long_trades > 0 else pd.Timedelta(0)
    avg_short_duration = (short_trades['ExitTime'] - short_trades['EntryTime']).mean() if num_short_trades > 0 else pd.Timedelta(0)

    # Общая прибыль/убыток и процентное изменение от начального капитала
    total_profit = trades['PnL'].sum()
    percentage_profit_loss = (total_profit / initial_capital) * 100

    # Соотношение риска и интерпретация
    risk_ratio = abs(avg_loss / avg_profit) if avg_profit != 0 else float('inf')

    # Метрики продолжительности сделок
    avg_long_days, avg_long_hours, avg_long_minutes, avg_long_seconds = avg_long_duration.components[:4]
    avg_short_days, avg_short_hours, avg_short_minutes, avg_short_seconds = avg_short_duration.components[:4]

    # Дополнительные метрики
    max_profit = profitable_trades['PnL'].max() if num_profitable_trades > 0 else 0
    min_profit = profitable_trades['PnL'].min() if num_profitable_trades > 0 else 0
    max_loss = unprofitable_trades['PnL'].min() if num_unprofitable_trades > 0 else 0
    min_loss = unprofitable_trades['PnL'].max() if num_unprofitable_trades > 0 else 0
    win_rate = (num_profitable_trades / len(trades)) * 100 if len(trades) > 0 else 0
    profit_factor = profitable_trades['PnL'].sum() / abs(unprofitable_trades['PnL'].sum()) if num_unprofitable_trades > 0 else float('inf')

    # Вывод всех результатов
    print("Результаты backtest:")
    print(f"Total Trades: {len(trades)}")
    print(f"Total PNL: {total_profit}")
    print(f"Percentage PNL (%): {percentage_profit_loss:.2f}")
    print(f"Win Rate (%): {win_rate:.2f}")
    print(f"Profit Factor: {profit_factor:.2f}")
    print(f"Risk Ratio: {risk_ratio:.2f}")
    print(f"Num Long Trades: {num_long_trades}")
    print(f"Num Short Trades: {num_short_trades}")
    print(f"Num Profitable Trades: {num_profitable_trades}")
    print(f"Num Unprofitable Trades: {num_unprofitable_trades}")
    print(f"Avg Profit: {avg_profit:.2f}")
    print(f"Avg Loss: {avg_loss:.2f}")
    print(f"Max Profit: {max_profit}")
    print(f"Min Profit: {min_profit}")
    print(f"Max Loss: {max_loss}")
    print(f"Min Loss: {min_loss}")
    print(f"Avg Long Duration {avg_long_days} days, {avg_long_hours} hours, {avg_long_minutes} minutes, {avg_long_seconds} seconds")
    print(f"Avg Short Duration {avg_short_days} days, {avg_short_hours} hours, {avg_short_minutes} minutes, {avg_short_seconds} seconds")

    # Возвращение результатов в виде словаря
    return {
        'Total Trades': len(trades),
        'Total PNL': total_profit,
        'Percentage PNL (%)': percentage_profit_loss,
        'Win Rate (%)': win_rate,
        'Profit Factor': profit_factor,
        'Risk Ratio': risk_ratio,
        'Num Long Trades': num_long_trades,
        'Num Short Trades': num_short_trades,
        'Num Profitable Trades': num_profitable_trades,
        'Num Unprofitable Trades': num_unprofitable_trades,
        'Avg Profit': avg_profit,
        'Avg Loss': avg_loss,
        'Max Profit': max_profit,
        'Min Profit': min_profit,
        'Max Loss': max_loss,
        'Min Loss': min_loss,
        'Avg Long Duration (days)': avg_long_days,
        'Avg Long Duration (hours)': avg_long_hours,
        'Avg Long Duration (minutes)': avg_long_minutes,
        'Avg Long Duration (seconds)': avg_long_seconds,
        'Avg Short Duration (days)': avg_short_days,
        'Avg Short Duration (hours)': avg_short_hours,
        'Avg Short Duration (minutes)': avg_short_minutes,
        'Avg Short Duration (seconds)': avg_short_seconds
    }


# Пример вызова функции
# res должен быть объектом с данными о сделках
# Пример:
# result_table = analyze_trades(res, initial_capital=1_000_000, symbol='BTCUSDT', interval='1h')
# print(result_table)


#### Plot Deposit and Price Changes

In [ ]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.io as pio


initial_capital = 1_000_000

def plot_deposit_and_price_change(trades, df, symbol, interval, basis, initial_capital=initial_capital):
    # Изменения депозита
    deposit_changes = [initial_capital]
    for trade in trades.itertuples():
        deposit_changes.append(deposit_changes[-1] + trade.PnL)

    # Находим начало и конец датасета
    start_date = min(df.index.min(), trades['EntryTime'].min())
    end_date = max(df.index.max(), trades['ExitTime'].max())


    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                        subplot_titles=f"Изменение депозита, {symbol}")

    # Изменение депозита
    fig.add_trace(go.Scatter(
        x=trades['ExitTime'],
        y=deposit_changes[1:],  # Skip initial capital for plotting
        mode='lines+markers',
        name='Изменение депозита',
        line=dict(color='blue'),
        hovertemplate='%{x}: Депозит: %{y:.2f}<br>Профит/Убыток: %{text}',
        text=[f'Профит: {trade.PnL:.2f}' if trade.PnL > 0 else f'Убыток: {trade.PnL:.2f}' for trade in trades.itertuples()]
    ), row=1, col=1)

    # Цена BTC
    fig.add_trace(go.Scatter(
        x=df.index,
        y=df['Close'],
        mode='lines',
        name='Цена BTC',
        line=dict(color='orange'),
        hovertemplate='Дата: %{x}<br>Цена: %{y:.2f}'
    ), row=2, col=1)

    # добавляем маркеры для сделок
    for trade in trades.itertuples():
        # Макрерком для изменения депозита является выход из сделки (ExitTime)
        marker_color = 'green' if trade.PnL > 0 else 'red'
        fig.add_trace(go.Scatter(
            x=[trade.ExitTime],
            y=[deposit_changes[trade.Index + 1]],
            mode='markers',
            marker=dict(color=marker_color, size=8, symbol='circle'),
            name='Профит' if trade.PnL > 0 else 'Убыток',
            hovertemplate=f'Сделка №{trade.Index + 1}: Профит: {trade.PnL:.2f}'
        ), row=1, col=1)

        # Стрелка для графика о движении цены, задаем в точке входа (EntryTime)
        arrow_symbol = '↑' if trade.Size > 0 else '↓'
        arrow_color = 'green' if trade.Size > 0 else 'red'
        fig.add_annotation(
            x=trade.EntryTime,
            y=df.loc[trade.EntryTime, 'Close'],
            text=arrow_symbol,
            showarrow=True,
            arrowhead=2,
            ax=0,
            ay=-30,
            font=dict(size=20, color=arrow_color),
            hovertext=f'Сделка №{trade.Index + 1} ({"Лонг" if trade.Size > 0 else "Шорт"})',
            hoverlabel=dict(bgcolor='white', bordercolor='black'),
            xref="x",
            yref="y2"
        )

    fig.update_layout(
        title=f'Изменение депозита и цены {symbol} ({interval})',
        xaxis=dict(title='Дата', range=[start_date, end_date]),
        xaxis2=dict(title='Дата', range=[start_date, end_date]),
        yaxis=dict(title='Депозит ($)'),
        yaxis2=dict(title=f'Цена {symbol} ($)'),
        legend=dict(x=0, y=1),
        height=1200,
        width=5000
    )

    fig.update_xaxes(tickangle=-45)
    fig.update_layout(xaxis_rangeslider_visible=False)

    fig.show()

    # Сохраняем гарфики в HTML file
    filename = f"plot_{symbol}_{interval}_{start_date.strftime('%Y-%m-%d')}_{end_date.strftime('%Y-%m-%d')}_basis.html"
    pio.write_html(fig, file=filename)
    print(f"График сохранен как {filename}")




# Пример вызова функции
# plot_deposit_and_price_changes(trades, df, symbol, interval)


#### Backtest Pipeline

In [ ]:
# pipeline для финального бэктеста
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta
import shutil

# Фильтр для отбора моделей с положительным PnL на заданном интервале
def filter_positive_pnl(df, symbol, interval):
    df.columns = [col.lower().replace(" ", "_") for col in df.columns]
    try:
        filtered_df = df[(df['symbol'] == symbol) & (df['interval'] == interval) & (df['total_pnl'] > 0)]
    except KeyError:
        filtered_df = df[(df['symbol'] == symbol) & (df['interval'] == interval) & (df['test_return_[%]'] > 0)]
    # Reset index and return the DataFrame
    return filtered_df.reset_index(drop=True)

# Генерация точных месячных интервалов
def generate_monthly_intervals(start_date, end_date, interval_count=12):
    intervals = []
    current_date = datetime.strptime(start_date, "%Y-%m")
    end_date = datetime.strptime(end_date, "%Y-%m")

    while current_date < end_date and len(intervals) < interval_count:
        next_date = current_date + relativedelta(months=1)
        intervals.append((current_date.strftime("%Y-%m"), next_date.strftime("%Y-%m")))
        current_date = next_date

    return intervals

# Основная функция бэктестинга с фильтрацией при загрузке
def pipeline(df_backtest_results, symbol, interval, start_date, end_date, strategy, initial_capital=1_000_000, interval_count=12):
    # Фильтрация моделей по профитности
    filtered_models = filter_positive_pnl(df_backtest_results, symbol=symbol, interval=interval)
    if filtered_models.empty:
        print("Нет профитных моделей для данного символа и интервала.")
        return pd.DataFrame()

    intervals = generate_monthly_intervals(start_date, end_date, interval_count)
    pnl_columns = [f"{start} - {end}" for start, end in intervals]

    results_df = pd.DataFrame(columns=[
        'model_name', 'symbol', 'interval', 'window_length', 'atr_length',
        'min_space', 'stop_loss_multiplier', 'take_profit_multiplier'
    ] + pnl_columns + ['total_pnl%', 'profit_month_count', 'loss_month_count'])

    # Загрузка свечей для каждого интервала
    monthly_data = {start: download_klines(symbol, interval, start, end) for start, end in intervals}

    # Бэктест для каждой отфильтрованной модели
    for model_index, row in filtered_models.iterrows():
        total_pnl = 0
        profit_month_count = 0
        loss_month_count = 0

        model_name = f"Model_{model_index+1}_{row['window_length']}.{row['sloping_atr_length']}."\
                     f"{int(row['min_space'] if not pd.isna(row['min_space']) else 5)}.{round(row['stop_loss_multiplier'], 2)}."\
                     f"{round(row['take_profit_multiplier'], 2)}"

        # Заполнение информации о модели
        results_df.loc[model_index, ['model_name', 'symbol', 'interval', 'window_length',
                                     'atr_length', 'min_space', 'stop_loss_multiplier',
                                     'take_profit_multiplier']] = [
            model_name, symbol, interval, row['window_length'], row['sloping_atr_length'],
            int(row['min_space'] if not pd.isna(row['min_space']) else 5), row['stop_loss_multiplier'], row['take_profit_multiplier']
        ]

        # Бэктест по каждому месяцу для текущей модели
        for (start, end), pnl_col in zip(intervals, pnl_columns):
            data = monthly_data[start]

            # Настройка параметров стратегии
            strategy.length = row['window_length']
            strategy.atr_length = row['sloping_atr_length']
            strategy.min_space = int(row['min_space'] if not pd.isna(row['min_space']) else 5)
            strategy.stop_loss_multiplier = row['stop_loss_multiplier']
            strategy.take_profit_multiplier = row['take_profit_multiplier']

            # Запуск бэктеста
            test = backtesting.Backtest(data=data, strategy=strategy, cash=initial_capital, commission=0.0005)
            res = test.run()
            pnl_month = round((res._trades['PnL'].sum() / initial_capital) * 100, 2)

            results_df.at[model_index, pnl_col] = pnl_month
            total_pnl += pnl_month

            # Подсчет профитных и убыточных месяцев
            if pnl_month > 0:
                profit_month_count += 1
            else:
                loss_month_count += 1

            # Промежуточный вывод результатов по текущей модели и интервалу
            print(f"Модель {model_name}, Интервал {start} - {end}: PnL={pnl_month}")

        # Итоговые результаты для модели
        results_df.loc[model_index, 'total_pnl%'] = total_pnl
        results_df.loc[model_index, 'profit_month_count'] = profit_month_count
        results_df.loc[model_index, 'loss_month_count'] = loss_month_count

        # Промежуточный вывод итогов по модели
        print(f"Обработана модель {model_name}: Total PnL%={total_pnl}, Profitable Months={profit_month_count}, Loss Months={loss_month_count}")

    return results_df

#### Hyperopt_by_month

In [ ]:
import pandas as pd
from datetime import datetime
from hyperopt import fmin, tpe, hp, Trials
import backtesting

# Генерация интервалов
def generate_monthly_intervals(start_date, end_date):
    start = datetime.strptime(start_date, "%Y-%m")
    end = datetime.strptime(end_date, "%Y-%m")
    intervals = []
    while start <= end:
        next_month = datetime(start.year + (start.month // 12), (start.month % 12) + 1, 1)
        intervals.append((start.strftime("%Y-%m"), next_month.strftime("%Y-%m")))
        start = next_month
    return intervals

# Анализ помесячно
def analyze_monthly_results(trades, intervals, initial_capital):
    trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
    trades.set_index('Close Time', inplace=True)
    trades['Trade Duration'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds()

    monthly_results = {}
    total_pnl = 0
    profit_months = 0
    loss_months = 0
    total_trade_duration = 0
    total_trades = len(trades)

    for start, end in intervals:
        start_date = datetime.strptime(start, "%Y-%m")
        end_date = datetime.strptime(end, "%Y-%m")

        # Фильтруем сделки для текущего месяца
        monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

        if monthly_trades.empty:
            monthly_results[start] = (0.0, 0)
            continue

        # Вычисляем PnL за месяц
        monthly_pnl = monthly_trades['PnL'].sum() / initial_capital * 100

        # Среднее время сделки
        avg_trade_duration = monthly_trades['Trade Duration'].mean() / 3600

        monthly_results[start] = (round(monthly_pnl, 2), round(avg_trade_duration, 2))

        total_pnl += monthly_pnl
        total_trade_duration += monthly_trades['Trade Duration'].sum()
        if monthly_pnl > 0:
            profit_months += 1
        else:
            loss_months += 1

    avg_trade_duration_total = (total_trade_duration / total_trades) / 3600 if total_trades > 0 else 0
    return total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration_total

# Полный бэктест
def full_period_backtest(strategy, data, initial_capital=1_000_000):
    data.index = pd.to_datetime(data.index).tz_localize(None)
    test = backtesting.Backtest(data, strategy, cash=initial_capital, commission=0.0005)
    res = test.run()
    trades = res._trades.copy()
    return trades, res


# Оптимизация стратегии
def optimize_strategy(data, symbol, interval, start_date, end_date, shared_folder_path, max_evals=10):
    intervals = generate_monthly_intervals(start_date, end_date)
    trials = Trials()

    space = {
        'window_length': hp.quniform('window_length', 10, 100, 2),
        'min_space': hp.quniform('min_space', 3, 20, 1),
        'sloping_atr_length': hp.quniform('sloping_atr_length', 7, 30, 1),
        'take_profit_multiplier': hp.uniform('take_profit_multiplier', 0.5, 3.1),
        'stop_loss_multiplier': hp.uniform('stop_loss_multiplier', 0.5, 3.1)
    }

    results = []

    print(f"Общее количество строк в загруженных данных: {len(data)}")
    print(f"Диапазон дат: {data.index.min()} - {data.index.max()}")
    print(f"Оптимизация для {symbol} ({interval}) с {start_date} по {end_date} началась...\n")

    def objective(params):
        SlopingStrategy.window_length = int(params['window_length'])
        SlopingStrategy.min_space = int(params['min_space'])
        SlopingStrategy.sloping_atr_length = int(params['sloping_atr_length'])
        SlopingStrategy.take_profit_multiplier = params['take_profit_multiplier']
        SlopingStrategy.stop_loss_multiplier = params['stop_loss_multiplier']

        try:
            trades, res = full_period_backtest(
                strategy=SlopingStrategy,
                data=data,
                initial_capital=1_000_000
            )

            total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration = analyze_monthly_results(
                trades=trades,
                intervals=intervals,
                initial_capital=1_000_000
            )

            print(f"Параметры: {params}")
            print(f"Общий профит: {total_pnl:.2f}%, Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}")
            print(f"Результаты по месяцам: {[(month, pnl) for month, (pnl, _) in monthly_results.items()]}\n")

            # Формируем результат с правильным порядком колонок
            row = {
                'symbol': symbol,
                'interval': interval,
                'window_length': int(params['window_length']),
                'min_space': int(params['min_space']),
                'sloping_atr_length': int(params['sloping_atr_length']),
                'take_profit_multiplier': params['take_profit_multiplier'],
                'stop_loss_multiplier': params['stop_loss_multiplier'],
                'total_pnl%': total_pnl,
                'avg_trade_duration_hours': avg_trade_duration,
                'profit_month_count': profit_months,
                'loss_month_count': loss_months
            }

            # Добавляем результаты по месяцам
            for month, (pnl, duration) in monthly_results.items():
                row[month] = pnl

            results.append(row)
            return {'loss': -total_pnl, 'status': 'ok', 'params': params}

        except Exception as e:
            print(f"Ошибка в objective: {e}")
            return {'loss': float('inf'), 'status': 'fail', 'params': params}
    try:
        fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    except Exception as e:
        print(f"Ошибка при выполнении fmin: {e}")
        return None

    results_df = pd.DataFrame(results)
    save_results_to_csv(results_df, shared_folder_path, "hyperopt_optimization_results.csv")

    if trials.best_trial:
        best_trial = trials.best_trial['result']
        best_params = best_trial['params']
        best_loss = best_trial['loss']
        return best_params, -best_loss
    else:
        print("Не удалось найти лучшие параметры.")
        return None

In [ ]:
import pandas as pd
from datetime import datetime
from hyperopt import fmin, tpe, hp, Trials
import backtesting


def analyze_monthly_results(trades, intervals, initial_capital):
    trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
    trades.set_index('Close Time', inplace=True)
    trades['Trade Duration'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds()

    monthly_results = {}
    total_pnl = 0
    profit_months = 0
    loss_months = 0
    total_trade_duration = 0
    total_trades = len(trades)

    for start, end in intervals:
        start_date = datetime.strptime(start, "%Y-%m")
        end_date = datetime.strptime(end, "%Y-%m")

        # Фильтруем сделки для текущего месяца
        monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

        if monthly_trades.empty:
            monthly_results[start] = (0.0, 0)
            continue

        # Вычисляем PnL за месяц
        monthly_pnl = monthly_trades['PnL'].sum() / initial_capital * 100

        # Среднее время сделки
        avg_trade_duration = monthly_trades['Trade Duration'].mean() / 3600

        monthly_results[start] = (round(monthly_pnl, 2), round(avg_trade_duration, 2))

        total_pnl += monthly_pnl
        total_trade_duration += monthly_trades['Trade Duration'].sum()
        if monthly_pnl > 0:
            profit_months += 1
        else:
            loss_months += 1

    avg_trade_duration_total = (total_trade_duration / total_trades) / 3600 if total_trades > 0 else 0

    # Вычисляем процентное соотношение профитных месяцев
    total_months = profit_months + loss_months
    profit_month_ratio = (profit_months / total_months * 100) if total_months > 0 else 0

    return total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration_total, profit_month_ratio


def optimize_strategy_without_min_space(data, symbol, interval, start_date, end_date, shared_folder_path, max_evals=10):
    intervals = generate_monthly_intervals(start_date, end_date)
    trials = Trials()

    # Пространство поиска
    space = {
        'window_length': hp.quniform('window_length', 10, 100, 2),
        'sloping_atr_length': hp.quniform('sloping_atr_length', 7, 30, 1),
        'take_profit_multiplier': hp.uniform('take_profit_multiplier', 0.5, 3.1),
        'stop_loss_multiplier': hp.uniform('stop_loss_multiplier', 0.5, 3.1)
    }

    fixed_min_space = 5
    results = []

    print(f"Общее количество строк в загруженных данных: {len(data)}")
    print(f"Диапазон дат: {data.index.min()} - {data.index.max()}")
    print(f"Оптимизация для {symbol} ({interval}) с {start_date} по {end_date} началась...\n")

    def objective(params):
        SlopingStrategy.window_length = int(params['window_length'])
        SlopingStrategy.min_space = fixed_min_space
        SlopingStrategy.sloping_atr_length = int(params['sloping_atr_length'])
        SlopingStrategy.take_profit_multiplier = params['take_profit_multiplier']
        SlopingStrategy.stop_loss_multiplier = params['stop_loss_multiplier']

        try:
            trades, res = full_period_backtest(
                strategy=SlopingStrategy,
                data=data,
                initial_capital=1_000_000
            )

            total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration, profit_month_ratio = analyze_monthly_results(
                trades=trades,
                intervals=intervals,
                initial_capital=1_000_000
            )

            print(f"Параметры: {params}")
            print(f"Общий профит: {total_pnl:.2f}%, Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}, Процент профитных месяцев: {profit_month_ratio:.2f}%")
            print(f"Результаты по месяцам: {[(month, pnl) for month, (pnl, _) in monthly_results.items()]}\n")

            row = {
                'symbol': symbol,
                'interval': interval,
                'window_length': int(params['window_length']),
                'min_space': fixed_min_space,
                'sloping_atr_length': int(params['sloping_atr_length']),
                'take_profit_multiplier': params['take_profit_multiplier'],
                'stop_loss_multiplier': params['stop_loss_multiplier'],
                'total_pnl%': total_pnl,
                'avg_trade_duration_hours': avg_trade_duration,
                'profit_month_count': profit_months,
                'loss_month_count': loss_months,
                'profit_month_ratio%': profit_month_ratio
            }

            for month, (pnl, duration) in monthly_results.items():
                row[month] = pnl

            results.append(row)
            return {'loss': -total_pnl, 'status': 'ok', 'params': params}

        except Exception as e:
            print(f"Ошибка в objective: {e}")
            return {'loss': float('inf'), 'status': 'fail', 'params': params}

    try:
        fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
    except Exception as e:
        print(f"Ошибка при выполнении fmin: {e}")
        return None

    results_df = pd.DataFrame(results)
    save_results_to_csv(results_df, shared_folder_path, "hyperopt_optimization_results.csv")

    # Отфильтровать результаты с учетом профитных месяцев
    filtered_results = results_df[results_df['profit_month_ratio%'] > 50]
    filtered_results.to_csv(f"{shared_folder_path}/filtered_results.csv", index=False)

    if trials.best_trial:
        best_trial = trials.best_trial['result']
        best_params = best_trial['params']
        best_loss = best_trial['loss']
        return best_params, -best_loss
    else:
        print("Не удалось найти лучшие параметры.")
        return None


#### Check

In [ ]:
# import pandas as pd
# from datetime import datetime
# from hyperopt import fmin, tpe, hp, Trials
# import backtesting

# # Генерация интервалов
# def generate_monthly_intervals(start_date, end_date):
#     start = datetime.strptime(start_date, "%Y-%m")
#     end = datetime.strptime(end_date, "%Y-%m")
#     intervals = []
#     while start <= end:
#         next_month = datetime(start.year + (start.month // 12), (start.month % 12) + 1, 1)
#         intervals.append((start.strftime("%Y-%m"), next_month.strftime("%Y-%m")))
#         start = next_month
#     return intervals

# # Анализ помесячно
# def analyze_monthly_results(trades, intervals, initial_capital):
#     trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
#     trades.set_index('Close Time', inplace=True)
#     trades['Trade Duration'] = (trades['ExitTime'] - trades['EntryTime']).dt.total_seconds()

#     monthly_results = {}
#     total_pnl = 0
#     profit_months = 0
#     loss_months = 0
#     total_trade_duration = 0
#     total_trades = len(trades)

#     for start, end in intervals:
#         start_date = datetime.strptime(start, "%Y-%m")
#         end_date = datetime.strptime(end, "%Y-%m")

#         # Фильтруем сделки для текущего месяца
#         monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

#         if monthly_trades.empty:
#             monthly_results[start] = (0.0, 0)
#             continue

#         # Вычисляем PnL за месяц
#         monthly_pnl = monthly_trades['PnL'].sum() / initial_capital * 100

#         # Среднее время сделки
#         avg_trade_duration = monthly_trades['Trade Duration'].mean() / 3600

#         monthly_results[start] = (round(monthly_pnl, 2), round(avg_trade_duration, 2))

#         total_pnl += monthly_pnl
#         total_trade_duration += monthly_trades['Trade Duration'].sum()
#         if monthly_pnl > 0:
#             profit_months += 1
#         else:
#             loss_months += 1

#     avg_trade_duration_total = (total_trade_duration / total_trades) / 3600 if total_trades > 0 else 0
#     return total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration_total

# # Полный бэктест
# def full_period_backtest(strategy, data, initial_capital=1_000_000, commission=0.0005):
#     data.index = pd.to_datetime(data.index).tz_localize(None)
#     test = backtesting.Backtest(data, strategy, cash=initial_capital, commission=commission)
#     res = test.run()
#     trades = res._trades.copy()
#     return trades, res


# # Оптимизация стратегии
# def optimize_strategy(data, symbol, interval, start_date, end_date, shared_folder_path, max_evals=10):
#     intervals = generate_monthly_intervals(start_date, end_date)
#     trials = Trials()

#     space = {
#         'window_length': hp.quniform('window_length', 10, 100, 2),
#         'min_space': hp.quniform('min_space', 3, 20, 1),
#         'sloping_atr_length': hp.quniform('sloping_atr_length', 7, 30, 1),
#         'take_profit_multiplier': hp.uniform('take_profit_multiplier', 1.0, 2.5),
#         'stop_loss_multiplier': hp.uniform('stop_loss_multiplier', 1.0, 2.5)
#     }

#     results = []

#     print(f"Общее количество строк в загруженных данных: {len(data)}")
#     print(f"Диапазон дат: {data.index.min()} - {data.index.max()}")
#     print(f"Оптимизация для {symbol} ({interval}) с {start_date} по {end_date} началась...\n")

#     def objective(params):
#         SlopingStrategy.window_length = int(params['window_length'])
#         SlopingStrategy.min_space = int(params['min_space'])
#         SlopingStrategy.sloping_atr_length = int(params['sloping_atr_length'])
#         SlopingStrategy.take_profit_multiplier = params['take_profit_multiplier']
#         SlopingStrategy.stop_loss_multiplier = params['stop_loss_multiplier']

#         try:
#             trades, res = full_period_backtest(
#                 strategy=SlopingStrategy,
#                 data=data,
#                 initial_capital=1_000_000
#             )

#             total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration = analyze_monthly_results(
#                 trades=trades,
#                 intervals=intervals,
#                 initial_capital=1_000_000
#             )

#             print(f"Параметры: {params}")
#             print(f"Общий профит: {total_pnl:.2f}%, Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}")
#             print(f"Результаты по месяцам: {[(month, pnl) for month, (pnl, _) in monthly_results.items()]}\n")

#             # Формируем результат с правильным порядком колонок
#             row = {
#                 'symbol': symbol,
#                 'interval': interval,
#                 'window_length': int(params['window_length']),
#                 'min_space': int(params['min_space']),
#                 'sloping_atr_length': int(params['sloping_atr_length']),
#                 'take_profit_multiplier': params['take_profit_multiplier'],
#                 'stop_loss_multiplier': params['stop_loss_multiplier'],
#                 'total_pnl%': total_pnl,
#                 'avg_trade_duration_hours': avg_trade_duration,
#                 'profit_month_count': profit_months,
#                 'loss_month_count': loss_months
#             }

#             # Добавляем результаты по месяцам
#             for month, (pnl, duration) in monthly_results.items():
#                 row[month] = pnl

#             results.append(row)
#             return {'loss': -total_pnl, 'status': 'ok', 'params': params}

#         except Exception as e:
#             print(f"Ошибка в objective: {e}")
#             return {'loss': float('inf'), 'status': 'fail', 'params': params}
#     try:
#         fmin(objective, space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
#     except Exception as e:
#         print(f"Ошибка при выполнении fmin: {e}")
#         return None

#     results_df = pd.DataFrame(results)
#     save_results_to_csv(results_df, shared_folder_path, "hyperopt_optimization_results.csv")

#     if trials.best_trial:
#         best_trial = trials.best_trial['result']
#         best_params = best_trial['params']
#         best_loss = best_trial['loss']
#         return best_params, -best_loss
#     else:
#         print("Не удалось найти лучшие параметры.")
#         return None

In [ ]:
import pandas as pd
from datetime import datetime
import backtesting






def check_monthly_results(trades, intervals, initial_capital):
    """
    Анализирует результаты бэктеста по месяцам.
    """
    if trades.empty:
        print("Нет сделок для анализа.")
        return 0.0, 0, 0, {}, 0.0

    # Преобразуем временные индексы
    trades['Close Time'] = pd.to_datetime(trades['ExitTime']).dt.tz_localize(None)
    trades['Entry Time'] = pd.to_datetime(trades['EntryTime']).dt.tz_localize(None)
    trades.set_index('Close Time', inplace=True)

    # Инициализация переменных
    monthly_results = {}
    total_pnl = 0.0
    profit_months = 0
    loss_months = 0
    total_trade_duration = 0
    total_trades = len(trades)

    for start, end in intervals:
        start_date = pd.to_datetime(start)
        end_date = pd.to_datetime(end)

        # Отбор сделок за месяц
        monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

        if monthly_trades.empty:
            monthly_results[start] = {'pnl': 0.0, 'trade_count': 0}
            continue

        # Рассчитываем метрики
        monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')
        monthly_pnl = monthly_trades['PnL'].sum() / initial_capital * 100
        trade_count = len(monthly_trades)
        total_trade_duration += (monthly_trades['ExitTime'] - monthly_trades['EntryTime']).dt.total_seconds().sum()

        monthly_results[start] = {'pnl': round(monthly_pnl, 2), 'trade_count': trade_count}

        total_pnl += monthly_pnl
        if monthly_pnl > 0:
            profit_months += 1
        else:
            loss_months += 1

    avg_trade_duration = total_trade_duration / total_trades / 3600 if total_trades > 0 else 0

    return total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration


def manual_backtest(data, strategy, symbol, interval, start_date, end_date, initial_capital=1_000_000,
                    window_length=32, min_space=5, sloping_atr_length=8,
                    take_profit_multiplier=1.57, stop_loss_multiplier=1.4):
    """
    Проводит бэктест стратегии с заданными параметрами и анализирует результаты по месяцам.
    """
    # Установка параметров стратегии
    strategy.window_length = window_length
    strategy.min_space = min_space
    strategy.sloping_atr_length = sloping_atr_length
    strategy.take_profit_multiplier = take_profit_multiplier
    strategy.stop_loss_multiplier = stop_loss_multiplier

    # Генерация интервалов
    intervals = generate_monthly_intervals(start_date, end_date)

    # Запуск бэктеста
    trades, res = full_period_backtest(strategy, data, initial_capital)

    # Анализ результатов
    total_pnl, profit_months, loss_months, monthly_results, avg_trade_duration = check_monthly_results(
        trades, intervals, initial_capital
    )

    # Дополнительные метрики
    monthly_pnls = [result['pnl'] for result in monthly_results.values()]
    avg_monthly_profit = round(sum(pnl for pnl in monthly_pnls if pnl > 0) / max(profit_months, 1), 2)
    avg_monthly_loss = round(sum(pnl for pnl in monthly_pnls if pnl < 0) / max(loss_months, 1), 2)
    max_monthly_profit = max(monthly_pnls, default=0)
    min_monthly_profit = min(monthly_pnls, default=0)
    total_trades = len(trades)
    successful_trades = len(trades[trades['PnL'] > 0]) if not trades.empty else 0
    winrate = round((successful_trades / total_trades) * 100, 2) if total_trades > 0 else 0.0
    monthly_winrate = round((profit_months / (profit_months + loss_months)) * 100, 2) if (profit_months + loss_months) > 0 else 0.0

    # Итоговый результат
    final_results = {
        'symbol': symbol,
        'interval': interval,
        'window_length': window_length,
        'min_space': min_space,
        'sloping_atr_length': sloping_atr_length,
        'take_profit_multiplier': take_profit_multiplier,
        'stop_loss_multiplier': stop_loss_multiplier,
        'total_pnl%': total_pnl,
        'avg_trade_duration_hours': avg_trade_duration,
        'profit_month_count': profit_months,
        'loss_month_count': loss_months,
        'avg_monthly_profit%': avg_monthly_profit,
        'avg_monthly_loss%': avg_monthly_loss,
        'max_monthly_profit%': max_monthly_profit,
        'min_monthly_profit%': min_monthly_profit,
        'overall_winrate%': winrate,
        'monthly_winrate%': monthly_winrate
    }

    print(f"\nРезультаты бэктеста для {symbol} ({interval}):")
    print(f"Общее количество сделок: {total_trades}")
    print(f"Общий профит: {total_pnl:.2f}%, Средняя длительность сделки: {avg_trade_duration:.2f} ч.")
    print(f"Профитные месяцы: {profit_months}, Убыточные месяцы: {loss_months}")
    print(f"Средний профит в месяц: {avg_monthly_profit:.2f}%, Средний убыток в месяц: {avg_monthly_loss:.2f}%")
    print(f"Максимальный профит в месяц: {max_monthly_profit:.2f}%, Минимальный профит в месяц: {min_monthly_profit:.2f}%")
    print(f"Общий winrate: {winrate:.2f}%, Winrate по месяцам: {monthly_winrate:.2f}%\n")

    print("Анализ по месяцам:")
    for month, result in monthly_results.items():
        print(f"{month}: PnL = {result['pnl']:.2f}%, Сделок = {result['trade_count']}")

    return final_results






## Backtesting and Optimization

### without min_space

#### 1

In [ ]:
# Настройка и вызов оптимизации
# symbol = 'LTCUSDT'
# interval = '1h'
# start_date = '2023-11'
# end_date = '2024-11'

# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")


Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-11-30 23:45:00+00:00


In [ ]:
# Запуск оптимизации
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy_without_min_space(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
# SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.min_space =5
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-11-30 23:45:00+00:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-11 началась...

Параметры: {'sloping_atr_length': 26.0, 'stop_loss_multiplier': 2.676323045630937, 'take_profit_multiplier': 0.6921648516441543, 'window_length': 58.0}
Общий профит: -52.23%, Профитные месяцы: 4, Убыточные месяцы: 10, Процент профитных месяцев: 28.57%
Результаты по месяцам: [('2023-10', -2.88), ('2023-11', -10.68), ('2023-12', -5.78), ('2024-01', -0.08), ('2024-02', 2.83), ('2024-03', 6.06), ('2024-04', -20.16), ('2024-05', -1.56), ('2024-06', -5.69), ('2024-07', -8.19), ('2024-08', -5.06), ('2024-09', 1.3), ('2024-10', 0.84), ('2024-11', -3.19)]

Параметры: {'sloping_atr_length': 10.0, 'stop_loss_multiplier': 1.4306347357037255, 'take_profit_multiplier': 1.2303025136455226, 'window_length': 50.0}
Общий профит: -47.11%, Профитные месяцы: 5, Убыточные месяцы: 9, Процент профитных месяцев: 35.71%
Результаты 

In [ ]:
# Запуск оптимизации
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy_without_min_space(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
# SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.min_space =5
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00 - 2024-11-30 23:45:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-11 началась...

Параметры: {'sloping_atr_length': 9.0, 'stop_loss_multiplier': 1.829528472412791, 'take_profit_multiplier': 1.0983357276071324, 'window_length': 82.0}
Общий профит: -50.10%, Профитные месяцы: 3, Убыточные месяцы: 11, Процент профитных месяцев: 21.43%
Результаты по месяцам: [('2023-10', -7.1), ('2023-11', 3.11), ('2023-12', -2.39), ('2024-01', -15.13), ('2024-02', -5.16), ('2024-03', 17.93), ('2024-04', -9.33), ('2024-05', -19.42), ('2024-06', -5.51), ('2024-07', -9.22), ('2024-08', -2.23), ('2024-09', 10.51), ('2024-10', -0.42), ('2024-11', -5.73)]

Параметры: {'sloping_atr_length': 26.0, 'stop_loss_multiplier': 2.6179254624031745, 'take_profit_multiplier': 1.0027190107320538, 'window_length': 62.0}
Общий профит: -65.90%, Профитные месяцы: 4, Убыточные месяцы: 10, Процент профитных месяцев: 28.57%
Результаты по месяца

In [ ]:
# Запуск оптимизации
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy_without_min_space(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
# SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.min_space =5
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00 - 2024-11-30 23:45:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-11 началась...

Параметры: {'sloping_atr_length': 18.0, 'stop_loss_multiplier': 0.9878569952080154, 'take_profit_multiplier': 2.7151085281802083, 'window_length': 62.0}
Общий профит: -70.09%, Профитные месяцы: 5, Убыточные месяцы: 9, Процент профитных месяцев: 35.71%
Результаты по месяцам: [('2023-10', -12.04), ('2023-11', -1.35), ('2023-12', -4.37), ('2024-01', 0.87), ('2024-02', 6.28), ('2024-03', -3.93), ('2024-04', -26.21), ('2024-05', -13.89), ('2024-06', 4.23), ('2024-07', -5.33), ('2024-08', -4.26), ('2024-09', 1.92), ('2024-10', 1.07), ('2024-11', -13.08)]

Параметры: {'sloping_atr_length': 23.0, 'stop_loss_multiplier': 1.032773577256214, 'take_profit_multiplier': 0.5272035133157916, 'window_length': 78.0}
Общий профит: -52.62%, Профитные месяцы: 5, Убыточные месяцы: 9, Процент профитных месяцев: 35.71%
Результаты по месяцам:

#### 2

In [ ]:
# Настройка и вызов оптимизации
# symbol = 'SUIUSDT'
# interval = '1h'
# start_date = '2024-11'
# end_date = '2024-11'

symbol = symbol_1
interval = interval_1
start_date = start_date_1
end_date = end_date_1
max_evals = max_evals_1

# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")
max_evals = max_evals_1

Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-11-30 23:45:00+00:00


In [ ]:
# Запуск оптимизации

shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy_without_min_space(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
# SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.min_space =5
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-11-30 23:45:00+00:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-11 началась...

Параметры: {'sloping_atr_length': 26.0, 'stop_loss_multiplier': 1.8094745753145491, 'take_profit_multiplier': 2.6051592790982214, 'window_length': 26.0}
Общий профит: 12.42%, Профитные месяцы: 5, Убыточные месяцы: 9, Процент профитных месяцев: 35.71%
Результаты по месяцам: [('2023-10', -10.65), ('2023-11', 29.8), ('2023-12', -11.27), ('2024-01', -5.79), ('2024-02', 19.21), ('2024-03', -7.18), ('2024-04', -33.29), ('2024-05', -4.61), ('2024-06', -1.67), ('2024-07', -7.12), ('2024-08', -1.79), ('2024-09', 7.77), ('2024-10', 11.14), ('2024-11', 27.89)]

Параметры: {'sloping_atr_length': 26.0, 'stop_loss_multiplier': 2.0885729503649797, 'take_profit_multiplier': 1.3559278832841466, 'window_length': 18.0}
Общий профит: 11.15%, Профитные месяцы: 8, Убыточные месяцы: 6, Процент профитных месяцев: 57.14%
Результат

In [ ]:
# Запуск оптимизаци
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy_without_min_space(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
# SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.min_space =5
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00 - 2024-11-30 23:45:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-11 началась...

Параметры: {'sloping_atr_length': 24.0, 'stop_loss_multiplier': 2.7605233411475627, 'take_profit_multiplier': 2.5304209669003974, 'window_length': 82.0}
Общий профит: -49.78%, Профитные месяцы: 5, Убыточные месяцы: 9, Процент профитных месяцев: 35.71%
Результаты по месяцам: [('2023-10', -11.71), ('2023-11', -13.97), ('2023-12', -1.79), ('2024-01', 6.92), ('2024-02', 3.3), ('2024-03', 41.59), ('2024-04', -20.19), ('2024-05', -14.2), ('2024-06', -5.56), ('2024-07', -10.34), ('2024-08', -28.81), ('2024-09', 2.65), ('2024-10', 4.86), ('2024-11', -2.52)]

Параметры: {'sloping_atr_length': 23.0, 'stop_loss_multiplier': 2.369118558611355, 'take_profit_multiplier': 2.362819541665705, 'window_length': 58.0}
Общий профит: -88.42%, Профитные месяцы: 3, Убыточные месяцы: 11, Процент профитных месяцев: 21.43%
Результаты по месяцам

In [ ]:
# Запуск оптимизации
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy_without_min_space(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
# SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.min_space =5
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00 - 2024-11-30 23:45:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-11 началась...

Параметры: {'sloping_atr_length': 8.0, 'stop_loss_multiplier': 1.072096789249105, 'take_profit_multiplier': 2.282730670602814, 'window_length': 52.0}
Общий профит: -62.60%, Профитные месяцы: 5, Убыточные месяцы: 9, Процент профитных месяцев: 35.71%
Результаты по месяцам: [('2023-10', -12.73), ('2023-11', 8.48), ('2023-12', 2.73), ('2024-01', 7.23), ('2024-02', 20.63), ('2024-03', -23.61), ('2024-04', -20.05), ('2024-05', -8.69), ('2024-06', -5.2), ('2024-07', -5.02), ('2024-08', -3.02), ('2024-09', 3.18), ('2024-10', -14.34), ('2024-11', -12.19)]

Параметры: {'sloping_atr_length': 9.0, 'stop_loss_multiplier': 1.9575927302041962, 'take_profit_multiplier': 2.0245535633875176, 'window_length': 38.0}
Общий профит: -55.26%, Профитные месяцы: 5, Убыточные месяцы: 9, Процент профитных месяцев: 35.71%
Результаты по месяцам: [

#### 3

In [ ]:
# Настройка и вызов оптимизации
# symbol = 'BNBUSDT'
# interval = '1h'
# start_date = '2024-11'
# end_date = '2024-11'

symbol = symbol_2
interval = interval_2
start_date = start_date_2
end_date = end_date_2
max_evals = max_evals_2

# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")

Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-11-30 23:45:00+00:00


In [ ]:
# Запуск оптимизации
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy_without_min_space(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
# SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.min_space =5
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

Общее количество строк в загруженных данных: 40992
Диапазон дат: 2023-10-01 00:00:00+00:00 - 2024-11-30 23:45:00+00:00
Оптимизация для SUIUSDT (15m) с 2023-10 по 2024-11 началась...

Параметры: {'sloping_atr_length': 7.0, 'stop_loss_multiplier': 2.3153593144564146, 'take_profit_multiplier': 1.9060420695107527, 'window_length': 88.0}
Общий профит: -75.62%, Профитные месяцы: 5, Убыточные месяцы: 9, Процент профитных месяцев: 35.71%
Результаты по месяцам: [('2023-10', -0.7), ('2023-11', -30.64), ('2023-12', 0.18), ('2024-01', -20.76), ('2024-02', 9.1), ('2024-03', 2.3), ('2024-04', -12.29), ('2024-05', -10.71), ('2024-06', -1.36), ('2024-07', -3.23), ('2024-08', -9.73), ('2024-09', 1.66), ('2024-10', 2.64), ('2024-11', -2.1)]

Параметры: {'sloping_atr_length': 21.0, 'stop_loss_multiplier': 2.594832793883783, 'take_profit_multiplier': 2.6613775506337287, 'window_length': 82.0}
Общий профит: -40.53%, Профитные месяцы: 4, Убыточные месяцы: 10, Процент профитных месяцев: 28.57%
Результаты по 

In [ ]:
# Запуск оптимизации
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy_without_min_space(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
# SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.min_space =5
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

In [ ]:
# Запуск оптимизации
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy_without_min_space(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
# SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.min_space =5
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

### with min_space

In [ ]:
# Настройка и вызов оптимизации
symbol = 'BTCUSDT'
interval = '1h'
start_date = '2020-10'
end_date = '2024-10'

# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)
print(f"Общее количество строк в загруженных данных: {len(df)}")
print(f"Диапазон дат: {df.index.min()} - {df.index.max()}")

Общее количество строк в загруженных данных: 35808
Диапазон дат: 2020-10-01 00:00:00+00:00 - 2024-10-31 23:00:00+00:00


In [ ]:
# Запуск оптимизации
max_evals = 50
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

Общее количество строк в загруженных данных: 35808
Диапазон дат: 2020-10-01 00:00:00+00:00 - 2024-10-31 23:00:00+00:00
Оптимизация для BTCUSDT (1h) с 2020-10 по 2024-10 началась...

Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expected 5)
Ошибка в objective: too many values to unpack (expe

TypeError: cannot unpack non-iterable NoneType object

In [ ]:
# Запуск оптимизации
max_evals = 50
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

In [ ]:
# Запуск оптимизации
max_evals = 50
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

In [ ]:
# Запуск оптимизации
max_evals = 50
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

In [ ]:
# Запуск оптимизации
max_evals = 50
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

In [ ]:
# Запуск оптимизации
max_evals = 50
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

In [ ]:
# Запуск оптимизации
max_evals = 50
shared_folder_path = '/content/drive/MyDrive/Sloping bot'
best_params, total_pnl = optimize_strategy(
    data=df,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    max_evals=max_evals,
    shared_folder_path=shared_folder_path
)

# Настройка стратегии с оптимизированными параметрами
SlopingStrategy.window_length = int(best_params['window_length'])
SlopingStrategy.min_space = int(best_params['min_space'])
SlopingStrategy.sloping_atr_length = int(best_params['sloping_atr_length'])
SlopingStrategy.take_profit_multiplier = best_params['take_profit_multiplier']
SlopingStrategy.stop_loss_multiplier = best_params['stop_loss_multiplier']

print("\nНастройка стратегии завершена.")
test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

## Final analysis

### Optimization results

#### full optimization

In [ ]:
import pandas as pd

# Считаем данные из файла
file_path = '/content/drive/MyDrive/Sloping bot/optimized_params.csv'
df_optimized = pd.read_csv(file_path)
df_optimized

,Symbol,Interval,Start Date,End Date,Test Return [%],window_length,min_space,sloping_atr_length,stop_loss_multiplier,take_profit_multiplier
0,NOTUSDT,30m,2024-05,2024-10,233.572465,80.0,NaN,17.0,3.471899,3.984025
1,NOTUSDT,30m,2024-05,2024-10,103.217958,14.0,NaN,20.0,1.086321,2.724040
2,NOTUSDT,30m,2024-05,2024-10,-38.082432,24.0,NaN,11.0,3.325552,3.461578
3,NOTUSDT,30m,2024-05,2024-10,24.518548,48.0,NaN,29.0,1.894256,1.208675
4,NOTUSDT,30m,2024-05,2024-10,77.741279,58.0,NaN,19.0,3.176453,2.906685
...,...,...,...,...,...,...,...,...,...,...
3032,APEUSDT,4h,2022-06,2024-10,-43.079953,52.0,10.0,21.0,1.500000,1.500000
3033,APEUSDT,4h,2022-06,2024-10,210.131328,22.0,18.0,17.0,1.500000,1.500000
3034,APEUSDT,4h,2022-06,2024-10,232.527972,38.0,18.0,33.0,1.500000,1.500000
3035,APEUSDT,4h,2022-06,2024-10,-3.252550,30.0,19.0,28.0,1.500000,1.500000


In [ ]:
df_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1400 entries, 0 to 1399
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Symbol                  1400 non-null   object 
 1   Interval                1400 non-null   object 
 2   Start Date              1400 non-null   object 
 3   End Date                1400 non-null   object 
 4   Test Return [%]         1400 non-null   float64
 5   window_length           1400 non-null   float64
 6   min_space               21 non-null     float64
 7   sloping_atr_length      1400 non-null   float64
 8   stop_loss_multiplier    1399 non-null   float64
 9   take_profit_multiplier  1399 non-null   float64
dtypes: float64(6), object(4)
memory usage: 109.5+ KB


In [ ]:
file_name = "optimized_params.csv"
df_optimized.to_csv(file_name, index=False)

In [ ]:
# Удалим дубликаты, если они есть
df_optimized.drop_duplicates(inplace=True)
df_optimized.reset_index(drop=True, inplace=True)
df_optimized.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3037 entries, 0 to 3036
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Symbol                  3037 non-null   object 
 1   Interval                3037 non-null   object 
 2   Start Date              3037 non-null   object 
 3   End Date                3037 non-null   object 
 4   Test Return [%]         3037 non-null   float64
 5   window_length           3037 non-null   float64
 6   min_space               178 non-null    float64
 7   sloping_atr_length      3037 non-null   float64
 8   stop_loss_multiplier    3035 non-null   float64
 9   take_profit_multiplier  3035 non-null   float64
dtypes: float64(6), object(4)
memory usage: 237.4+ KB


In [ ]:
df_optimized

,Symbol,Interval,Start Date,End Date,Test Return [%],window_length,min_space,sloping_atr_length,stop_loss_multiplier,take_profit_multiplier
0,NOTUSDT,30m,2024-05,2024-10,233.572465,80.0,NaN,17.0,3.471899,3.984025
1,NOTUSDT,30m,2024-05,2024-10,103.217958,14.0,NaN,20.0,1.086321,2.724040
2,NOTUSDT,30m,2024-05,2024-10,-38.082432,24.0,NaN,11.0,3.325552,3.461578
3,NOTUSDT,30m,2024-05,2024-10,24.518548,48.0,NaN,29.0,1.894256,1.208675
4,NOTUSDT,30m,2024-05,2024-10,77.741279,58.0,NaN,19.0,3.176453,2.906685
...,...,...,...,...,...,...,...,...,...,...
3032,APEUSDT,4h,2022-06,2024-10,-43.079953,52.0,10.0,21.0,1.500000,1.500000
3033,APEUSDT,4h,2022-06,2024-10,210.131328,22.0,18.0,17.0,1.500000,1.500000
3034,APEUSDT,4h,2022-06,2024-10,232.527972,38.0,18.0,33.0,1.500000,1.500000
3035,APEUSDT,4h,2022-06,2024-10,-3.252550,30.0,19.0,28.0,1.500000,1.500000


In [ ]:
coin_dfs = {}

symbols = df_optimized['Symbol'].unique()

for symbol in symbols:
    # Filter data by symbol
    coin_data = df_optimized[df_optimized['Symbol'] == symbol]

    # Sort data by 'Interval' and 'Test Return [%]' in descending order
    sorted_coin_data = coin_data.sort_values(by=['Interval', 'Test Return [%]'], ascending=[True, False])

    # Store the dataframe in the dictionary using the symbol as the key
    coin_dfs[f"{symbol}_optimized_df"] = sorted_coin_data

    # Save to CSV file
    file_name = f"{symbol}_optimization_params.csv"
    sorted_coin_data.to_csv(file_name, index=False)

    print(f'File {file_name} created for download, data saved in {symbol}_optimized_df.')

File NOTUSDT_optimization_params.csv created for download, data saved in NOTUSDT_optimized_df.
File WLDUSDT_optimization_params.csv created for download, data saved in WLDUSDT_optimized_df.
File TIAUSDT_optimization_params.csv created for download, data saved in TIAUSDT_optimized_df.
File XRPUSDT_optimization_params.csv created for download, data saved in XRPUSDT_optimized_df.
File TONUSDT_optimization_params.csv created for download, data saved in TONUSDT_optimized_df.
File SOLUSDT_optimization_params.csv created for download, data saved in SOLUSDT_optimized_df.
File APEUSDT_optimization_params.csv created for download, data saved in APEUSDT_optimized_df.
File WIFUSDT_optimization_params.csv created for download, data saved in WIFUSDT_optimized_df.
File ADAUSDT_optimization_params.csv created for download, data saved in ADAUSDT_optimized_df.
File BTCUSDT_optimization_params.csv created for download, data saved in BTCUSDT_optimized_df.
File ETHUSDT_optimization_params.csv created for d

In [ ]:
import pandas as pd

# Задаём пользовательский порядок сортировки для interval
interval_order = {'4h': 0, '1h': 1, '15m': 2, '5m': 3, '1m': 4}

# Добавляем новый столбец для пользовательской сортировки
df_filtered = df_optimized[df_optimized['Test Return [%]'] > 100].copy()
df_filtered['interval_order'] = df_filtered['Interval'].map(interval_order)

# Сортировка по символу, interval_order и total_pnl% (по убыванию)
df_sorted = df_filtered.sort_values(
    by=['Symbol', 'interval_order', 'Test Return [%]'],
    ascending=[True, True, False]
).drop(columns=['interval_order'])  # Убираем временный столбец

# Сохраняем в CSV (если нужно)
df_sorted.to_csv('sorted_data.csv', index=False, encoding='utf-8')

print("Файл успешно сохранён как sorted_data.csv")

Файл успешно сохранён как sorted_data.csv


In [ ]:
# NOTUSDT_optimized_df

In [ ]:
sorted_coin_data

,Symbol,Interval,Start Date,End Date,Test Return [%],window_length,min_space,sloping_atr_length,stop_loss_multiplier,take_profit_multiplier
1298,FILUSDT,4h,2024-09,2024-10,inf,50.0,NaN,14.0,1.5,1.5
1295,FILUSDT,4h,2024-09,2024-10,33.752706,10.0,NaN,7.0,NaN,NaN
1297,FILUSDT,4h,2024-09,2024-10,9.964792,50.0,NaN,14.0,1.5,1.5
1296,FILUSDT,4h,2024-09,2024-10,7.473594,50.0,NaN,14.0,1.5,1.5


#### hyperopt optimization by month

In [ ]:
import pandas as pd

# Считаем данные из файла
file_path_by_month = '/content/drive/MyDrive/Sloping bot/hyperopt_optimization_results.csv'
df_optimized_by_month = pd.read_csv(file_path_by_month)
df_optimized_by_month

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%
0,BTCUSDT,4h,37,8,10,1.856871,1.252420,-65.807486,26.230366,18,...,-1.96,-5.87,-3.75,-1.40,-0.83,0.21,1.26,-0.91,1.13,36.734694
1,BTCUSDT,4h,32,10,22,1.290515,1.469040,-57.109051,23.248677,20,...,-0.38,-11.27,-0.11,-1.76,-0.91,3.01,-3.38,-2.25,1.38,40.816327
2,BTCUSDT,4h,31,14,21,1.663784,1.531341,-76.389729,30.827586,18,...,-1.56,-5.95,0.04,-0.72,-0.36,-0.49,1.49,-0.68,0.51,36.734694
3,BTCUSDT,4h,34,10,29,1.133459,1.282490,-45.549995,18.742268,24,...,-4.05,-9.26,0.11,-2.29,-3.93,3.99,-8.61,-2.66,1.67,48.979592
4,BTCUSDT,4h,43,11,15,1.396247,1.982624,-65.743008,31.259740,17,...,0.56,-0.13,0.47,0.89,-0.36,0.79,2.70,-1.91,1.05,34.693878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36967,INJUSDT,1h,96,5,17,1.833637,1.866539,-0.145921,9.566532,12,...,-5.91,7.30,8.74,6.09,-5.81,-5.34,-6.81,-24.15,0.76,24.489796
36968,INJUSDT,1h,80,5,14,1.627263,2.105363,-66.265195,9.567780,9,...,-13.29,3.08,-5.94,7.53,4.53,0.64,-6.68,-9.85,-1.04,18.367347
36969,INJUSDT,1h,92,5,16,1.441189,2.013496,-27.279900,7.976699,10,...,-4.68,11.01,-1.65,11.60,-5.74,15.02,-7.84,-9.56,5.55,20.408163
36970,INJUSDT,1h,88,5,20,1.952279,1.727292,-20.327936,10.030488,13,...,-18.14,7.59,-6.44,10.61,-5.08,13.29,-9.67,-15.88,6.08,26.530612


In [ ]:
# df_optimized_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5842 entries, 0 to 5841
Data columns (total 60 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   symbol                    5842 non-null   object 
 1   interval                  5842 non-null   object 
 2   window_length             5842 non-null   int64  
 3   min_space                 5842 non-null   int64  
 4   sloping_atr_length        5842 non-null   int64  
 5   take_profit_multiplier    5842 non-null   float64
 6   stop_loss_multiplier      5842 non-null   float64
 7   total_pnl%                5842 non-null   float64
 8   avg_trade_duration_hours  5842 non-null   float64
 9   profit_month_count        5842 non-null   int64  
 10  loss_month_count          5842 non-null   int64  
 11  2020-10                   5142 non-null   float64
 12  2020-11                   5142 non-null   float64
 13  2020-12                   5142 non-null   float64
 14  2021-01 

In [ ]:
# Удалим дубликаты, если они есть
df_optimized_by_month.drop_duplicates(inplace=True)
df_optimized_by_month.reset_index(drop=True, inplace=True)

file_name = "hyperopt_optimization_results.csv"
df_optimized_by_month.to_csv(file_name, index=False)

df_optimized_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36972 entries, 0 to 36971
Data columns (total 61 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   symbol                    36972 non-null  object 
 1   interval                  36972 non-null  object 
 2   window_length             36972 non-null  int64  
 3   min_space                 36972 non-null  int64  
 4   sloping_atr_length        36972 non-null  int64  
 5   take_profit_multiplier    36972 non-null  float64
 6   stop_loss_multiplier      36972 non-null  float64
 7   total_pnl%                36972 non-null  float64
 8   avg_trade_duration_hours  36972 non-null  float64
 9   profit_month_count        36972 non-null  int64  
 10  loss_month_count          36972 non-null  int64  
 11  2020-10                   36972 non-null  float64
 12  2020-11                   36972 non-null  float64
 13  2020-12                   36972 non-null  float64
 14  2021-0

In [ ]:
df_optimized_by_month

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%
0,BTCUSDT,4h,37,8,10,1.856871,1.252420,-65.807486,26.230366,18,...,-1.96,-5.87,-3.75,-1.40,-0.83,0.21,1.26,-0.91,1.13,36.734694
1,BTCUSDT,4h,32,10,22,1.290515,1.469040,-57.109051,23.248677,20,...,-0.38,-11.27,-0.11,-1.76,-0.91,3.01,-3.38,-2.25,1.38,40.816327
2,BTCUSDT,4h,31,14,21,1.663784,1.531341,-76.389729,30.827586,18,...,-1.56,-5.95,0.04,-0.72,-0.36,-0.49,1.49,-0.68,0.51,36.734694
3,BTCUSDT,4h,34,10,29,1.133459,1.282490,-45.549995,18.742268,24,...,-4.05,-9.26,0.11,-2.29,-3.93,3.99,-8.61,-2.66,1.67,48.979592
4,BTCUSDT,4h,43,11,15,1.396247,1.982624,-65.743008,31.259740,17,...,0.56,-0.13,0.47,0.89,-0.36,0.79,2.70,-1.91,1.05,34.693878
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
36967,INJUSDT,1h,96,5,17,1.833637,1.866539,-0.145921,9.566532,12,...,-5.91,7.30,8.74,6.09,-5.81,-5.34,-6.81,-24.15,0.76,24.489796
36968,INJUSDT,1h,80,5,14,1.627263,2.105363,-66.265195,9.567780,9,...,-13.29,3.08,-5.94,7.53,4.53,0.64,-6.68,-9.85,-1.04,18.367347
36969,INJUSDT,1h,92,5,16,1.441189,2.013496,-27.279900,7.976699,10,...,-4.68,11.01,-1.65,11.60,-5.74,15.02,-7.84,-9.56,5.55,20.408163
36970,INJUSDT,1h,88,5,20,1.952279,1.727292,-20.327936,10.030488,13,...,-18.14,7.59,-6.44,10.61,-5.08,13.29,-9.67,-15.88,6.08,26.530612


In [ ]:
import pandas as pd

# Задаём пользовательский порядок сортировки для interval
interval_order = {'4h': 0, '1h': 1, '15m': 2, '5m': 3, '1m': 4}

# Добавляем новый столбец для пользовательской сортировки
df_filtered_by_month = df_optimized_by_month[df_optimized_by_month['profit_month_ratio%'] > 60].copy()
df_filtered_by_month['interval_order'] = df_filtered_by_month['interval'].map(interval_order)

# Сортировка по символу, interval_order и total_pnl% (по убыванию)
df_sorted_by_month = df_filtered_by_month.sort_values(
    by=['symbol', 'interval_order', 'total_pnl%'],
    ascending=[True, True, False]
).drop(columns=['interval_order'])  # Убираем временный столбец

# Сохраняем в CSV (если нужно)
df_sorted_by_month.to_csv('sorted_data_by_month.csv', index=False, encoding='utf-8')

print("Файл успешно сохранён как sorted_data_by_month.csv")

Файл успешно сохранён как sorted_data_by_month.csv


In [ ]:
df_sorted_by_month

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-02,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%
9233,AAVEUSDT,1h,40,5,20,2.068720,2.569667,799.315500,15.845994,32,...,-143.60,119.04,-281.14,112.37,8.18,-433.23,-103.65,216.17,-36.72,65.306122
9644,AAVEUSDT,1h,38,5,11,1.123961,1.958893,660.249337,6.211949,30,...,-118.28,-86.41,-176.09,122.56,7.54,-181.91,-271.27,45.92,-35.91,61.224490
9243,AAVEUSDT,1h,40,5,17,1.958720,2.074550,612.429635,12.073973,33,...,-132.64,103.98,-214.22,83.10,13.55,-442.18,-198.81,87.61,-43.18,67.346939
9249,AAVEUSDT,1h,40,5,23,2.032958,2.255794,489.564487,13.697967,30,...,-125.79,93.23,-213.14,59.10,18.70,-238.03,-191.29,51.40,10.66,61.224490
9635,AAVEUSDT,1h,40,5,7,1.905312,2.134411,454.493966,11.856115,30,...,-225.09,26.65,-139.98,102.74,-35.30,-225.09,-110.06,95.17,6.12,61.224490
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13697,XRPUSDT,1h,28,5,21,2.082175,2.580568,15.729815,18.663717,30,...,10.20,14.19,3.27,-17.61,-4.91,11.59,-5.27,9.69,3.01,61.224490
14602,XRPUSDT,1h,24,5,17,2.029621,2.223821,7.201157,16.595761,30,...,-16.98,11.40,2.92,-32.50,-16.39,-9.65,-13.19,3.02,4.82,61.224490
15237,XRPUSDT,1h,26,5,13,2.171290,1.633855,-2.098721,13.692363,31,...,-5.29,12.22,7.48,-16.17,-1.30,0.06,3.04,8.99,9.52,63.265306
13686,XRPUSDT,1h,26,5,29,2.300613,2.459552,-12.219113,19.989218,30,...,4.88,18.54,4.18,-22.04,-7.02,-1.54,-17.15,11.59,1.32,61.224490


#### Check

In [ ]:
data = pd.read_excel("final check1.xlsx")
data

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
0,XRPUSDT,1h,42,5,17,1.758816,1.854125,857.496764,11.730132,36,...,161.11,-169.84,-111.03,-40.49,47.25,10.10,37.27,15.07,73.469388,NaN
1,XRPUSDT,1h,42,5,16,2.577894,2.417126,773.604304,24.419664,36,...,202.23,-33.25,-113.22,32.64,41.90,-56.35,85.87,64.58,73.469388,NaN
2,XRPUSDT,1h,42,5,15,1.551276,2.561768,771.772117,13.895495,35,...,208.83,-29.31,-104.29,25.71,115.66,27.84,-26.18,4.81,71.428571,NaN
3,XRPUSDT,1h,42,5,9,1.807181,1.921478,567.091708,12.918033,37,...,138.83,-117.53,-65.91,-37.51,13.67,25.34,24.72,35.32,75.510204,NaN
4,XRPUSDT,1h,42,5,11,1.500657,2.385732,488.832491,12.900783,35,...,137.27,-12.21,-58.83,19.43,47.06,44.79,-27.66,4.31,71.428571,NaN
5,XRPUSDT,1h,44,5,13,1.370583,1.871185,38.842517,8.898667,10,...,3.48,15.94,-14.45,-0.60,8.29,12.36,4.52,-3.44,71.428571,-10.7


In [ ]:
# data.info()

In [ ]:
symbol = data['symbol'].iloc[0]
interval = data['interval'].iloc[0]
start_date = '2023-10'
end_date = '2024-11'

df = download_klines(symbol, interval, start_date, end_date)

for i in range(len(data)):
  print(i+1, '/', (len(data)))

  window_length = data['window_length'].iloc[i]
  min_space = data['min_space'].iloc[i]
  sloping_atr_length = data['sloping_atr_length'].iloc[i]
  take_profit_multiplier = data['take_profit_multiplier'].iloc[i]
  stop_loss_multiplier = data['stop_loss_multiplier'].iloc[i]

  print(f'{symbol, interval, start_date, end_date}, window_length={window_length}, min_space={min_space}, slopes_atr_length={sloping_atr_length}, take_profit_multiplier={take_profit_multiplier}, stop_loss_multiplier={stop_loss_multiplier}')
  manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
    )
    # Если нужно сохранить результаты в CSV
  manual_results_df = pd.DataFrame([manual_results])
  manual_results_df
  # manual_results_df.to_csv("manual_backtest_results.csv", index=False)


  test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
  res = test.run()
  print(symbol)
  print()
  print("Результаты с оптимизированными параметрами:", res)
  print('---' * 30)


1 / 6
('XRPUSDT', '1h', '2023-10', '2024-11'), window_length=42, min_space=5, slopes_atr_length=17, take_profit_multiplier=1.75881584391403, stop_loss_multiplier=1.85412540177548


<ipython-input-30-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для XRPUSDT (1h):
Общее количество сделок: 345
Общий профит: 6.09%, Средняя длительность сделки: 11.88 ч.
Профитные месяцы: 8, Убыточные месяцы: 6
Средний профит в месяц: 7.22%, Средний убыток в месяц: -8.61%
Максимальный профит в месяц: 19.32%, Минимальный профит в месяц: -20.36%
Общий winrate: 52.46%, Winrate по месяцам: 57.14%

Анализ по месяцам:
2023-10: PnL = -0.72%, Сделок = 22
2023-11: PnL = 9.74%, Сделок = 22
2023-12: PnL = -3.74%, Сделок = 20
2024-01: PnL = 1.20%, Сделок = 26
2024-02: PnL = 14.38%, Сделок = 27
2024-03: PnL = 19.32%, Сделок = 29
2024-04: PnL = -20.36%, Сделок = 26
2024-05: PnL = -13.31%, Сделок = 29
2024-06: PnL = -4.85%, Сделок = 25
2024-07: PnL = 5.66%, Сделок = 22
2024-08: PnL = 1.21%, Сделок = 23
2024-09: PnL = 4.47%, Сделок = 31
2024-10: PnL = 1.81%, Сделок = 22
2024-11: PnL = -8.70%, Сделок = 21
XRPUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End                       2024-11-30 23:00

<ipython-input-30-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для XRPUSDT (1h):
Общее количество сделок: 239
Общий профит: 20.82%, Средняя длительность сделки: 24.58 ч.
Профитные месяцы: 10, Убыточные месяцы: 4
Средний профит в месяц: 9.31%, Средний убыток в месяц: -18.06%
Максимальный профит в месяц: 36.27%, Минимальный профит в месяц: -35.87%
Общий winrate: 51.46%, Winrate по месяцам: 71.43%

Анализ по месяцам:
2023-10: PnL = 2.20%, Сделок = 14
2023-11: PnL = 3.26%, Сделок = 16
2023-12: PnL = 2.93%, Сделок = 18
2024-01: PnL = 7.28%, Сделок = 21
2024-02: PnL = 0.78%, Сделок = 21
2024-03: PnL = 36.27%, Сделок = 17
2024-04: PnL = -5.96%, Сделок = 19
2024-05: PnL = -20.31%, Сделок = 21
2024-06: PnL = 5.85%, Сделок = 15
2024-07: PnL = 7.51%, Сделок = 19
2024-08: PnL = -10.11%, Сделок = 14
2024-09: PnL = 15.40%, Сделок = 20
2024-10: PnL = 11.58%, Сделок = 11
2024-11: PnL = -35.87%, Сделок = 13
XRPUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End                       2024-11-30 23

...


Результаты бэктеста для XRPUSDT (1h):
Общее количество сделок: 329
Общий профит: 13.98%, Средняя длительность сделки: 13.49 ч.
Профитные месяцы: 8, Убыточные месяцы: 6
Средний профит в месяц: 7.98%, Средний убыток в месяц: -8.31%
Максимальный профит в месяц: 24.87%, Минимальный профит в месяц: -21.05%
Общий winrate: 53.80%, Winrate по месяцам: 57.14%

Анализ по месяцам:
2023-10: PnL = -0.76%, Сделок = 22
2023-11: PnL = 14.95%, Сделок = 22
2023-12: PnL = -4.01%, Сделок = 21
2024-01: PnL = 2.41%, Сделок = 27
2024-02: PnL = 3.87%, Сделок = 26
2024-03: PnL = 24.87%, Сделок = 26
2024-04: PnL = -21.05%, Сделок = 26
2024-05: PnL = -11.81%, Сделок = 26
2024-06: PnL = -6.72%, Сделок = 20
2024-07: PnL = 2.45%, Сделок = 22
2024-08: PnL = 4.54%, Сделок = 23
2024-09: PnL = 4.43%, Сделок = 27
2024-10: PnL = 6.33%, Сделок = 23
2024-11: PnL = -5.51%, Сделок = 18
XRPUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End                       2024-11-30 23:0

<ipython-input-30-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для XRPUSDT (1h):
Общее количество сделок: 331
Общий профит: 36.85%, Средняя длительность сделки: 12.98 ч.
Профитные месяцы: 8, Убыточные месяцы: 6
Средний профит в месяц: 12.35%, Средний убыток в месяц: -10.32%
Максимальный профит в месяц: 36.77%, Минимальный профит в месяц: -20.86%
Общий winrate: 63.14%, Winrate по месяцам: 57.14%

Анализ по месяцам:
2023-10: PnL = 2.04%, Сделок = 25
2023-11: PnL = 14.93%, Сделок = 22
2023-12: PnL = -8.00%, Сделок = 21
2024-01: PnL = -6.65%, Сделок = 25
2024-02: PnL = 14.10%, Сделок = 27
2024-03: PnL = 36.77%, Сделок = 24
2024-04: PnL = -3.27%, Сделок = 28
2024-05: PnL = -15.76%, Сделок = 26
2024-06: PnL = 5.20%, Сделок = 20
2024-07: PnL = 12.61%, Сделок = 25
2024-08: PnL = 12.00%, Сделок = 24
2024-09: PnL = -7.41%, Сделок = 26
2024-10: PnL = 1.15%, Сделок = 21
2024-11: PnL = -20.86%, Сделок = 17
XRPUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End                       2024-11-30

<ipython-input-30-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для XRPUSDT (1h):
Общее количество сделок: 375
Общий профит: 38.84%, Средняя длительность сделки: 8.90 ч.
Профитные месяцы: 10, Убыточные месяцы: 4
Средний профит в месяц: 6.80%, Средний убыток в месяц: -7.30%
Максимальный профит в месяц: 15.94%, Минимальный профит в месяц: -14.45%
Общий winrate: 61.33%, Winrate по месяцам: 71.43%

Анализ по месяцам:
2023-10: PnL = 2.16%, Сделок = 26
2023-11: PnL = 5.59%, Сделок = 22
2023-12: PnL = 4.63%, Сделок = 27
2024-01: PnL = 0.55%, Сделок = 28
2024-02: PnL = 10.49%, Сделок = 29
2024-03: PnL = 3.48%, Сделок = 27
2024-04: PnL = 15.94%, Сделок = 28
2024-05: PnL = -14.45%, Сделок = 32
2024-06: PnL = -0.60%, Сделок = 26
2024-07: PnL = 8.29%, Сделок = 23
2024-08: PnL = 12.36%, Сделок = 24
2024-09: PnL = 4.52%, Сделок = 31
2024-10: PnL = -3.44%, Сделок = 28
2024-11: PnL = -10.70%, Сделок = 24
XRPUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End                       2024-11-30 23:00

In [ ]:
window_length = 98
min_space = 5
sloping_atr_length = 7
take_profit_multiplier = 1.661
stop_loss_multiplier = 1.834

manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
)
# Если нужно сохранить результаты в CSV
manual_results_df = pd.DataFrame([manual_results])
manual_results_df
# manual_results_df.to_csv("manual_backtest_results.csv", index=False)


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

<ipython-input-30-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для 1000PEPEUSDT (1h):
Общее количество сделок: 248
Общий профит: 367.21%, Средняя длительность сделки: 10.61 ч.
Профитные месяцы: 11, Убыточные месяцы: 3
Средний профит в месяц: 68.05%, Средний убыток в месяц: -127.12%
Максимальный профит в месяц: 197.65%, Минимальный профит в месяц: -334.90%
Общий winrate: 60.48%, Winrate по месяцам: 78.57%

Анализ по месяцам:
2023-10: PnL = 7.15%, Сделок = 18
2023-11: PnL = 16.29%, Сделок = 16
2023-12: PnL = 7.88%, Сделок = 15
2024-01: PnL = 46.17%, Сделок = 16
2024-02: PnL = 37.80%, Сделок = 12
2024-03: PnL = 189.62%, Сделок = 19
2024-04: PnL = -20.56%, Сделок = 20
2024-05: PnL = 197.65%, Сделок = 16
2024-06: PnL = -25.91%, Сделок = 21
2024-07: PnL = 25.20%, Сделок = 14
2024-08: PnL = 24.22%, Сделок = 16
2024-09: PnL = 27.88%, Сделок = 21
2024-10: PnL = 168.73%, Сделок = 22
2024-11: PnL = -334.90%, Сделок = 22
1000PEPEUSDT

Результаты с оптимизированными параметрами: Start                     2023-10-01 00:00:00
End            

In [ ]:
window_length = 76
min_space = 5
sloping_atr_length = 26
take_profit_multiplier = 1.54
stop_loss_multiplier = 1.00


manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
)
# Если нужно сохранить результаты в CSV
manual_results_df = pd.DataFrame([manual_results])
manual_results_df
# manual_results_df.to_csv("manual_backtest_results.csv", index=False)


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

<ipython-input-28-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для FTMUSDT (1h):
Общее количество сделок: 1267
Общий профит: 205.50%, Средняя длительность сделки: 4.60 ч.
Профитные месяцы: 29, Убыточные месяцы: 20
Средний профит в месяц: 22.41%, Средний убыток в месяц: -22.22%
Максимальный профит в месяц: 125.30%, Минимальный профит в месяц: -66.62%
Общий winrate: 43.73%, Winrate по месяцам: 59.18%

Анализ по месяцам:
2020-10: PnL = -3.46%, Сделок = 21
2020-11: PnL = 23.33%, Сделок = 25
2020-12: PnL = -25.07%, Сделок = 29
2021-01: PnL = 44.24%, Сделок = 24
2021-02: PnL = 20.07%, Сделок = 23
2021-03: PnL = 11.53%, Сделок = 19
2021-04: PnL = -49.11%, Сделок = 29
2021-05: PnL = -1.40%, Сделок = 31
2021-06: PnL = 14.32%, Сделок = 30
2021-07: PnL = -17.17%, Сделок = 20
2021-08: PnL = 19.51%, Сделок = 23
2021-09: PnL = -47.74%, Сделок = 24
2021-10: PnL = 2.41%, Сделок = 30
2021-11: PnL = 7.54%, Сделок = 22
2021-12: PnL = 27.29%, Сделок = 23
2022-01: PnL = 26.37%, Сделок = 26
2022-02: PnL = 14.84%, Сделок = 21
2022-03: PnL = 23.30%, 

In [ ]:
manual_results_df.to_csv("manual_backtest_results.csv", index=False)

In [ ]:
window_length = 36
min_space = 5
sloping_atr_length = 23
take_profit_multiplier = 1.006
stop_loss_multiplier = 1.462


manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
)
# Если нужно сохранить результаты в CSV
manual_results_df = pd.DataFrame([manual_results])
manual_results_df
# manual_results_df.to_csv("manual_backtest_results.csv", index=False)


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

<ipython-input-28-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для ATOMUSDT (1h):
Общее количество сделок: 1731
Общий профит: 775.88%, Средняя длительность сделки: 4.49 ч.
Профитные месяцы: 31, Убыточные месяцы: 18
Средний профит в месяц: 50.28%, Средний убыток в месяц: -43.48%
Максимальный профит в месяц: 172.36%, Минимальный профит в месяц: -195.50%
Общий winrate: 62.16%, Winrate по месяцам: 63.27%

Анализ по месяцам:
2020-10: PnL = 6.58%, Сделок = 29
2020-11: PnL = 9.51%, Сделок = 29
2020-12: PnL = -3.61%, Сделок = 33
2021-01: PnL = -19.63%, Сделок = 38
2021-02: PnL = 18.76%, Сделок = 38
2021-03: PnL = 8.17%, Сделок = 33
2021-04: PnL = 56.50%, Сделок = 35
2021-05: PnL = 20.22%, Сделок = 39
2021-06: PnL = 54.67%, Сделок = 38
2021-07: PnL = 29.12%, Сделок = 33
2021-08: PnL = -3.30%, Сделок = 38
2021-09: PnL = 97.20%, Сделок = 36
2021-10: PnL = -15.91%, Сделок = 35
2021-11: PnL = -31.21%, Сделок = 35
2021-12: PnL = 84.50%, Сделок = 36
2022-01: PnL = 91.69%, Сделок = 29
2022-02: PnL = 79.68%, Сделок = 35
2022-03: PnL = 43.57%, 

In [ ]:
symbol = 'APEUSDT'
interval = '30m'
start_date = '2022-05'
end_date = '2024-10'
# Загрузка данных
df = download_klines(symbol, interval, start_date, end_date)

In [ ]:
window_length = 78
min_space =5
sloping_atr_length = 29
take_profit_multiplier = 1.48
stop_loss_multiplier = 2.59


manual_results = manual_backtest(
    data=df,
    strategy=SlopingStrategy,
    symbol=symbol,
    interval=interval,
    start_date=start_date,
    end_date=end_date,
    initial_capital=1_000_000,
    window_length=window_length ,
    min_space=min_space,
    sloping_atr_length=sloping_atr_length,
    take_profit_multiplier=take_profit_multiplier,
    stop_loss_multiplier=stop_loss_multiplier
)
# Если нужно сохранить результаты в CSV
manual_results_df = pd.DataFrame([manual_results])
manual_results_df
# manual_results_df.to_csv("manual_backtest_results.csv", index=False)


test = backtesting.Backtest(data=df, strategy=SlopingStrategy, cash=1_000_000, commission=0.0005)
res = test.run()
print(symbol)
print()
print("Результаты с оптимизированными параметрами:", res)

<ipython-input-28-03ecb89204fb>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  monthly_trades['PnL'] = pd.to_numeric(monthly_trades['PnL'], errors='coerce')



Результаты бэктеста для APEUSDT (30m):
Общее количество сделок: 1156
Общий профит: 190.72%, Средняя длительность сделки: 5.70 ч.
Профитные месяцы: 20, Убыточные месяцы: 10
Средний профит в месяц: 18.89%, Средний убыток в месяц: -18.71%
Максимальный профит в месяц: 43.54%, Минимальный профит в месяц: -34.80%
Общий winrate: 66.52%, Winrate по месяцам: 66.67%

Анализ по месяцам:
2022-05: PnL = 31.90%, Сделок = 30
2022-06: PnL = 1.26%, Сделок = 43
2022-07: PnL = 18.58%, Сделок = 35
2022-08: PnL = -13.99%, Сделок = 37
2022-09: PnL = 18.25%, Сделок = 42
2022-10: PnL = -8.26%, Сделок = 43
2022-11: PnL = 7.27%, Сделок = 31
2022-12: PnL = 1.23%, Сделок = 38
2023-01: PnL = 21.11%, Сделок = 43
2023-02: PnL = 19.84%, Сделок = 38
2023-03: PnL = 24.25%, Сделок = 41
2023-04: PnL = -1.03%, Сделок = 31
2023-05: PnL = -20.76%, Сделок = 37
2023-06: PnL = 40.98%, Сделок = 38
2023-07: PnL = -34.80%, Сделок = 39
2023-08: PnL = 11.53%, Сделок = 38
2023-09: PnL = -8.51%, Сделок = 32
2023-10: PnL = -31.57%, С

### Find best combinations

#### MAX Profit

In [ ]:
import pandas as pd

# Считаем данные из файла
file_path_by_month = '/content/drive/MyDrive/Sloping bot/hyperopt_optimization_results.csv'
df_optimized_by_month = pd.read_csv(file_path_by_month)
df_optimized_by_month

# Удалим дубликаты, если они есть
df_optimized_by_month.drop_duplicates(inplace=True)
df_optimized_by_month.reset_index(drop=True, inplace=True)

file_name = "hyperopt_optimization_results.csv"
df_optimized_by_month.to_csv(file_name, index=False)

df_optimized_by_month.info()


# Задаём пользовательский порядок сортировки для interval
interval_order = {'4h': 0, '1h': 1, '15m': 2, '5m': 3, '1m': 4}

# Добавляем новый столбец для пользовательской сортировки
df_filtered_by_month = df_optimized_by_month[df_optimized_by_month['profit_month_ratio%'] > 60].copy()
df_filtered_by_month['interval_order'] = df_filtered_by_month['interval'].map(interval_order)

# Сортировка по символу, interval_order и total_pnl% (по убыванию)
df_sorted_by_month = df_filtered_by_month.sort_values(
    by=['symbol', 'interval_order', 'total_pnl%'],
    ascending=[True, True, False]
).drop(columns=['interval_order'])  # Убираем временный столбец

# Сохраняем в CSV (если нужно)
df_sorted_by_month.to_csv('sorted_data_by_month.csv', index=False, encoding='utf-8')

print("Файл успешно сохранён как sorted_data_by_month.csv")
df_sorted_by_month

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47272 entries, 0 to 47271
Data columns (total 62 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   symbol                    47272 non-null  object 
 1   interval                  47272 non-null  object 
 2   window_length             47272 non-null  int64  
 3   min_space                 47272 non-null  int64  
 4   sloping_atr_length        47272 non-null  int64  
 5   take_profit_multiplier    47272 non-null  float64
 6   stop_loss_multiplier      47272 non-null  float64
 7   total_pnl%                47272 non-null  float64
 8   avg_trade_duration_hours  47272 non-null  float64
 9   profit_month_count        47272 non-null  int64  
 10  loss_month_count          47272 non-null  int64  
 11  2020-10                   38172 non-null  float64
 12  2020-11                   38172 non-null  float64
 13  2020-12                   38172 non-null  float64
 14  2021-0

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
21771,1000PEPEUSDT,1h,98,5,12,1.817388,1.809748,700.797272,10.322148,11,...,175.23,-12.53,163.23,-1.43,34.16,53.40,34.77,168.92,64.705882,NaN
18946,1000PEPEUSDT,1h,98,5,7,1.693179,2.360533,683.638414,12.476868,11,...,214.06,-85.78,188.20,-16.64,19.30,-14.33,18.94,184.99,64.705882,NaN
21736,1000PEPEUSDT,1h,98,5,7,1.661477,1.834137,629.728243,10.239203,12,...,172.45,-18.66,179.80,-23.53,22.95,22.06,25.40,153.57,70.588235,NaN
18891,1000PEPEUSDT,1h,92,5,7,1.457082,2.430666,621.659050,11.260000,14,...,178.86,-89.45,47.38,78.15,78.64,3.72,18.61,155.90,82.352941,NaN
21806,1000PEPEUSDT,1h,98,5,10,1.484407,2.238799,612.382192,9.550000,14,...,194.50,-66.93,89.04,18.54,18.71,22.76,26.43,160.70,82.352941,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14602,XRPUSDT,1h,24,5,17,2.029621,2.223821,7.201157,16.595761,30,...,11.40,2.92,-32.50,-16.39,-9.65,-13.19,3.02,4.82,61.224490,NaN
15237,XRPUSDT,1h,26,5,13,2.171290,1.633855,-2.098721,13.692363,31,...,12.22,7.48,-16.17,-1.30,0.06,3.04,8.99,9.52,63.265306,NaN
13686,XRPUSDT,1h,26,5,29,2.300613,2.459552,-12.219113,19.989218,30,...,18.54,4.18,-22.04,-7.02,-1.54,-17.15,11.59,1.32,61.224490,NaN
16577,XRPUSDT,1h,24,5,8,1.910206,1.957664,-19.014306,14.184783,30,...,6.64,14.10,-23.00,-4.72,-12.45,0.56,1.90,4.72,60.416667,NaN


In [ ]:
import pandas as pd
from itertools import combinations

# df = pd.read_excel("sloping final results.xlsx")
# df = pd.read_csv("sorted_data_by_month (3).csv")


df = df_sorted_by_month
# 2. Создание уникальных комбинаций "монета-интервал"
df["coin_interval"] = df["symbol"] + "_" + df["interval"]

# 3. Определение списка уникальных "монета-интервал"
coin_intervals = df["coin_interval"].unique()

# 4. Параметры анализа
combination_sizes = range(1, len(coin_intervals) + 1)  # Размеры комбинаций (от 1 до числа уникальных записей)
top_n = 10  # Количество лучших комбинаций для вывода

# 5. Список столбцов с ежемесячными профитами
monthly_columns = [col for col in df.columns if col.startswith("202")]

# 6. Перебор комбинаций
results = []
for size in combination_sizes:
    for combo in combinations(coin_intervals, size):
        # Проверка, что в комбинации каждая монета используется только один раз
        symbols = [item.split("_")[0] for item in combo]
        if len(symbols) != len(set(symbols)):  # Если есть дублирующиеся символы, пропускаем
            continue

        # Фильтрация данных по выбранным "монета-интервал"
        filtered_df = df[df["coin_interval"].isin(combo)]

        # Суммарный профит за год для комбинации
        total_pnl = filtered_df[monthly_columns].sum().sum()

        # Месячный профит (средний)
        avg_monthly_pnl = total_pnl / len(monthly_columns)

        # Минимальный месячный профит (убыток)
        min_monthly_pnl = filtered_df[monthly_columns].sum(axis=0).min()

        # PnL по месяцам
        monthly_pnl = filtered_df[monthly_columns].sum(axis=0).to_dict()

        # Сохранение результата
        results.append({
            "combo": combo,
            "total_pnl": total_pnl,
            "avg_monthly_pnl": avg_monthly_pnl,
            "min_monthly_pnl": min_monthly_pnl,
            "monthly_pnl": monthly_pnl,
        })

# 7. Сортировка комбинаций по общему профиту
sorted_results = sorted(results, key=lambda x: x["total_pnl"], reverse=True)

# 8. Вывод лучших комбинаций
print(f"Топ-{top_n} комбинаций:")
for i, result in enumerate(sorted_results[:top_n], 1):
    combo = ", ".join(result["combo"])
    print(f"{i}. Комбинация: {combo}")
    print(f"   Общий профит: {result['total_pnl']:.2f}%")
    print(f"   Средний месячный профит: {result['avg_monthly_pnl']:.2f}%")
    print(f"   Минимальный месячный профит (убыток): {result['min_monthly_pnl']:.2f}%")
    print(f"   PnL по месяцам:")
    for month, pnl in result["monthly_pnl"].items():
        print(f"      {month}: {pnl:.2f}%")



Топ-10 комбинаций:
1. Комбинация: ADAUSDT_1h, BTCUSDT_1h, FTMUSDT_1h, LINKUSDT_1h, XRPUSDT_1h
   Общий профит: 21175.27%
   Средний месячный профит: 1628.87%
   Минимальный месячный профит (убыток): -5202.81%
   PnL по месяцам:
      2023-10: -2422.86%
      2023-11: 3705.05%
      2023-12: 1436.60%
      2024-01: 693.46%
      2024-02: 287.22%
      2024-03: 14473.47%
      2024-04: -3199.81%
      2024-05: -5202.81%
      2024-06: 2518.86%
      2024-07: 3621.44%
      2024-08: -473.98%
      2024-09: 2961.99%
      2024-10: 2776.64%
2. Комбинация: ADAUSDT_1h, ATOMUSDT_1h, BTCUSDT_1h, FTMUSDT_1h, LINKUSDT_1h, XRPUSDT_1h
   Общий профит: 21126.79%
   Средний месячный профит: 1625.14%
   Минимальный месячный профит (убыток): -5235.45%
   PnL по месяцам:
      2023-10: -2547.12%
      2023-11: 3906.23%
      2023-12: 1894.63%
      2024-01: 872.83%
      2024-02: 208.59%
      2024-03: 14595.40%
      2024-04: -3342.15%
      2024-05: -5235.45%
      2024-06: 2590.48%
      2024-07: 367

#### MIN Risks

In [ ]:
import pandas as pd
from itertools import combinations

# Загрузка данных
file_path_by_month = '/content/drive/MyDrive/Sloping bot/hyperopt_optimization_results.csv'
df = pd.read_csv(file_path_by_month)

# Удаление дубликатов
df.drop_duplicates(inplace=True)
df.reset_index(drop=True, inplace=True)
df = df[df['profit_month_count'] > 1].copy()

# Определяем столбцы с данными по месяцам
monthly_columns = [col for col in df.columns if col.startswith("202")]

# Фильтрация: топ-10 по `total_pnl%` и `profit_month_ratio%` для каждой монеты
# Топ-10 по `total_pnl%`
top_total_pnl = (
    df.sort_values(by="total_pnl%", ascending=False)
    .groupby("symbol")
    .head(10)
)

# Топ-10 по `profit_month_ratio%`
top_month_ratio = (
    df.sort_values(by="profit_month_ratio%", ascending=False)
    .groupby("symbol")
    .head(10)
)

# Объединение и удаление дубликатов
filtered_top = pd.concat([top_total_pnl, top_month_ratio]).drop_duplicates()

# Финальная сортировка для удобства
filtered_top = filtered_top.sort_values(
    by=["symbol", "interval", "total_pnl%"],
    ascending=[True, True, False]
)

filtered_top = filtered_top[(filtered_top['profit_month_ratio%'] > 70) & (filtered_top['min_space'] == 5)].copy()
# Сохранение предварительно отфильтрованных данных
filtered_top.to_csv("filtered_top_models.csv", index=False)
print("Файл успешно сохранён как filtered_top_models.csv")
filtered_top

Файл успешно сохранён как filtered_top_models.csv


,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
21736,1000PEPEUSDT,1h,98,5,7,1.661477,1.834137,629.728243,10.239203,12,...,172.45,-18.66,179.80,-23.53,22.95,22.06,25.40,153.57,70.588235,NaN
18891,1000PEPEUSDT,1h,92,5,7,1.457082,2.430666,621.659050,11.260000,14,...,178.86,-89.45,47.38,78.15,78.64,3.72,18.61,155.90,82.352941,NaN
21806,1000PEPEUSDT,1h,98,5,10,1.484407,2.238799,612.382192,9.550000,14,...,194.50,-66.93,89.04,18.54,18.71,22.76,26.43,160.70,82.352941,NaN
19285,1000PEPEUSDT,1h,94,5,11,1.481311,2.416515,510.363391,10.325342,14,...,117.17,-25.59,39.09,92.14,45.53,15.26,11.88,103.98,82.352941,NaN
19274,1000PEPEUSDT,1h,96,5,11,1.482654,2.223942,473.693979,9.178218,14,...,129.53,-35.89,30.38,6.43,24.20,34.73,18.39,155.59,82.352941,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15101,XRPUSDT,1h,42,5,17,1.758816,1.854125,857.496764,11.730132,36,...,161.11,-169.84,-111.03,-40.49,47.25,10.10,37.27,15.07,73.469388,NaN
15883,XRPUSDT,1h,42,5,16,2.577894,2.417126,773.604304,24.419664,36,...,202.23,-33.25,-113.22,32.64,41.90,-56.35,85.87,64.58,73.469388,NaN
14582,XRPUSDT,1h,42,5,15,1.551276,2.561768,771.772117,13.895495,35,...,208.83,-29.31,-104.29,25.71,115.66,27.84,-26.18,4.81,71.428571,NaN
11262,XRPUSDT,1h,42,5,9,1.807181,1.921478,567.091708,12.918033,37,...,138.83,-117.53,-65.91,-37.51,13.67,25.34,24.72,35.32,75.510204,NaN


In [ ]:
# скачиваем все модели с диска

# import pandas as pd

# # Считаем данные из файла
# file_path_by_month = '/content/drive/MyDrive/Sloping bot/hyperopt_optimization_results.csv'
# df_optimized_by_month = pd.read_csv(file_path_by_month)
# df_optimized_by_month

# # Удалим дубликаты, если они есть
# df_optimized_by_month.drop_duplicates(inplace=True)
# df_optimized_by_month.reset_index(drop=True, inplace=True)

# file_name = "hyperopt_optimization_results.csv"
# df_optimized_by_month.to_csv(file_name, index=False)

# df_optimized_by_month.info()


# # Задаём пользовательский порядок сортировки для interval
# interval_order = {'4h': 0, '1h': 1, '15m': 2, '5m': 3, '1m': 4}

# # Добавляем новый столбец для пользовательской сортировки
# df_filtered_by_month = df_optimized_by_month[df_optimized_by_month['profit_month_ratio%'] > 70].copy()
# df_filtered_by_month['interval_order'] = df_filtered_by_month['interval'].map(interval_order)

# # Сортировка по символу, interval_order и total_pnl% (по убыванию)
# df_sorted_by_month = df_filtered_by_month.sort_values(
#     by=['symbol', 'interval_order', 'total_pnl%'],
#     ascending=[True, True, False]
# ).drop(columns=['interval_order'])  # Убираем временный столбец

# # Сохраняем в CSV (если нужно)
# df_sorted_by_month.to_csv('sorted_data_by_month.csv', index=False, encoding='utf-8')

# print("Файл успешно сохранён как sorted_data_by_month.csv")
# df_sorted_by_month

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50172 entries, 0 to 50171
Data columns (total 62 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   symbol                    50172 non-null  object 
 1   interval                  50172 non-null  object 
 2   window_length             50172 non-null  int64  
 3   min_space                 50172 non-null  int64  
 4   sloping_atr_length        50172 non-null  int64  
 5   take_profit_multiplier    50172 non-null  float64
 6   stop_loss_multiplier      50172 non-null  float64
 7   total_pnl%                50172 non-null  float64
 8   avg_trade_duration_hours  50172 non-null  float64
 9   profit_month_count        50172 non-null  int64  
 10  loss_month_count          50172 non-null  int64  
 11  2020-10                   39722 non-null  float64
 12  2020-11                   39722 non-null  float64
 13  2020-12                   39722 non-null  float64
 14  2021-0

,symbol,interval,window_length,min_space,sloping_atr_length,take_profit_multiplier,stop_loss_multiplier,total_pnl%,avg_trade_duration_hours,profit_month_count,...,2024-03,2024-04,2024-05,2024-06,2024-07,2024-08,2024-09,2024-10,profit_month_ratio%,2024-11
21736,1000PEPEUSDT,1h,98,5,7,1.661477,1.834137,629.728243,10.239203,12,...,172.45,-18.66,179.80,-23.53,22.95,22.06,25.40,153.57,70.588235,NaN
18891,1000PEPEUSDT,1h,92,5,7,1.457082,2.430666,621.659050,11.260000,14,...,178.86,-89.45,47.38,78.15,78.64,3.72,18.61,155.90,82.352941,NaN
21806,1000PEPEUSDT,1h,98,5,10,1.484407,2.238799,612.382192,9.550000,14,...,194.50,-66.93,89.04,18.54,18.71,22.76,26.43,160.70,82.352941,NaN
19264,1000PEPEUSDT,1h,92,5,8,1.089490,2.507078,612.128629,7.924699,14,...,139.73,-36.66,9.18,59.87,51.82,49.91,45.29,164.63,82.352941,NaN
18704,1000PEPEUSDT,1h,72,5,10,1.418003,2.246889,606.723701,9.440111,12,...,120.03,-57.45,6.45,134.64,143.95,35.89,34.10,98.77,70.588235,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15101,XRPUSDT,1h,42,5,17,1.758816,1.854125,857.496764,11.730132,36,...,161.11,-169.84,-111.03,-40.49,47.25,10.10,37.27,15.07,73.469388,NaN
15883,XRPUSDT,1h,42,5,16,2.577894,2.417126,773.604304,24.419664,36,...,202.23,-33.25,-113.22,32.64,41.90,-56.35,85.87,64.58,73.469388,NaN
14582,XRPUSDT,1h,42,5,15,1.551276,2.561768,771.772117,13.895495,35,...,208.83,-29.31,-104.29,25.71,115.66,27.84,-26.18,4.81,71.428571,NaN
11262,XRPUSDT,1h,42,5,9,1.807181,1.921478,567.091708,12.918033,37,...,138.83,-117.53,-65.91,-37.51,13.67,25.34,24.72,35.32,75.510204,NaN


In [ ]:
import numpy as np
import pandas as pd
from itertools import combinations, product
from tqdm import tqdm
from google.colab import drive
import os
import shutil


# filtered_top = pd.read_excel("final models.xlsx")


# Подключение Google Drive
drive.mount('/content/drive')
shared_folder_path = '/content/drive/MyDrive/Sloping bot'

# Локальный путь для сохранения
local_file_name = "final_combos.csv"
drive_file_name = "final_combos.csv"

# Загрузка данных
# df = pd.read_csv("filtered_top_models (4).csv")
# df = pd.read_excel("df.xlsx")
# df = filtered_top

df = pd.read_excel("final models.xlsx")

# Параметры ввода
start_date = "2023-10"
end_date = "2024-11"

# Параметры фильтрации
max_loss_threshold = -10  # Заданный порог убытка
max_symbols = 7          # Максимальное количество монет
results = []



# Генерация списка месячных колонок
date_range = pd.date_range(start=start_date, end=end_date, freq='MS')  # MS = начало месяца
monthly_columns = [date.strftime("%Y-%m") for date in date_range]

# Проверка на пропуски и замена NaN
df[monthly_columns] = df[monthly_columns].replace("", np.nan).astype(float)

# Обработка всех комбинаций
for size in range(1, min(max_symbols, len(df['symbol'].unique())) + 1):
    for symbol_combo in combinations(df['symbol'].unique(), size):
        models_combo = [df[df['symbol'] == symbol].to_dict(orient="records") for symbol in symbol_combo]
        parameter_combinations = product(*models_combo)

        # Убираем лишние сообщения tqdm
        for combo in tqdm(parameter_combinations, desc=f"Processing {size}-coin combinations", leave=False):
            total_pnl = sum(item['total_pnl%'] for item in combo)
            avg_monthly_pnl = total_pnl / len(monthly_columns)
            monthly_pnl = {col: sum(item[col] for item in combo if not pd.isna(item[col])) for col in monthly_columns}
            min_monthly_pnl = min(monthly_pnl.values(), default=0)

            # Условия фильтрации
            if min_monthly_pnl < max_loss_threshold:
                continue

            # Форматирование параметров моделей
            models_params = "; ".join([
                f"{item['symbol']}_{item['interval']} "
                f"(window={item['window_length']}, atr_length={item['sloping_atr_length']}, "
                f"min_space={item['min_space']}, take={item['take_profit_multiplier']}, loss={item['stop_loss_multiplier']})"
                for item in combo
            ])

            results.append({
                "combo": ", ".join(symbol_combo),
                "total_pnl": total_pnl,
                "avg_monthly_pnl": avg_monthly_pnl,
                "min_monthly_pnl": min_monthly_pnl,
                **monthly_pnl,
                "models_params": models_params
            })

# Сортировка результатов по общему профиту
results_df = pd.DataFrame(results)
top_results = results_df.sort_values(by="total_pnl", ascending=False).head(10)

# Сохранение с вашей функцией
if not top_results.empty:
    save_results_to_csv(top_results, shared_folder_path, drive_file_name)  # На Google Drive
    save_results_to_csv(top_results, ".", local_file_name)  # Локально
else:
    print("Не найдено подходящих комбинаций!")

# Вывод результатов в консоль
if not top_results.empty:
    for i, result in enumerate(top_results.to_dict(orient="records"), start=1):
        print('---' * 25)
        print(f"{i}. Комбинация: {result['combo']}")
        print(f"   Общий профит: {result['total_pnl']:.2f}%")
        print(f"   Средний месячный профит: {result['avg_monthly_pnl']:.2f}%")
        print(f"   Минимальный месячный профит: {result['min_monthly_pnl']:.2f}%")
        print(f"   PnL по месяцам:")
        for month in monthly_columns:
            if month in result:
                print(f"      {month}: {result[month]:.2f}%")
        print(f"   Параметры моделей: {result['models_params']}")
        print()  # Пустая строка между комбинациями
else:
    print("Не найдено подходящих комбинаций!")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Резервная копия создана: /content/drive/MyDrive/Sloping bot/final_combos.csv.backup
Обновленные данные сохранены в /content/drive/MyDrive/Sloping bot/final_combos.csv
Файл не существует, будет создан новый файл: ./final_combos.csv
Обновленные данные сохранены в ./final_combos.csv
---------------------------------------------------------------------------
1. Комбинация: 1000PEPEUSDT, APEUSDT, LINKUSDT, ORDIUSDT, SOLUSDT, WIFUSDT, WLDUSDT
   Общий профит: 2570.76%
   Средний месячный профит: 183.63%
   Минимальный месячный профит: 18.80%
   PnL по месяцам:
      2023-10: 18.80%
      2023-11: 220.52%
      2023-12: 26.78%
      2024-01: 78.75%
      2024-02: 73.96%
      2024-03: 159.58%
      2024-04: 180.41%
      2024-05: 151.92%
      2024-06: 247.32%
      2024-07: 306.81%
      2024-08: 281.30%
      2024-09: 115.11%
      2024-10: 477.22%
      2024-11: 288.68%
   Параметры моделей: 1000PEPEUSDT_30m (window=86, atr_length=18, min_space=5, take=1.80623414832759, loss=2.599026140883

In [ ]:
# import numpy as np
# import pandas as pd
# from itertools import combinations, product
# from tqdm import tqdm
# from pandas.tseries.offsets import MonthBegin

# # Загрузка данных
# df = pd.read_excel("df.xlsx")
# # df = pd.read_csv("filtered_top_models (4).csv")
# # Параметры ввода
# start_date = "2023-10"
# end_date = "2024-11"

# # Генерация списка месячных колонок
# date_range = pd.date_range(start=start_date, end=end_date, freq='MS')  # MS = начало месяца
# monthly_columns = [date.strftime("%Y-%m") for date in date_range]

# # Проверка на пропуски и замена NaN
# df[monthly_columns] = df[monthly_columns].replace("", np.nan).astype(float)

# # Параметры фильтрации
# max_loss_threshold = -10  # Заданный порог убытка
# max_symbols = 3           # Максимальное количество монет
# results = []

# # Проходим по всем возможным комбинациям до 5 символов
# for size in range(1, min(max_symbols, len(df['symbol'].unique())) + 1):
#     for symbol_combo in combinations(df['symbol'].unique(), size):
#         # print(f"Обрабатываем комбинацию монет: {symbol_combo}")

#         models_combo = [df[df['symbol'] == symbol].to_dict(orient="records") for symbol in symbol_combo]
#         parameter_combinations = product(*models_combo)

#         for combo in tqdm(parameter_combinations): #desc="Processing combinations"
#             total_pnl = sum(item['total_pnl%'] for item in combo)
#             avg_monthly_pnl = total_pnl / len(monthly_columns)
#             monthly_pnl = {col: sum(item[col] for item in combo if not pd.isna(item[col])) for col in monthly_columns}
#             min_monthly_pnl = min(monthly_pnl.values(), default=0)

#             # Условия фильтрации
#             if min_monthly_pnl < max_loss_threshold:
#                 continue

#             results.append({
#                 "combo": symbol_combo,
#                 "total_pnl": total_pnl,
#                 "avg_monthly_pnl": avg_monthly_pnl,
#                 "min_monthly_pnl": min_monthly_pnl,
#                 "monthly_pnl": monthly_pnl,
#                 "models_params": [{**item, "coin_interval": f"{item['symbol']}_{item['interval']}"} for item in combo]
#             })

# # Сортировка результатов по общему профиту и выбор топ-10
# top_results = sorted(results, key=lambda x: x["total_pnl"], reverse=True)[:10]

# # Форматированный вывод результатов
# if not top_results:
#     print("Не найдено подходящих комбинаций!")
# else:
#     for i, result in enumerate(top_results, start=1):
#         print(f"{i}. Комбинация: {', '.join(['{}_{}'.format(symbol, item['interval']) for symbol, item in zip(result['combo'], result['models_params'])])}")
#         print(f"   Общий профит: {result['total_pnl']:.2f}%")
#         print(f"   Средний месячный профит: {result['avg_monthly_pnl']:.2f}%")
#         print(f"   Минимальный месячный профит: {result['min_monthly_pnl']:.2f}%")
#         print(f"   PnL по месяцам:")
#         for month, pnl in result['monthly_pnl'].items():
#             print(f"      {month}: {pnl:.2f}%")
#         print(f"   Параметры моделей в комбинации:")
#         for params in result['models_params']:
#             print(f"      {params}")
#         print()  # Пустая строка между комбинациями

In [ ]:
# import numpy as np
# import pandas as pd
# from itertools import combinations, product
# from tqdm import tqdm
# from pandas.tseries.offsets import MonthBegin

# # Загрузка данных
# df = pd.read_excel("df.xlsx")
# # df = filtered_top
# # Параметры ввода
# start_date = "2023-10"
# end_date = "2024-11"

# # Генерация списка месячных колонок
# date_range = pd.date_range(start=start_date, end=end_date, freq='MS')  # MS = начало месяца
# monthly_columns = [date.strftime("%Y-%m") for date in date_range]

# # Проверка на пропуски и замена NaN
# df[monthly_columns] = df[monthly_columns].replace("", np.nan).astype(float)

# # Параметры фильтрации
# max_loss_threshold = -10 # Заданный порог убытка
# results = []

# # Проходим по всем возможным комбинациям
# for size in range(1, len(df['symbol'].unique()) + 1):
#     for symbol_combo in combinations(df['symbol'].unique(), size):
#         print(f"Обрабатываем комбинацию монет: {symbol_combo}")

#         models_combo = [df[df['symbol'] == symbol].to_dict(orient="records") for symbol in symbol_combo]
#         parameter_combinations = product(*models_combo)

#         for combo in tqdm(parameter_combinations, desc="Processing combinations"):
#             total_pnl = sum(item['total_pnl%'] for item in combo)
#             avg_monthly_pnl = total_pnl / len(monthly_columns)
#             monthly_pnl = {col: sum(item[col] for item in combo if not pd.isna(item[col])) for col in monthly_columns}
#             min_monthly_pnl = min(monthly_pnl.values(), default=0)

#             # Условия фильтрации
#             if min_monthly_pnl < max_loss_threshold:
#                 continue

#             results.append({
#                 "combo": symbol_combo,
#                 "total_pnl": total_pnl,
#                 "avg_monthly_pnl": avg_monthly_pnl,
#                 "min_monthly_pnl": min_monthly_pnl,
#                 "monthly_pnl": monthly_pnl,
#                 "models_params": [{**item, "coin_interval": f"{item['symbol']}_{item['interval']}"} for item in combo]
#             })

# # Сортировка результатов по общему профиту и выбор топ-10
# top_results = sorted(results, key=lambda x: x["total_pnl"], reverse=True)[:10]

# # Форматированный вывод результатов
# if not top_results:
#     print("Не найдено подходящих комбинаций!")
# else:
#     for i, result in enumerate(top_results, start=1):
#         print(f"{i}. Комбинация: {', '.join(['{}_{}'.format(symbol, item['interval']) for symbol, item in zip(result['combo'], result['models_params'])])}")
#         print(f"   Общий профит: {result['total_pnl']:.2f}%")
#         print(f"   Средний месячный профит: {result['avg_monthly_pnl']:.2f}%")
#         print(f"   Минимальный месячный профит: {result['min_monthly_pnl']:.2f}%")
#         print(f"   PnL по месяцам:")
#         for month, pnl in result['monthly_pnl'].items():
#             print(f"      {month}: {pnl:.2f}%")
#         print(f"   Параметры моделей в комбинации:")
#         for params in result['models_params']:
#             print(f"      {params}")
#         print()  # Пустая строка между комбинациями




#### final Backtest

###### Тест 8/12/2024

In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest, Strategy


# Класс стратегии для backtesting
class SlopingStrategy(Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5
    take_profit_multiplier = 1.5
    trade_on_close = True
    leverage = 1.0

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])
        signal = self.slope_indicator.get_value()

        if signal and signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass

        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
                self.sell(sl=stop_loss, tp=take_profit)
            except:
                pass


# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    return df


def analyze_and_display_results(trades_combined, initial_cash):
    # Преобразуем список сделок в DataFrame
    df = pd.DataFrame(trades_combined)

    # Сортировка сделок по времени открытия
    df['OpenDate'] = pd.to_datetime(df['OpenDate'])
    df['CloseDate'] = pd.to_datetime(df['CloseDate'])
    df.sort_values(by='OpenDate', inplace=True)

    # Вычисление времени длительности сделок в часах
    df['Duration'] = (df['CloseDate'] - df['OpenDate']).dt.total_seconds() / 3600  # В часах

    # Инициализация баланса
    balance = initial_cash
    balances = []
    cumulative_pnls = []

    # Последовательный пересчёт PnL и баланса
    for _, trade in df.iterrows():
        balance += trade['PnL']
        cumulative_pnls.append(balance - initial_cash)
        balances.append(balance)

    # Добавление пересчитанных данных в DataFrame
    df['CumulativePnL'] = cumulative_pnls
    df['Balance'] = balances
    df['CumulativePnLPercent'] = (df['CumulativePnL'] / initial_cash) * 100

    # Сохранение детальной информации в файл
    df.to_csv('sequential_backtest_results.csv', index=False)
    print("Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.")

    # Группировка по месяцам и анализ
    monthly_stats = []
    grouped = df.groupby(df['OpenDate'].dt.to_period('M'))
    for month, group in grouped:
        total_pnl = group['PnL'].sum()
        percent_pnl = (total_pnl / initial_cash) * 100
        ending_balance = group['Balance'].iloc[-1]
        total_trades = len(group)

        # Среднее время длительности сделок в часах
        avg_duration = group['Duration'].mean()

        # Разделение сделок на профитные и убыточные
        profitable_trades = group[group['PnL'] > 0]
        losing_trades = group[group['PnL'] <= 0]

        # Разделение по монетам
        trades_by_symbol = group.groupby('Symbol')['PnL'].agg(['count', 'sum']).reset_index()
        symbol_stats = {
            row.Symbol: {
                'trades': row['count'],
                'pnl': row['sum'],
                'percent_pnl': (row['sum'] / initial_cash) * 100
            }
            for _, row in trades_by_symbol.iterrows()
        }

        # Подсчет количества сделок по каждому символу
        profitable_counts = profitable_trades['Symbol'].value_counts()
        losing_counts = losing_trades['Symbol'].value_counts()

        monthly_stats.append({
            'month': str(month),
            'total_pnl': total_pnl,
            'percent_pnl': percent_pnl,
            'ending_balance': ending_balance,
            'total_trades': total_trades,
            'avg_duration_hours': avg_duration,  # Средняя длительность сделок в часах
            'profitable_trades': len(profitable_trades),
            'losing_trades': len(losing_trades),
            'trades_by_symbol': symbol_stats,
            'profitable_by_symbol': profitable_counts.to_dict(),
            'losing_by_symbol': losing_counts.to_dict()
        })

    # Вывод статистики по месяцам
    print("\n===== Статистика по месяцам =====\n")
    for stat in monthly_stats:
        print(f"Месяц: {stat['month']}")
        print(f"Общий PnL: {stat['total_pnl']:.2f} ({stat['percent_pnl']:.2f}%)")
        print(f"Баланс на конец месяца: {stat['ending_balance']:.2f}")
        print(f"Количество сделок: {stat['total_trades']} (разбивка по монетам: {stat['trades_by_symbol']})")
        print(f"Средняя длительность сделок: {stat['avg_duration_hours']:.2f} часов")
        print(f"Профитных сделок: {stat['profitable_trades']} (по монетам: {stat['profitable_by_symbol']})")
        print(f"Убыточных сделок: {stat['losing_trades']} (по монетам: {stat['losing_by_symbol']})\n")
    print("=================================\n")

    # Сохранение статистики по месяцам в CSV
    stats_df = pd.DataFrame(monthly_stats)
    stats_df.to_csv('monthly_stats.csv', index=False)
    print("Статистика по месяцам сохранена в 'monthly_stats.csv'.")


def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    all_trades = []  # Список для хранения всех сделок

    for coin in coins_config:
        print(f"Запуск тестирования для {coin['symbol']}...")

        # Загрузка данных
        data = download_klines(coin['symbol'], coin['interval'], start_date, end_date)

        # Настройка параметров стратегии
        strategy_config = {
            'window_length': coin['window_length'],
            'min_space': coin['min_space'],
            'sloping_atr_length': coin['sloping_atr_length'],
            'take_profit_multiplier': coin['take_profit_multiplier'],
            'stop_loss_multiplier': coin['stop_loss_multiplier'],
            'leverage': coin['leverage']
        }

        # Запуск бэктеста
        bt = Backtest(
            data,
            SlopingStrategy,
            cash=initial_cash,
            trade_on_close=True,
            exclusive_orders=True,
            commission=0.0005
        )
        stats = bt.run(**strategy_config)

        # Извлечение данных о сделках
        trades = stats['_trades']
        if not trades.empty:
            for _, trade in trades.iterrows():
                all_trades.append({
                    'Symbol': coin['symbol'],
                    'OpenDate': trade['EntryTime'],
                    'CloseDate': trade['ExitTime'],
                    'PnL': trade['PnL'],
                    'EntryPrice': trade['EntryPrice'],
                    'ExitPrice': trade['ExitPrice'],
                    'Size': trade['Size']
                })

        print(f"Тестирование для {coin['symbol']} завершено.")

    return all_trades


# Основная функция запуска
def run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash):
    trades_combined = run_combined_backtest(coins_config, start_date, end_date, initial_cash)
    if trades_combined:
        analyze_and_display_results(trades_combined, initial_cash)
    else:
        print("Сделок не найдено. Проверьте данные и параметры стратегии.")

In [ ]:
# final_models_check = pd.read_excel("final_models_check.xlsx")

In [ ]:
import pandas as pd
import re

def parse_final_models_check(file_path):
    coins_config = []

    # Регулярное выражение для извлечения параметров монеты из строки
    model_params_pattern = re.compile(
        r"(?P<symbol>\w+)_((?P<interval>\d+[mh])) \(window=(?P<window>\d+), atr_length=(?P<atr_length>\d+), min_space=(?P<min_space>\d+), take=(?P<take>[\d.]+), loss=(?P<loss>[\d.]+)"
    )

    # Чтение файла Excel в DataFrame
    data = pd.read_excel(file_path)

    for _, row in data.iterrows():
        models_params = row["models_params"]

        for match in model_params_pattern.finditer(models_params):
            symbol = match.group("symbol")
            interval = match.group("interval")
            coins_config.append({
                "symbol": symbol,
                "interval": interval,
                "window_length": int(match.group("window")),
                "min_space": int(match.group("min_space")),
                "sloping_atr_length": int(match.group("atr_length")),
                "take_profit_multiplier": float(match.group("take")),
                "stop_loss_multiplier": float(match.group("loss")),
                "trade_amount": 10,  # Укажите ваше значение
                "leverage": 20  # Укажите ваше значение
            })

    return coins_config

# Пример использования для Google Colab
file_path = "final_models_check 1.xlsx"  # Укажите путь к вашему файлу Excel
coins_config = parse_final_models_check(file_path)

# Для проверки результата
for coin in coins_config:
    print(coin)


{'symbol': '1000PEPEUSDT', 'interval': '30m', 'window_length': 86, 'min_space': 5, 'sloping_atr_length': 18, 'take_profit_multiplier': 1.80623414832759, 'stop_loss_multiplier': 2.59902614088353, 'trade_amount': 10, 'leverage': 20}
{'symbol': 'APEUSDT', 'interval': '1h', 'window_length': 90, 'min_space': 5, 'sloping_atr_length': 20, 'take_profit_multiplier': 2.01297570047132, 'stop_loss_multiplier': 2.3111568893801, 'trade_amount': 10, 'leverage': 20}
{'symbol': 'ORDIUSDT', 'interval': '1h', 'window_length': 40, 'min_space': 5, 'sloping_atr_length': 30, 'take_profit_multiplier': 2.46269916422318, 'stop_loss_multiplier': 1.00722624957696, 'trade_amount': 10, 'leverage': 20}
{'symbol': 'SOLUSDT', 'interval': '1h', 'window_length': 100, 'min_space': 5, 'sloping_atr_length': 16, 'take_profit_multiplier': 2.030368420358, 'stop_loss_multiplier': 1.8828792771351, 'trade_amount': 10, 'leverage': 20}
{'symbol': 'SUIUSDT', 'interval': '1h', 'window_length': 10, 'min_space': 5, 'sloping_atr_length

Комбинация: 1000PEPEUSDT, BNBUSDT, LINAUSDT, ORDIUSDT, WIFUSDT, WLDUSDT, XRPUSDT
   Общий профит: 6020.08%
   Средний месячный профит: 430.01%
   Минимальный месячный профит: -2.10%
   PnL по месяцам:
      2023-10: 162.65%
      2023-11: 236.37%
      2023-12: -2.10%
      2024-01: 223.20%
      2024-02: 239.31%
      2024-03: 192.12%
      2024-04: 33.95%
      2024-05: 27.71%
      2024-06: 58.65%
      2024-07: 298.94%
      2024-08: 251.16%
      2024-09: 96.04%
      2024-10: 287.84%
      2024-11: 311.65%
   Параметры моделей: 1000PEPEUSDT_1h (window=100, atr_length=9, min_space=5, take=1.8740350422203365, loss=2.09759483903459); BNBUSDT_1h (window=34, atr_length=20, min_space=5, take=1.0003717941136006, loss=2.39353768020516); LINAUSDT_1h (window=96, atr_length=27, min_space=5, take=2.049448110407128, loss=2.0580648628801086); ORDIUSDT_1h (window=42, atr_length=30, min_space=5, take=2.561307156240609, loss=1.0179889676917246); WIFUSDT_1h (window=84, atr_length=7, min_space=5, take=2.003858833178518, loss=2.3952446942683343); WLDUSDT_1h (window=68, atr_length=29, min_space=5, take=2.42160013881293, loss=2.551619203185885); XRPUSDT_1h (window=42, atr_length=17, min_space=5, take=1.7588158439140311, loss=1.854125401775483)


In [ ]:
start_date = '2024-11'
end_date = '2024-11'
initial_cash = 30
display(coins_config)
# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

[{'symbol': '1000PEPEUSDT',
  'interval': '30m',
  'window_length': 86,
  'min_space': 5,
  'sloping_atr_length': 18,
  'take_profit_multiplier': 1.80623414832759,
  'stop_loss_multiplier': 2.59902614088353,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'APEUSDT',
  'interval': '1h',
  'window_length': 90,
  'min_space': 5,
  'sloping_atr_length': 20,
  'take_profit_multiplier': 2.01297570047132,
  'stop_loss_multiplier': 2.3111568893801,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ORDIUSDT',
  'interval': '1h',
  'window_length': 40,
  'min_space': 5,
  'sloping_atr_length': 30,
  'take_profit_multiplier': 2.46269916422318,
  'stop_loss_multiplier': 1.00722624957696,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'SOLUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 16,
  'take_profit_multiplier': 2.030368420358,
  'stop_loss_multiplier': 1.8828792771351,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'SUIUSDT',
 

Запуск тестирования для 1000PEPEUSDT...
Тестирование для 1000PEPEUSDT завершено.
Запуск тестирования для APEUSDT...
Тестирование для APEUSDT завершено.
Запуск тестирования для ORDIUSDT...


<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для ORDIUSDT завершено.
Запуск тестирования для SOLUSDT...


<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Тестирование для SUIUSDT завершено.
Запуск тестирования для WIFUSDT...
Тестирование для WIFUSDT завершено.
Запуск тестирования для WLDUSDT...
Тестирование для WLDUSDT завершено.
Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.

===== Статистика по месяцам =====

Месяц: 2024-11
Общий PnL: 19.15 (63.83%)
Баланс на конец месяца: 49.15
Количество сделок: 99 (разбивка по монетам: {'1000PEPEUSDT': {'trades': 29, 'pnl': -1.8386401830665606, 'percent_pnl': -6.1288006102218695}, 'APEUSDT': {'trades': 15, 'pnl': 4.428885235759581, 'percent_pnl': 14.76295078586527}, 'SUIUSDT': {'trades': 22, 'pnl': 7.046264616889257, 'percent_pnl': 23.48754872296419}, 'WIFUSDT': {'trades': 21, 'pnl': -0.18736708857191697, 'percent_pnl': -0.62455696190639}, 'WLDUSDT': {'trades': 12, 'pnl': 9.699551178393488, 'percent_pnl': 32.33183726131163}})
Средняя длительность сделок: 13.91 часов
Профитных сд

<ipython-input-31-a9b4d2fccb8d>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))


In [ ]:
start_date = '2020-10'
end_date = '2024-11'
initial_cash = 30
display(coins_config)
# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

[{'symbol': '1000PEPEUSDT',
  'interval': '30m',
  'window_length': 86,
  'min_space': 5,
  'sloping_atr_length': 18,
  'take_profit_multiplier': 1.80623414832759,
  'stop_loss_multiplier': 2.59902614088353,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ADAUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 14,
  'take_profit_multiplier': 1.930628726507,
  'stop_loss_multiplier': 2.44128263607204,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'APEUSDT',
  'interval': '1h',
  'window_length': 90,
  'min_space': 5,
  'sloping_atr_length': 20,
  'take_profit_multiplier': 2.01297570047132,
  'stop_loss_multiplier': 2.3111568893801,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ARBUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 14,
  'take_profit_multiplier': 2.00967846755187,
  'stop_loss_multiplier': 2.09159862679683,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'DOGEUSDT',

Запуск тестирования для 1000PEPEUSDT...
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-202

<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для ETHUSDT завершено.
Запуск тестирования для LINAUSDT...
Ошибка: Файл klines/LINAUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Тестирование для LINAUSDT завершено.
Запуск тестирования для LINKUSDT...


<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


...

Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Ошибка: Файл klines/SUIUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-08.zip поврежден или не является 

<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для TRBUSDT завершено.
Запуск тестирования для WIFUSDT...
Ошибка: Файл klines/WIFUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-08.zip поврежден или не является 

<ipython-input-31-a9b4d2fccb8d>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))



===== Статистика по месяцам =====

Месяц: 2020-10
Общий PnL: 28.21 (94.03%)
Баланс на конец месяца: 58.21
Количество сделок: 98 (разбивка по монетам: {'ADAUSDT': {'trades': 16, 'pnl': 5.50422192830471, 'percent_pnl': 18.347406427682365}, 'DOGEUSDT': {'trades': 15, 'pnl': -1.2412584086804594, 'percent_pnl': -4.137528028934865}, 'EOSUSDT': {'trades': 18, 'pnl': 1.6663959464138434, 'percent_pnl': 5.554653154712811}, 'LINKUSDT': {'trades': 21, 'pnl': 2.0794031186057165, 'percent_pnl': 6.931343728685721}, 'SOLUSDT': {'trades': 15, 'pnl': 10.681883951219923, 'percent_pnl': 35.60627983739975}, 'TRBUSDT': {'trades': 13, 'pnl': 9.517557316126414, 'percent_pnl': 31.725191053754713}})
Средняя длительность сделок: 13.04 часов
Профитных сделок: 68 (по монетам: {'LINKUSDT': 15, 'EOSUSDT': 13, 'ADAUSDT': 13, 'SOLUSDT': 11, 'TRBUSDT': 9, 'DOGEUSDT': 7})
Убыточных сделок: 30 (по монетам: {'DOGEUSDT': 8, 'LINKUSDT': 6, 'EOSUSDT': 5, 'SOLUSDT': 4, 'TRBUSDT': 4, 'ADAUSDT': 3})

Месяц: 2020-11
Общий PnL: 

In [ ]:
start_date = '2020-10'
end_date = '2024-11'
initial_cash = 30
display(coins_config)
# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

[{'symbol': '1000PEPEUSDT',
  'interval': '30m',
  'window_length': 86,
  'min_space': 5,
  'sloping_atr_length': 18,
  'take_profit_multiplier': 1.80623414832759,
  'stop_loss_multiplier': 2.59902614088353,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ADAUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 14,
  'take_profit_multiplier': 1.930628726507,
  'stop_loss_multiplier': 2.44128263607204,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'APEUSDT',
  'interval': '1h',
  'window_length': 90,
  'min_space': 5,
  'sloping_atr_length': 20,
  'take_profit_multiplier': 2.01297570047132,
  'stop_loss_multiplier': 2.3111568893801,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'ARBUSDT',
  'interval': '1h',
  'window_length': 100,
  'min_space': 5,
  'sloping_atr_length': 14,
  'take_profit_multiplier': 2.00967846755187,
  'stop_loss_multiplier': 2.09159862679683,
  'trade_amount': 10,
  'leverage': 20},
 {'symbol': 'DOGEUSDT',

Запуск тестирования для 1000PEPEUSDT...
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/1000PEPEUSDT-30m-202

<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для ETHUSDT завершено.
Запуск тестирования для LINAUSDT...
Ошибка: Файл klines/LINAUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/LINAUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Тестирование для LINAUSDT завершено.
Запуск тестирования для LINKUSDT...


<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


...

Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Ошибка: Файл klines/SUIUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/SUIUSDT-1h-2021-08.zip поврежден или не является 

<ipython-input-31-a9b4d2fccb8d>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для TRBUSDT завершено.
Запуск тестирования для WIFUSDT...
Ошибка: Файл klines/WIFUSDT-1h-2020-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2020-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2020-12.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-01.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-02.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-03.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-04.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-05.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-06.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-07.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2021-08.zip поврежден или не является 

<ipython-input-31-a9b4d2fccb8d>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))



===== Статистика по месяцам =====

Месяц: 2020-10
Общий PnL: 26.54 (88.47%)
Баланс на конец месяца: 56.54
Количество сделок: 80 (разбивка по монетам: {'ADAUSDT': {'trades': 16, 'pnl': 5.50422192830471, 'percent_pnl': 18.347406427682365}, 'DOGEUSDT': {'trades': 15, 'pnl': -1.2412584086804594, 'percent_pnl': -4.137528028934865}, 'LINKUSDT': {'trades': 21, 'pnl': 2.0794031186057165, 'percent_pnl': 6.931343728685721}, 'SOLUSDT': {'trades': 15, 'pnl': 10.681883951219923, 'percent_pnl': 35.60627983739975}, 'TRBUSDT': {'trades': 13, 'pnl': 9.517557316126414, 'percent_pnl': 31.725191053754713}})
Средняя длительность сделок: 13.89 часов
Профитных сделок: 55 (по монетам: {'LINKUSDT': 15, 'ADAUSDT': 13, 'SOLUSDT': 11, 'TRBUSDT': 9, 'DOGEUSDT': 7})
Убыточных сделок: 25 (по монетам: {'DOGEUSDT': 8, 'LINKUSDT': 6, 'SOLUSDT': 4, 'TRBUSDT': 4, 'ADAUSDT': 3})

Месяц: 2020-11
Общий PnL: -18.01 (-60.04%)
Баланс на конец месяца: 38.53
Количество сделок: 92 (разбивка по монетам: {'ADAUSDT': {'trades': 12,

In [ ]:
# НАчало декабря

# Конфигурация теста
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '1h',
        'window_length': 100,
        'min_space': 5,
        'sloping_atr_length': 9,
        'take_profit_multiplier': 1.874,
        'stop_loss_multiplier': 2.097,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'BNBUSDT',
        'interval': '1h',
        'window_length': 34,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.00,
        'stop_loss_multiplier': 2.393,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'LINAUSDT',
        'interval': '1h',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 27,
        'take_profit_multiplier': 2.049,
        'stop_loss_multiplier': 2.058,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WIFUSDT',
        'interval': '1h',
        'window_length': 84,
        'min_space': 5,
        'sloping_atr_length': 7,
        'take_profit_multiplier': 2.003,
        'stop_loss_multiplier': 2.395,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WLDUSDT',
        'interval': '1h',
        'window_length': 68,
        'min_space': 5,
        'sloping_atr_length': 29,
        'take_profit_multiplier': 2.421,
        'stop_loss_multiplier': 2.551,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'XRPUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 17,
        'take_profit_multiplier': 1.758,
        'stop_loss_multiplier': 1.854,
        'trade_amount': 10,
        'leverage': 20
    },
]

start_date = '2023-10'
end_date = '2024-11'
initial_cash = 100

# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

Запуск тестирования для 1000PEPEUSDT...
Тестирование для 1000PEPEUSDT завершено.
Запуск тестирования для BNBUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для BNBUSDT завершено.
Запуск тестирования для LINAUSDT...
Тестирование для LINAUSDT завершено.
Запуск тестирования для ORDIUSDT...
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Тестирование для ORDIUSDT завершено.
Запуск тестирования для WIFUSDT...
Ошибка: Файл klines/WIFUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2023-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2023-12.zip поврежден или не является ZIP. Пропускаем.
Тестирование для WIFUSDT завершено.
Запуск тестирования для WLDUSDT...
Тестирование для WLDUSDT завершено.
Запуск тестирования для XRPUSDT...
Тестирование для XRPUSDT завершено.
Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.

===== Статистика по месяцам =====

Месяц: 2023-10
Общий PnL: 47.04 (47.04%)
Баланс на конец месяца: 147.04
Количество сделок: 65 (разбивка по монетам: {'1000PEPE

<ipython-input-41-941383555082>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))


Параметры моделей: 1000PEPEUSDT_30m (window=96, atr_length=10, min_space=5, take=2.0157635444989057, loss=2.56089497721195); BNBUSDT_1h (window=34, atr_length=20, min_space=5, take=1.0003717941136006, loss=2.39353768020516); LINAUSDT_1h (window=96, atr_length=27, min_space=5, take=2.049448110407128, loss=2.0580648628801086); ORDIUSDT_1h (window=42, atr_length=30, min_space=5, take=2.561307156240609, loss=1.0179889676917246); WIFUSDT_1h (window=84, atr_length=10, min_space=5, take=2.538796573657456, loss=2.591463669038581); WLDUSDT_1h (window=68, atr_length=29, min_space=5, take=2.42160013881293, loss=2.551619203185885); XRPUSDT_1h (window=42, atr_length=9, min_space=5, take=1.8071809921665287, loss=1.9214782453444468)


In [ ]:
# Конфигурация теста
# Начало декабря


coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.015,
        'stop_loss_multiplier': 2.560,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'BNBUSDT',
        'interval': '1h',
        'window_length': 34,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.00,
        'stop_loss_multiplier': 2.393,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'LINAUSDT',
        'interval': '1h',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 27,
        'take_profit_multiplier': 2.049,
        'stop_loss_multiplier': 2.058,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 10,
        'leverage': 20
    },
    # {
    #     'symbol': 'WIFUSDT',
    #     'interval': '1h',
    #     'window_length': 84,
    #     'min_space': 5,
    #     'sloping_atr_length': 10,
    #     'take_profit_multiplier': 2.538,
    #     'stop_loss_multiplier': 2.591,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
    # {
    #     'symbol': 'WLDUSDT',
    #     'interval': '1h',
    #     'window_length': 68,
    #     'min_space': 5,
    #     'sloping_atr_length': 29,
    #     'take_profit_multiplier': 2.421,
    #     'stop_loss_multiplier': 2.551,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
    {
        'symbol': 'XRPUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 9,
        'take_profit_multiplier': 1.807,
        'stop_loss_multiplier': 1.921,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'AAVEUSDT',
        'interval': '1h',
        'window_length': 40,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 2.068,
        'stop_loss_multiplier': 2.569,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'OPUSDT',
        'interval': '1h',
        'window_length': 80,
        'min_space': 5,
        'sloping_atr_length': 14,
        'take_profit_multiplier': 1.28,
        'stop_loss_multiplier': 1.408,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SOLUSDT',
        'interval': '1h',
        'window_length': 38,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 1.671,
        'stop_loss_multiplier': 1.239,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.973,
        'stop_loss_multiplier': 2.438,
        'trade_amount': 10,
        'leverage': 20
    },
    # {
    #     'symbol': 'TIAUSDT',
    #     'interval': '1h',
    #     'window_length': 54,
    #     'min_space': 5,
    #     'sloping_atr_length': 21,
    #     'take_profit_multiplier': 1.257,
    #     'stop_loss_multiplier': 2.332,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
#     {
#         'symbol': 'TIAUSDT',
#         'interval': '1h',
#         'window_length': 56,
#         'min_space': 5,
#         'sloping_atr_length': 20,
#         'take_profit_multiplier': 1.316,
#         'stop_loss_multiplier': 2.047,
#         'trade_amount': 10,
#         'leverage': 20
#     },

]

start_date = '2023-10'
end_date = '2024-11'
initial_cash = 100

# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

Запуск тестирования для 1000PEPEUSDT...
Тестирование для 1000PEPEUSDT завершено.
Запуск тестирования для BNBUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для BNBUSDT завершено.
Запуск тестирования для LINAUSDT...
Тестирование для LINAUSDT завершено.
Запуск тестирования для ORDIUSDT...
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Тестирование для ORDIUSDT завершено.
Запуск тестирования для XRPUSDT...
Тестирование для XRPUSDT завершено.
Запуск тестирования для AAVEUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для AAVEUSDT завершено.
Запуск тестирования для OPUSDT...
Тестирование для OPUSDT завершено.
Запуск тестирования для SOLUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Тестирование для SUIUSDT завершено.
Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.

===== Статистика по месяцам =====

Месяц: 2023-10
Общий PnL: -12.15 (-12.15%)
Баланс на конец месяца: 87.85
Количество сделок: 182 (разбивка по монетам: {'1000PEPEUSDT': {'trades': 27, 'pnl': -19.914922523508512, 'percent_pnl': -19.914922523508512}, 'AAVEUSDT': {'trades': 20, 'pnl': 1.7779318715654924, 'percent_pnl': 1.7779318715654924}, 'LINAUSDT': {'trades': 16, 'pnl': 1.096200644607555, 'percent_pnl': 1.096200644607555}, 'OPUSDT': {'trades': 25, 'pnl': -12.591938018177625, 'percent_pnl': -12.591938018177625}, 'SOLUSDT': {'trades': 34, 'pnl': -4.19361133233684, 'percent_pnl': -4.19361133233684}, 'SUIUSDT': {'trades': 38, 'pnl': 22.485519920776007, 'percent_pnl': 22.485519920776007}, 'XRPUSDT': {'trades': 22, 'pnl': -0.8127402904857002, 'percent_pnl': -0.8127402904857003}})
Средняя 

<ipython-input-41-941383555082>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))


In [ ]:
# Конфигурация теста
# Начало декабьряч
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.015,
        'stop_loss_multiplier': 2.560,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'BNBUSDT',
        'interval': '1h',
        'window_length': 34,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.00,
        'stop_loss_multiplier': 2.393,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'LINAUSDT',
        'interval': '1h',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 27,
        'take_profit_multiplier': 2.049,
        'stop_loss_multiplier': 2.058,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WIFUSDT',
        'interval': '1h',
        'window_length': 84,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.538,
        'stop_loss_multiplier': 2.591,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WLDUSDT',
        'interval': '1h',
        'window_length': 68,
        'min_space': 5,
        'sloping_atr_length': 29,
        'take_profit_multiplier': 2.421,
        'stop_loss_multiplier': 2.551,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'XRPUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 9,
        'take_profit_multiplier': 1.807,
        'stop_loss_multiplier': 1.921,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'AAVEUSDT',
        'interval': '1h',
        'window_length': 40,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 2.068,
        'stop_loss_multiplier': 2.569,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'OPUSDT',
        'interval': '1h',
        'window_length': 80,
        'min_space': 5,
        'sloping_atr_length': 14,
        'take_profit_multiplier': 1.28,
        'stop_loss_multiplier': 1.408,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SOLUSDT',
        'interval': '1h',
        'window_length': 38,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 1.671,
        'stop_loss_multiplier': 1.239,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.973,
        'stop_loss_multiplier': 2.438,
        'trade_amount': 10,
        'leverage': 20
    },
    # {
    #     'symbol': 'TIAUSDT',
    #     'interval': '1h',
    #     'window_length': 54,
    #     'min_space': 5,
    #     'sloping_atr_length': 21,
    #     'take_profit_multiplier': 1.257,
    #     'stop_loss_multiplier': 2.332,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
    {
        'symbol': 'TIAUSDT',
        'interval': '1h',
        'window_length': 56,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.316,
        'stop_loss_multiplier': 2.047,
        'trade_amount': 10,
        'leverage': 20
    },

]

start_date = '2023-10'
end_date = '2024-11'
initial_cash = 100

# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

Запуск тестирования для 1000PEPEUSDT...
Тестирование для 1000PEPEUSDT завершено.
Запуск тестирования для BNBUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для BNBUSDT завершено.
Запуск тестирования для LINAUSDT...
Тестирование для LINAUSDT завершено.
Запуск тестирования для ORDIUSDT...
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Тестирование для ORDIUSDT завершено.
Запуск тестирования для WIFUSDT...
Ошибка: Файл klines/WIFUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2023-11.zip поврежден или не является ZIP. Пропускаем.
Ошибка: Файл klines/WIFUSDT-1h-2023-12.zip поврежден или не является ZIP. Пропускаем.
Тестирование для WIFUSDT завершено.
Запуск тестирования для WLDUSDT...
Тестирование для WLDUSDT завершено.
Запуск тестирования для XRPUSDT...
Тестирование для XRPUSDT завершено.
Запуск тестирования для AAVEUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для AAVEUSDT завершено.
Запуск тестирования для OPUSDT...
Тестирование для OPUSDT завершено.
Запуск тестирования для SOLUSDT...


<ipython-input-41-941383555082>:231: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(


Тестирование для SOLUSDT завершено.
Запуск тестирования для SUIUSDT...
Тестирование для SUIUSDT завершено.
Запуск тестирования для TIAUSDT...
Тестирование для TIAUSDT завершено.
Детальная статистика сделок (включая длительность) сохранена в 'sequential_backtest_results.csv'.

===== Статистика по месяцам =====

Месяц: 2023-10
Общий PnL: 15.79 (15.79%)
Баланс на конец месяца: 115.79
Количество сделок: 197 (разбивка по монетам: {'1000PEPEUSDT': {'trades': 27, 'pnl': -19.914922523508512, 'percent_pnl': -19.914922523508512}, 'AAVEUSDT': {'trades': 20, 'pnl': 1.7779318715654924, 'percent_pnl': 1.7779318715654924}, 'LINAUSDT': {'trades': 16, 'pnl': 1.096200644607555, 'percent_pnl': 1.096200644607555}, 'OPUSDT': {'trades': 25, 'pnl': -12.591938018177625, 'percent_pnl': -12.591938018177625}, 'SOLUSDT': {'trades': 34, 'pnl': -4.19361133233684, 'percent_pnl': -4.19361133233684}, 'SUIUSDT': {'trades': 38, 'pnl': 22.485519920776007, 'percent_pnl': 22.485519920776007}, 'WLDUSDT': {'trades': 15, 'pnl

<ipython-input-41-941383555082>:150: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  grouped = df.groupby(df['OpenDate'].dt.to_period('M'))


In [ ]:
# Конфигурация теста
# Начало декабьряч
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.015,
        'stop_loss_multiplier': 2.560,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'BNBUSDT',
        'interval': '1h',
        'window_length': 34,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.00,
        'stop_loss_multiplier': 2.393,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'LINAUSDT',
        'interval': '1h',
        'window_length': 96,
        'min_space': 5,
        'sloping_atr_length': 27,
        'take_profit_multiplier': 2.049,
        'stop_loss_multiplier': 2.058,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WIFUSDT',
        'interval': '1h',
        'window_length': 84,
        'min_space': 5,
        'sloping_atr_length': 10,
        'take_profit_multiplier': 2.538,
        'stop_loss_multiplier': 2.591,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'WLDUSDT',
        'interval': '1h',
        'window_length': 68,
        'min_space': 5,
        'sloping_atr_length': 29,
        'take_profit_multiplier': 2.421,
        'stop_loss_multiplier': 2.551,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'XRPUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 9,
        'take_profit_multiplier': 1.807,
        'stop_loss_multiplier': 1.921,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'AAVEUSDT',
        'interval': '1h',
        'window_length': 40,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 2.068,
        'stop_loss_multiplier': 2.569,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'OPUSDT',
        'interval': '1h',
        'window_length': 80,
        'min_space': 5,
        'sloping_atr_length': 14,
        'take_profit_multiplier': 1.28,
        'stop_loss_multiplier': 1.408,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SOLUSDT',
        'interval': '1h',
        'window_length': 38,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 1.671,
        'stop_loss_multiplier': 1.239,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.973,
        'stop_loss_multiplier': 2.438,
        'trade_amount': 10,
        'leverage': 20
    },
    # {
    #     'symbol': 'TIAUSDT',
    #     'interval': '1h',
    #     'window_length': 54,
    #     'min_space': 5,
    #     'sloping_atr_length': 21,
    #     'take_profit_multiplier': 1.257,
    #     'stop_loss_multiplier': 2.332,
    #     'trade_amount': 10,
    #     'leverage': 20
    # },
    {
        'symbol': 'TIAUSDT',
        'interval': '1h',
        'window_length': 56,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.316,
        'stop_loss_multiplier': 2.047,
        'trade_amount': 10,
        'leverage': 20
    },

]

start_date = '2023-10'
end_date = '2024-11'
initial_cash = 100

# Запуск комбинированного бэктеста
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

##### Draft

In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest, Strategy


# Класс стратегии для backtesting
class SlopingStrategy(Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5
    take_profit_multiplier = 1.5
    trade_on_close = True
    leverage = 1.0

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])
        signal = self.slope_indicator.get_value()

        if signal and signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass

        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
                self.sell(sl=stop_loss, tp=take_profit)
            except:
                pass


# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    return df


def analyze_and_display_results(trades_combined, initial_cash):
    # Преобразуем список сделок в DataFrame для удобного анализа
    df = pd.DataFrame(trades_combined)
    df['Month'] = df['OpenDate'].dt.to_period('M')  # Добавляем столбец с месяцем сделки

    # Группировка по месяцам и анализ
    monthly_stats = []
    grouped = df.groupby('Month')
    for month, group in grouped:
        total_pnl = group['PnL'].sum()
        percent_pnl = (total_pnl / initial_cash) * 100
        total_trades = len(group)

        # Разделение сделок на профитные и убыточные
        profitable_trades = group[group['PnL'] > 0]
        losing_trades = group[group['PnL'] <= 0]

        # Разделение по монетам
        trades_by_symbol = group.groupby('Symbol')['PnL'].agg(['count', 'sum']).reset_index()
        symbol_stats = {}
        for row in trades_by_symbol.itertuples():
            symbol_stats[row.Symbol] = {
                'trades': row.count,
                'pnl': row.sum,
                'percent_pnl': (row.sum / initial_cash) * 100
            }

        # Подсчет количества сделок по каждому символу
        profitable_counts = profitable_trades['Symbol'].value_counts()
        losing_counts = losing_trades['Symbol'].value_counts()

        monthly_stats.append({
            'month': str(month),
            'total_pnl': total_pnl,
            'percent_pnl': percent_pnl,
            'total_trades': total_trades,
            'profitable_trades': len(profitable_trades),
            'losing_trades': len(losing_trades),
            'trades_by_symbol': symbol_stats,
            'profitable_by_symbol': profitable_counts.to_dict(),
            'losing_by_symbol': losing_counts.to_dict()
        })

    # Вывод статистики по месяцам
    print("\n===== Статистика по месяцам =====\n")
    for stat in monthly_stats:
        print(f"Месяц: {stat['month']}")
        print(f"Общий PnL: {stat['total_pnl']:.2f} ({stat['percent_pnl']:.2f}%)")
        print(f"Количество сделок: {stat['total_trades']} (разбивка по монетам: {stat['trades_by_symbol']})")
        print(f"Профитных сделок: {stat['profitable_trades']} (по монетам: {stat['profitable_by_symbol']})")
        print(f"Убыточных сделок: {stat['losing_trades']} (по монетам: {stat['losing_by_symbol']})\n")
    print("=================================\n")

    # Сохранение статистики в CSV
    stats_df = pd.DataFrame(monthly_stats)
    stats_df.to_csv('monthly_stats.csv', index=False)
    print("Статистика по месяцам сохранена в 'monthly_stats.csv'.")

# Подключение к функции запуска
def run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash):
    trades_combined = run_combined_backtest(coins_config, start_date, end_date, initial_cash)
    if trades_combined:
        analyze_and_display_results(trades_combined, initial_cash)
    else:
        print("Сделок не найдено. Проверьте данные и параметры стратегии.")


# Конфигурация монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.806,
        'stop_loss_multiplier': 2.599,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 100,
        'leverage': 20
    },
]

# Запуск теста
start_date = '2023-10'
end_date = '2024-11'
initial_cash = 1000
run_combined_backtest_with_analysis(coins_config, start_date, end_date, initial_cash)

Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.

===== Статистика по месяцам =====

Месяц: 2023-10
Общий PnL: -1.12 (-0.11%)
Количество сделок: 49 (разбивка по монетам: {'1000PEPEUSDT': {'trades': 49, 'pnl': -1.1212077358018895, 'percent_pnl': -0.11212077358018895}})
Профитных сделок: 23 (по монетам: {'1000PEPEUSDT': 23})
Убыточных сделок: 26 (по монетам: {'1000PEPEUSDT': 26})

Месяц: 2023-11
Общий PnL: 72.76 (7.28%)
Количество сделок: 81 (разбивка по монетам: {'1000PEPEUSDT': {'trades': 59, 'pnl': -0.5040447431808212, 'percent_pnl': -0.05040447431808212}, 'ORDIUSDT': {'trades': 22, 'pnl': 73.26688083201046, 'percent_pnl': 7.326688083201045}})
Профитных сделок: 47 (по монетам: {'1000PEPEUSDT': 30, 'ORDIUSDT': 17})
Убыточных сделок: 34 (по монетам: {'1000PEPEUSDT': 29, 'ORDIUSDT': 5})

Месяц: 2023-12
Общий PnL: 13.52 (1.35%)
Количество сделок: 86 (разбивка по монетам: {'1000PEPEUSDT': {'trades': 55, 'pnl': 0.7614415295440153, 'percent_pnl': 0.07614

<ipython-input-55-0eaf86141714>:124: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  df['Month'] = df['OpenDate'].dt.to_period('M')  # Добавляем столбец с месяцем сделки


In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest, Strategy


# Класс стратегии для backtesting
class SlopingStrategy(Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5
    take_profit_multiplier = 1.5
    trade_on_close = True
    leverage = 1.0

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])
        signal = self.slope_indicator.get_value()

        if signal and signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass

        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
                self.sell(sl=stop_loss, tp=take_profit)
            except:
                pass


# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    return df


# Последовательный бэктест с анализом
def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    cash = initial_cash
    trades_combined = []  # Объединенные сделки для всех монет

    for coin in coins_config:
        symbol = coin['symbol']
        interval = coin['interval']

        try:
            data = download_klines(symbol, interval, start_date, end_date)
            if data.empty:
                print(f"Нет данных для {symbol}. Пропускаем.")
                continue
        except Exception as e:
            print(f"Ошибка загрузки данных для {symbol}: {e}")
            continue

        bt = Backtest(data, SlopingStrategy, cash=coin['trade_amount'], commission=0.0005)
        stats = bt.run(trade_on_close=True, leverage=coin['leverage'])

        trades = stats['_trades']
        for trade in trades.itertuples():
            trades_combined.append({
                'Symbol': symbol,
                'OpenDate': getattr(trade, 'EntryTime'),
                'CloseDate': getattr(trade, 'ExitTime'),
                'EntryPrice': getattr(trade, 'EntryPrice'),
                'ExitPrice': getattr(trade, 'ExitPrice'),
                'PnL': getattr(trade, 'PnL'),
                'PercentPnL': getattr(trade, 'PnL') / initial_cash * 100
            })

    trades_combined.sort(key=lambda x: x['OpenDate'])
    return trades_combined


# Конфигурация монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.806,
        'stop_loss_multiplier': 2.599,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 100,
        'leverage': 20
    },
]

# Запуск теста
start_date = '2023-10'
end_date = '2024-11'
initial_cash = 1000
run_sequential_backtest(coins_config, start_date, end_date, initial_cash)


Начальный депозит: 1000


Обработка монеты 1/2: 1000PEPEUSDT
Найдено 496 сделок для 1000PEPEUSDT.

Обработка монеты 2/2: ORDIUSDT
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Найдено 373 сделок для ORDIUSDT.

Детали всех сделок сохранены в 'sequential_backtest_results.csv'.


[{'Symbol': '1000PEPEUSDT',
  'OpenDate': Timestamp('2023-10-03 17:00:00+0000', tz='UTC'),
  'CloseDate': Timestamp('2023-10-04 00:00:00+0000', tz='UTC'),
  'EntryPrice': 0.0007331664,
  'ExitPrice': 0.0007115156276212902,
  'PnL': -0.2952948844732219,
  'Balance': 999.7047051155267},
 {'Symbol': '1000PEPEUSDT',
  'OpenDate': Timestamp('2023-10-04 03:00:00+0000', tz='UTC'),
  'CloseDate': Timestamp('2023-10-05 14:00:00+0000', tz='UTC'),
  'EntryPrice': 0.0007153575,
  'ExitPrice': 0.0007306617086324737,
  'PnL': 0.2076168943081391,
  'Balance': 1000.2076168943081},
 {'Symbol': '1000PEPEUSDT',
  'OpenDate': Timestamp('2023-10-05 17:00:00+0000', tz='UTC'),
  'CloseDate': Timestamp('2023-10-06 12:30:00+0000', tz='UTC'),
  'EntryPrice': 0.0007066465,
  'ExitPrice': 0.0006925967624585471,
  'PnL': 0.19707566849395916,
  'Balance': 1000.197075668494},
 {'Symbol': '1000PEPEUSDT',
  'OpenDate': Timestamp('2023-10-06 16:00:00+0000', tz='UTC'),
  'CloseDate': Timestamp('2023-10-06 21:30:00+0000'

In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest, Strategy


# Класс стратегии для backtesting
class SlopingStrategy(Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5
    take_profit_multiplier = 1.5
    trade_on_close = True
    leverage = 1.0

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])
        signal = self.slope_indicator.get_value()

        if signal and signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass

        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
                self.sell(sl=stop_loss, tp=take_profit)
            except:
                pass


# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)

    return df


# Последовательный бэктест
def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    cash = initial_cash
    trades_combined = []  # Объединенные сделки для всех монет
    total_pnl = 0

    print(f"Начальный депозит: {cash}\n")

    for i, coin in enumerate(coins_config):
        symbol = coin['symbol']
        interval = coin['interval']
        print(f"\nОбработка монеты {i + 1}/{len(coins_config)}: {symbol}")

        try:
            # Загрузка данных
            data = download_klines(symbol, interval, start_date, end_date)
            if data.empty:
                print(f"Нет данных для {symbol}. Пропускаем.")
                continue
        except Exception as e:
            print(f"Ошибка загрузки данных для {symbol}: {e}")
            continue

        # Настройка параметров стратегии
        SlopingStrategy.window_length = coin['window_length']
        SlopingStrategy.min_space = coin['min_space']
        SlopingStrategy.sloping_atr_length = coin['sloping_atr_length']
        SlopingStrategy.take_profit_multiplier = coin['take_profit_multiplier']
        SlopingStrategy.stop_loss_multiplier = coin['stop_loss_multiplier']

        # Выполнение бэктеста
        bt = Backtest(data, SlopingStrategy, cash=coin['trade_amount'], commission=0.0005)
        stats = bt.run(trade_on_close=True, leverage=coin['leverage'])

        trades = stats['_trades']
        print(f"Найдено {len(trades)} сделок для {symbol}.")

        # Добавление сделок в общий список
        for trade in trades.itertuples():
            trades_combined.append({
                'Symbol': symbol,
                'OpenDate': getattr(trade, 'EntryTime'),
                'CloseDate': getattr(trade, 'ExitTime'),
                'EntryPrice': getattr(trade, 'EntryPrice'),
                'ExitPrice': getattr(trade, 'ExitPrice'),
                'PnL': getattr(trade, 'PnL')
            })

        # Суммируем PnL
        total_pnl += stats['Equity Final [$]'] - coin['trade_amount']

    # Сортировка сделок по дате открытия
    trades_combined.sort(key=lambda x: x['OpenDate'])

    # Подсчет итогового PnL и проверка депозита
    running_cash = cash
    for trade in trades_combined:
        running_cash += trade['PnL']
        trade['Balance'] = running_cash
        trade['PnL%'] = (trade['PnL'] / initial_cash) * 100
        if running_cash <= 0:
            print(f"Депозит исчерпан на сделке: {trade}")
            break

    # Вывод результатов
    print(f"\nИтоговый депозит: {running_cash}")
    trade_df = pd.DataFrame(trades_combined)
    trade_df.to_csv('combined_trades.csv', index=False)
    print("\nДетали всех сделок сохранены в 'combined_trades.csv'.")



# Конфигурация монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.806,
        'stop_loss_multiplier': 2.599,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.561,
        'stop_loss_multiplier': 1.018,
        'trade_amount': 100,
        'leverage': 20
    },
]

# Запуск теста
start_date = '2023-10'
end_date = '2024-11'
initial_cash = 1000
run_sequential_backtest(coins_config, start_date, end_date, initial_cash)


Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.

Результаты последовательного бэктеста сохранены в 'sequential_backtest_results.csv'.

Тест завершён. Итоговый депозит: 1752.94


,symbol,open_date,close_date,entry_price,exit_price,pnl,percent_pnl,balance_after_trade
0,1000PEPEUSDT,2023-10-03 17:00:00+00:00,2023-10-04 00:00:00+00:00,0.000733,0.000712,-0.295295,-2.952949,999.704705
1,1000PEPEUSDT,2023-10-04 03:00:00+00:00,2023-10-05 14:00:00+00:00,0.000715,0.000731,0.207617,2.076169,999.912322
2,1000PEPEUSDT,2023-10-05 17:00:00+00:00,2023-10-06 12:30:00+00:00,0.000707,0.000693,0.197076,1.970757,1000.109398
3,1000PEPEUSDT,2023-10-06 16:00:00+00:00,2023-10-06 21:30:00+00:00,0.000716,0.000727,0.159339,1.593393,1000.268737
4,1000PEPEUSDT,2023-10-07 19:30:00+00:00,2023-10-08 00:00:00+00:00,0.000697,0.000710,-0.183481,-1.834809,1000.085256
...,...,...,...,...,...,...,...,...
864,ORDIUSDT,2024-11-28 16:00:00+00:00,2024-11-28 16:00:00+00:00,39.580200,40.465434,-18.589917,-18.589917,1784.909833
865,1000PEPEUSDT,2024-11-29 08:00:00+00:00,2024-11-29 10:30:00+00:00,0.019519,0.020203,-2.190541,-21.905408,1782.719292
866,ORDIUSDT,2024-11-29 08:00:00+00:00,2024-11-29 09:00:00+00:00,40.233873,41.075349,-16.829520,-16.829520,1765.889772
867,ORDIUSDT,2024-11-30 09:00:00+00:00,2024-11-30 16:00:00+00:00,41.120430,41.845701,-13.780155,-13.780155,1752.109617


In [ ]:
import os
import zipfile
from collections import defaultdict

# Функция загрузки данных
def load_data(symbol, interval, start_date, end_date):
    # Эта функция должна возвращать данные за указанный интервал
    # Для упрощения проверки наличия файлов, просто проверим наличие ZIP-файлов
    data_path = f"klines/{symbol}-{interval}-{start_date}.zip"
    if not os.path.exists(data_path):
        return None
    try:
        with zipfile.ZipFile(data_path, 'r') as zip_ref:
            zip_ref.testzip()  # Проверка целостности архива
        return f"Data for {symbol} from {start_date}"
    except zipfile.BadZipFile:
        print(f"Ошибка: Файл {data_path} поврежден или не является ZIP. Пропускаем.")
        return None

# Бэктестирование
def backtest_strategy(data, params):
    # Заглушка для стратегии, здесь будет ваша логика
    trades = 100  # Количество сделок
    profitable_trades = trades // 2
    losing_trades = trades - profitable_trades
    pnl = trades * 100  # Общий PnL
    return {
        'PnL': pnl,
        'Trades': trades,
        'Profitable': profitable_trades,
        'Losing': losing_trades,
        'PnL_percent': pnl / 1_000_000 * 100  # Пример процента от $1,000,000
    }

# Основной бэктест для каждой монеты
def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    results_by_month = defaultdict(lambda: defaultdict(dict))
    initial_balance = initial_cash

    for coin in coins_config:
        symbol = coin['symbol']
        interval = coin['interval']
        print(f"Начинаем тестирование {symbol} с {start_date} по {end_date}.")

        # Определяем доступный стартовый месяц
        current_date = start_date
        while current_date <= end_date:
            data = load_data(symbol, interval, current_date, end_date)
            if data:
                print(f"Для {symbol} данные доступны с {current_date}.")
                break
            else:
                # Переход к следующему месяцу
                year, month = map(int, current_date.split('-'))
                month += 1
                if month > 12:
                    year += 1
                    month = 1
                current_date = f"{year:04d}-{month:02d}"
        else:
            print(f"Для {symbol} нет доступных данных. Пропускаем.")
            continue

        # Бэктест по месяцам
        while current_date <= end_date:
            print(f"Обрабатываем {symbol} за {current_date}.")
            data = load_data(symbol, interval, current_date, end_date)
            if data:
                results = backtest_strategy(data, coin)
                results_by_month[current_date][symbol] = results

            # Переход к следующему месяцу
            year, month = map(int, current_date.split('-'))
            month += 1
            if month > 12:
                year += 1
                month = 1
            current_date = f"{year:04d}-{month:02d}"

    # Итоги
    detailed_summary, total_pnl, total_trades = generate_detailed_monthly_summary(results_by_month)
    print_detailed_summary(detailed_summary, total_pnl, initial_balance)

# Генерация детализированного вывода
def generate_detailed_monthly_summary(results_by_month):
    output = []
    total_pnl = 0
    total_trades = 0
    for month, monthly_data in results_by_month.items():
        month_summary = {
            'month': month,
            'total_pnl': sum(data['PnL'] for data in monthly_data.values()),
            'total_trades': sum(data['Trades'] for data in monthly_data.values()),
            'profitable_trades': sum(data['Profitable'] for data in monthly_data.values()),
            'losing_trades': sum(data['Losing'] for data in monthly_data.values()),
            'details': [],
        }

        for symbol, data in monthly_data.items():
            month_summary['details'].append({
                'symbol': symbol,
                'PnL': data['PnL'],
                'Trades': data['Trades'],
                'Profitable': data['Profitable'],
                'Losing': data['Losing'],
                'PnL_percent': data['PnL_percent']
            })

        total_pnl += month_summary['total_pnl']
        total_trades += month_summary['total_trades']
        output.append(month_summary)

    return output, total_pnl, total_trades

# Печать детализированного вывода
def print_detailed_summary(detailed_summary, total_pnl, initial_balance):
    print(f"Итоговый PnL: {total_pnl:.2f}")
    print(f"Итоговый баланс: {initial_balance + total_pnl:.2f}")
    for summary in detailed_summary:
        details = ', '.join(
            [f"{detail['symbol']} PnL={detail['PnL']:.2f} ({detail['PnL_percent']:.2f}%)" for detail in summary['details']]
        )
        print(
            f"Месяц {summary['month']}: "
            f"Общий PnL={summary['total_pnl']:.2f}, "
            f"ОбщийPnL%={summary['total_pnl'] / initial_balance * 100:.2f}%, "
            f"Сделки={summary['total_trades']} ({details}), "
            f"Профитные={summary['profitable_trades']}, Убыточные={summary['losing_trades']}"
        )


# Конфигурация монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.80623414832759,
        'stop_loss_multiplier': 2.59902614088353
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.5613071562406,
        'stop_loss_multiplier': 1.01798896769172
    },
]

# Запуск
start_date = '2023-10'
end_date = '2024-11'
initial_cash = 1_000_000
run_combined_backtest(coins_config, start_date, end_date, initial_cash)


Начинаем тестирование 1000PEPEUSDT с 2023-10 по 2024-11.
Для 1000PEPEUSDT данные доступны с 2023-10.
Обрабатываем 1000PEPEUSDT за 2023-10.
Обрабатываем 1000PEPEUSDT за 2023-11.
Обрабатываем 1000PEPEUSDT за 2023-12.
Обрабатываем 1000PEPEUSDT за 2024-01.
Обрабатываем 1000PEPEUSDT за 2024-02.
Обрабатываем 1000PEPEUSDT за 2024-03.
Обрабатываем 1000PEPEUSDT за 2024-04.
Обрабатываем 1000PEPEUSDT за 2024-05.
Обрабатываем 1000PEPEUSDT за 2024-06.
Обрабатываем 1000PEPEUSDT за 2024-07.
Обрабатываем 1000PEPEUSDT за 2024-08.
Обрабатываем 1000PEPEUSDT за 2024-09.
Обрабатываем 1000PEPEUSDT за 2024-10.
Обрабатываем 1000PEPEUSDT за 2024-11.
Начинаем тестирование ORDIUSDT с 2023-10 по 2024-11.
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Для ORDIUSDT данные доступны с 2023-11.
Обрабатываем ORDIUSDT за 2023-11.
Обрабатываем ORDIUSDT за 2023-12.
Обрабатываем ORDIUSDT за 2024-01.
Обрабатываем ORDIUSDT за 2024-02.
Обрабатываем ORDIUSDT за 2024-03.
Обрабатываем ORDI

##### Близок к завершению

In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest, Strategy

# Класс стратегии для backtesting
class SlopingStrategy(Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5
    take_profit_multiplier = 1.5
    trade_on_close = True

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        self.slope_indicator.add_kline(
            self.data.index[-1].timestamp(),
            self.data.Open[-1],
            self.data.High[-1],
            self.data.Low[-1],
            self.data.Close[-1]
        )

        signal = self.slope_indicator.get_value()

        if signal and signal.side and not self.position:
            current_atr = signal.atr
            stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
            take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
            self.buy(sl=stop_loss, tp=take_profit)

        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
            take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
            self.sell(sl=stop_loss, tp=take_profit)

# Генерация месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []
    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date += relativedelta(months=1)
    return months

# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {'Date': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []}

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path):
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except zipfile.BadZipFile:
            print(f"Ошибка: файл {file_path} поврежден. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

# Основной тест с общим капиталом
def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    total_cash = initial_cash
    trade_details = []

    print(f"Начальный капитал: {total_cash:.2f}")

    for coin in coins_config:
        symbol = coin['symbol']
        interval = coin['interval']
        data = download_klines(symbol, interval, start_date, end_date)

        # Рассчитываем сумму, выделяемую на сделку (процент от общего капитала)
        trade_cash = total_cash * coin['allocation']
        if trade_cash <= 0:
            print(f"Недостаточно капитала для торговли {symbol}. Пропускаем.")
            continue

        # Настройка стратегии
        SlopingStrategy.window_length = coin['window_length']
        SlopingStrategy.min_space = coin['min_space']
        SlopingStrategy.sloping_atr_length = coin['sloping_atr_length']
        SlopingStrategy.take_profit_multiplier = coin['take_profit_multiplier']
        SlopingStrategy.stop_loss_multiplier = coin['stop_loss_multiplier']

        # Запуск бэктеста
        bt = Backtest(data, SlopingStrategy, cash=trade_cash, commission=0.0005, margin=1/coin['leverage'])
        stats = bt.run(trade_on_close=True)

        # Обновляем общий капитал
        pnl = stats['Equity Final [$]'] - trade_cash
        total_cash += pnl
        print(f"{symbol}: PnL={pnl:.2f}, Общий капитал: {total_cash:.2f}")

        # Сохраняем детали сделок
        trades = stats['_trades']
        for trade in trades.itertuples():
            trade_details.append([symbol, trade.EntryPrice, trade.ExitPrice, trade.PnL])

    # Сохраняем результаты сделок в CSV
    trade_df = pd.DataFrame(trade_details, columns=['Symbol', 'EntryPrice', 'ExitPrice', 'PnL'])
    trade_df.to_csv('trade_details.csv', index=False)
    print("\nДетали сделок сохранены в 'trade_details.csv'.")
    print(f"\nИтоговый капитал: {total_cash:.2f}")

# Пример конфигурации
coins_config = [
    {
        'symbol': 'BTCUSDT',
        'interval': '1h',
        'window_length': 50,
        'min_space': 5,
        'sloping_atr_length': 14,
        'take_profit_multiplier': 2.0,
        'stop_loss_multiplier': 1.5,
        'leverage': 10,
        'allocation': 0.5  # 50% от общего капитала
    },
    {
        'symbol': 'ETHUSDT',
        'interval': '1h',
        'window_length': 40,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.8,
        'stop_loss_multiplier': 1.2,
        'leverage': 5,
        'allocation': 0.5  # 50% от общего капитала
    }
]

# Запуск
start_date = '2023-10'
end_date = '2024-11'
initial_cash = 1000
run_combined_backtest(coins_config, start_date, end_date, initial_cash)


Начальный капитал: 1000.00


<ipython-input-67-a156ed8f2e0d>:129: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SlopingStrategy, cash=trade_cash, commission=0.0005, margin=1/coin['leverage'])


BTCUSDT: PnL=0.00, Общий капитал: 1000.00


<ipython-input-67-a156ed8f2e0d>:129: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SlopingStrategy, cash=trade_cash, commission=0.0005, margin=1/coin['leverage'])


ETHUSDT: PnL=-74.23, Общий капитал: 925.77

Детали сделок сохранены в 'trade_details.csv'.

Итоговый капитал: 925.77


In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest, Strategy

# Класс стратегии для backtesting
class SlopingStrategy(Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5
    take_profit_multiplier = 1.5
    trade_on_close = True

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        self.slope_indicator.add_kline(
            self.data.index[-1].timestamp(),
            self.data.Open[-1],
            self.data.High[-1],
            self.data.Low[-1],
            self.data.Close[-1]
        )

        signal = self.slope_indicator.get_value()

        if signal and signal.side and not self.position:
            current_atr = signal.atr
            stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
            take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
            self.buy(sl=stop_loss, tp=take_profit)

        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
            take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
            self.sell(sl=stop_loss, tp=take_profit)

# Генерация месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []
    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date += relativedelta(months=1)
    return months

# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {'Date': [], 'Open': [], 'High': [], 'Low': [], 'Close': [], 'Volume': []}

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        if not os.path.exists(file_path):
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
        except zipfile.BadZipFile:
            print(f"Ошибка: файл {file_path} поврежден. Пропускаем.")
            continue

    if not klines['Date']:
        raise ValueError(f"Нет данных для {symbol} в диапазоне {start_date} - {end_date}")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

# Основной тест с фиксированной суммой
def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    total_cash = initial_cash
    trade_details = []

    print(f"Начальный капитал: {total_cash:.2f}")

    for coin in coins_config:
        print(f"\nЗапуск теста для {coin['symbol']}...")
        try:
            data = download_klines(coin['symbol'], coin['interval'], start_date, end_date)

            SlopingStrategy.window_length = coin['window_length']
            SlopingStrategy.min_space = coin['min_space']
            SlopingStrategy.sloping_atr_length = coin['sloping_atr_length']
            SlopingStrategy.take_profit_multiplier = coin['take_profit_multiplier']
            SlopingStrategy.stop_loss_multiplier = coin['stop_loss_multiplier']

            margin = 1 / coin['leverage']
            bt = Backtest(data, SlopingStrategy, cash=coin['fixed_trade_amount'], commission=0.0005, margin=margin)
            stats = bt.run()
            trades = stats['_trades']

            pnl = stats['Equity Final [$]'] - coin['fixed_trade_amount']
            total_cash += pnl
            print(f"{coin['symbol']}: PnL={pnl:.2f}, Общий капитал: {total_cash:.2f}")

            # Сохраняем детали сделок
            for trade in trades.itertuples():
                trade_details.append({
                    'Symbol': coin['symbol'],
                    'EntryPrice': trade.EntryPrice,
                    'ExitPrice': trade.ExitPrice,
                    'PnL': trade.PnL
                })

        except Exception as e:
            print(f"Ошибка для {coin['symbol']}: {e}")

    # Сохранение данных о сделках в CSV
    trade_df = pd.DataFrame(trade_details)
    if not trade_df.empty:
        trade_df.to_csv('trade_details.csv', index=False)
        print("\nДетали сделок сохранены в 'trade_details.csv'.")
    else:
        print("\nНе удалось сохранить сделки, так как их нет.")

    print(f"\nИтоговый капитал: {total_cash:.2f}")


# Пример конфигурации
coins_config = [
    {
        'symbol': 'BTCUSDT',
        'interval': '1h',
        'window_length': 50,
        'min_space': 5,
        'sloping_atr_length': 14,
        'take_profit_multiplier': 2.0,
        'stop_loss_multiplier': 1.5,
        'leverage': 10,
        'fixed_trade_amount': 10  # Фиксированная сумма для торговли
    },
    {
        'symbol': 'ETHUSDT',
        'interval': '1h',
        'window_length': 40,
        'min_space': 5,
        'sloping_atr_length': 20,
        'take_profit_multiplier': 1.8,
        'stop_loss_multiplier': 1.2,
        'leverage': 20,
        'fixed_trade_amount': 10  # Фиксированная сумма для торговли
    }
]

# Запуск
start_date = '2023-10'
end_date = '2024-11'
initial_cash = 1000
run_combined_backtest(coins_config, start_date, end_date, initial_cash)



Начальный капитал: 1000.00

Запуск теста для BTCUSDT...


<ipython-input-63-0949f70200aa>:122: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SlopingStrategy, cash=coin['fixed_trade_amount'], commission=0.0005, margin=margin)


BTCUSDT: PnL=0.00, Общий капитал: 1000.00

Запуск теста для ETHUSDT...


<ipython-input-63-0949f70200aa>:122: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SlopingStrategy, cash=coin['fixed_trade_amount'], commission=0.0005, margin=margin)


ETHUSDT: PnL=0.00, Общий капитал: 1000.00

Не удалось сохранить сделки, так как их нет.

Итоговый капитал: 1000.00


In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest, Strategy


# Класс стратегии для backtesting
class SlopingStrategy(backtesting.Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5  # Мультиплеер для стоп-лосса
    take_profit_multiplier = 1.5  # Мультиплеер для тейк-профита
    trade_on_close = True  # Добавляем trade_on_close как переменную класса
    leverage = 1.0  # Начальное плечо

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        # Добавляем текущие данные свечей в индикатор
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])

        # Получаем сигнал от индикатора
        signal = self.slope_indicator.get_value()

        # Если сигнал на покупку и нет позиции
        if signal and signal.side and not self.position:
            # Текущее значение ATR для текущей свечи
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass

        # Если сигнал на продажу и нет позиции
        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
                self.sell(sl=stop_loss, tp=take_profit)
            except:
                pass


# Генерация месяцев
def generate_months(start_date, end_date):
    """Генерирует список месяцев в заданном интервале."""
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months


# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    """Скачивает исторические данные свечей с Binance и возвращает DataFrame."""
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    first_valid_month = None

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        # Скачиваем файл, если он отсутствует
        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        # Проверяем, что файл является корректным ZIP и содержит данные
        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():  # Пропускаем заголовки
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
            if first_valid_month is None and klines['Date']:
                first_valid_month = month  # Устанавливаем первый валидный месяц
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    # Если данных нет, вызываем ошибку
    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    print(f"Для {symbol} данные доступны с {first_valid_month}.")

    # Создаем DataFrame
    df = pd.DataFrame(klines)
    if not df.empty:
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)

    return df


# Основной тест с учетом общего депозита и плеча
def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    import pandas as pd
    from backtesting import Backtest

    cash = initial_cash  # Общий капитал
    monthly_stats = {}
    trade_details = []  # Для сохранения данных сделок в CSV файл

    # Список месяцев в диапазоне
    months = generate_months(start_date, end_date)

    # Итерация по месяцам
    for month in months:
        print(f"\nОбработка месяца: {month}")

        for coin in coins_config:
            symbol = coin['symbol']
            interval = coin['interval']

            # Загрузка данных только для текущего месяца
            try:
                data = download_klines(symbol, interval, month, month)
                if data.empty:
                    print(f"Данные для {symbol} за {month} пусты. Пропускаем.")
                    continue
            except Exception as e:
                print(f"Ошибка при загрузке данных для {symbol}: {e}")
                continue

            # Расчет доступного капитала для этой монеты
            available_cash = max(cash * coin['trade_amount'] / initial_cash, 0.01)  # Минимум 0.01 для торговли

            if available_cash < 0.01:
                print(f"Капитал для {symbol} слишком мал ({available_cash:.2f}). Пропускаем.")
                continue

            # Настройка параметров стратегии
            SlopingStrategy.window_length = coin['window_length']
            SlopingStrategy.min_space = coin['min_space']
            SlopingStrategy.sloping_atr_length = coin['sloping_atr_length']
            SlopingStrategy.take_profit_multiplier = coin['take_profit_multiplier']
            SlopingStrategy.stop_loss_multiplier = coin['stop_loss_multiplier']

            # Выполняем бэктест
            try:
                bt = Backtest(data, SlopingStrategy, cash=available_cash, commission=0.0005)
                stats = bt.run(trade_on_close=True, leverage=coin['leverage'])
            except Exception as e:
                print(f"Ошибка при выполнении бэктеста для {symbol}: {e}")
                continue

            # Обновляем капитал
            pnl = stats['Equity Final [$]'] - available_cash  # PnL для этой монеты
            cash += pnl
            print(f"{symbol} за {month}: PnL={pnl:.2f}, Остаток капитала: {cash:.2f}")

            # Обновление статистики
            if month not in monthly_stats:
                monthly_stats[month] = {'total_pnl': 0, 'trades': 0, 'profit_trades': 0, 'loss_trades': 0}

            monthly_stats[month]['total_pnl'] += pnl
            trades = stats['_trades']
            monthly_stats[month]['trades'] += trades.shape[0]

            profit_trades = trades[trades['PnL'] > 0]
            loss_trades = trades[trades['PnL'] <= 0]

            monthly_stats[month]['profit_trades'] += profit_trades.shape[0]
            monthly_stats[month]['loss_trades'] += loss_trades.shape[0]

            # Сохранение данных о сделках
            for trade in trades.itertuples():
                entry_price = getattr(trade, 'EntryPrice', 'N/A')
                exit_price = getattr(trade, 'ExitPrice', 'N/A')
                pnl_trade = getattr(trade, 'PnL', 'N/A')
                trade_details.append([symbol, month, entry_price, exit_price, pnl_trade])

            # Проверяем, не обнулился ли общий депозит
            if cash <= 0:
                print("Общий капитал исчерпан. Прекращение теста.")
                break

        # Если капитал обнулился, выходим из цикла по месяцам
        if cash <= 0:
            print("Общий капитал исчерпан. Завершение всех тестов.")
            break

    # Вывод статистики по месяцам
    print(f"\nИтоговый PnL: {cash - initial_cash:.2f}")
    print(f"Итоговый баланс: {cash:.2f}")

    for month, stats in monthly_stats.items():
        total_trades = stats['trades']
        profit_trades = stats['profit_trades']
        loss_trades = stats['loss_trades']
        success_percentage = (profit_trades / total_trades) * 100 if total_trades > 0 else 0
        print(f"\nМесяц {month}:")
        print(f"  Всего сделок: {total_trades}")
        print(f"  Профитных сделок: {profit_trades}")
        print(f"  Убыточных сделок: {loss_trades}")
        print(f"  Процент успешных сделок: {success_percentage:.2f}%")
        print(f"  Общий PnL: {stats['total_pnl']:.2f}")

    # Сохранение данных о сделках в CSV файл
    trade_df = pd.DataFrame(trade_details, columns=['Symbol', 'Month', 'EntryPrice', 'ExitPrice', 'PnL'])
    trade_df.to_csv('trade_details.csv', index=False)
    print("\nДетали сделок сохранены в 'trade_details.csv'.")


coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.80623414832759,
        'stop_loss_multiplier': 2.59902614088353,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'BTCUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.5613071562406,
        'stop_loss_multiplier': 1.01798896769172,
        'trade_amount': 10,
        'leverage': 20
    },
]

# Запуск теста
start_date = '2023-10'
end_date = '2024-11'
run_combined_backtest(coins_config, start_date, end_date, initial_cash=30)



Обработка месяца: 2023-10
Для 1000PEPEUSDT данные доступны с 2023-10.
1000PEPEUSDT за 2023-10: PnL=-0.36, Остаток капитала: 29.64
Для BTCUSDT данные доступны с 2023-10.


<ipython-input-76-d328daa641d9>:186: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SlopingStrategy, cash=available_cash, commission=0.0005)


BTCUSDT за 2023-10: PnL=0.00, Остаток капитала: 29.64

Обработка месяца: 2023-11
Для 1000PEPEUSDT данные доступны с 2023-11.
1000PEPEUSDT за 2023-11: PnL=2.63, Остаток капитала: 32.28
Для BTCUSDT данные доступны с 2023-11.


<ipython-input-76-d328daa641d9>:186: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SlopingStrategy, cash=available_cash, commission=0.0005)


BTCUSDT за 2023-11: PnL=0.00, Остаток капитала: 32.28

Обработка месяца: 2023-12
Для 1000PEPEUSDT данные доступны с 2023-12.
1000PEPEUSDT за 2023-12: PnL=-1.68, Остаток капитала: 30.60
Для BTCUSDT данные доступны с 2023-12.


...

BTCUSDT за 2024-09: PnL=0.00, Остаток капитала: 51.99

Обработка месяца: 2024-10
Для 1000PEPEUSDT данные доступны с 2024-10.
1000PEPEUSDT за 2024-10: PnL=3.75, Остаток капитала: 55.73
Для BTCUSDT данные доступны с 2024-10.


<ipython-input-76-d328daa641d9>:186: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SlopingStrategy, cash=available_cash, commission=0.0005)


BTCUSDT за 2024-10: PnL=0.00, Остаток капитала: 55.73

Обработка месяца: 2024-11
Для 1000PEPEUSDT данные доступны с 2024-11.
1000PEPEUSDT за 2024-11: PnL=-1.13, Остаток капитала: 54.60
Для BTCUSDT данные доступны с 2024-11.


<ipython-input-76-d328daa641d9>:186: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SlopingStrategy, cash=available_cash, commission=0.0005)


BTCUSDT за 2024-11: PnL=0.00, Остаток капитала: 54.60

Итоговый PnL: 24.60
Итоговый баланс: 54.60

Месяц 2023-10:
  Всего сделок: 31
  Профитных сделок: 16
  Убыточных сделок: 15
  Процент успешных сделок: 51.61%
  Общий PnL: -0.36

Месяц 2023-11:
  Всего сделок: 30
  Профитных сделок: 20
  Убыточных сделок: 10
  Процент успешных сделок: 66.67%
  Общий PnL: 2.63

Месяц 2023-12:
  Всего сделок: 37
  Профитных сделок: 21
  Убыточных сделок: 16
  Процент успешных сделок: 56.76%
  Общий PnL: -1.68

Месяц 2024-01:
  Всего сделок: 33
  Профитных сделок: 17
  Убыточных сделок: 16
  Процент успешных сделок: 51.52%
  Общий PnL: -0.84

Месяц 2024-02:
  Всего сделок: 33
  Профитных сделок: 19
  Убыточных сделок: 14
  Процент успешных сделок: 57.58%
  Общий PnL: 0.62

Месяц 2024-03:
  Всего сделок: 39
  Профитных сделок: 26
  Убыточных сделок: 13
  Процент успешных сделок: 66.67%
  Общий PnL: 1.79

Месяц 2024-04:
  Всего сделок: 33
  Профитных сделок: 23
  Убыточных сделок: 10
  Процент успешных с

In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest, Strategy




# Класс стратегии для backtesting
class SlopingStrategy(backtesting.Strategy):

    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5  # Мультиплеер для стоп-лосса
    take_profit_multiplier = 1.5  # Мультиплеер для тейк-профита
    trade_on_close = True # Add trade_on_close as a class variable
    leverage = 1.0


    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        # Добавляем текущие данные свечей в индикатор
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])

        # Получаем сигнал от индикатора
        signal = self.slope_indicator.get_value()


        # Если сигнал на покупку и нет позиции
        if signal and signal.side and not self.position:
          # Текущее значение ATR для текущей свечи
          current_atr = signal.atr
          try:
            stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
            take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
            self.buy(sl=stop_loss, tp=take_profit)
          except:
            pass

        # Если сигнал на продажу и нет позиции
        elif signal and not signal.side and not self.position:
          current_atr = signal.atr
          try:
            stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
            take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
            self.sell(sl=stop_loss, tp=take_profit)
          except:
            pass


# Генерация месяцев
def generate_months(start_date, end_date):
    """Генерирует список месяцев в заданном интервале."""
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months

# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    """Скачивает исторические данные свечей с Binance и возвращает DataFrame."""
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    first_valid_month = None

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        # Скачиваем файл, если он отсутствует
        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        # Проверяем, что файл является корректным ZIP и содержит данные
        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():  # Пропускаем заголовки
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
            if first_valid_month is None and klines['Date']:
                first_valid_month = month  # Устанавливаем первый валидный месяц
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    # Если данных нет, вызываем ошибку
    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    print(f"Для {symbol} данные доступны с {first_valid_month}.")

    # Создаем DataFrame
    df = pd.DataFrame(klines)
    if not df.empty:
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)

    return df

# Основной тест с учетом общего депозита и плеча
def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    cash = initial_cash
    monthly_stats = {}

    # Сбор сделок для всех монет
    for coin in coins_config:
        symbol = coin['symbol']
        interval = coin['interval']
        data = download_klines(symbol, interval, start_date, end_date)

        # Настройка параметров стратегии
        SlopingStrategy.window_length = coin['window_length']
        SlopingStrategy.min_space = coin['min_space']
        SlopingStrategy.sloping_atr_length = coin['sloping_atr_length']
        SlopingStrategy.take_profit_multiplier = coin['take_profit_multiplier']
        SlopingStrategy.stop_loss_multiplier = coin['stop_loss_multiplier']

        # Выполняем бэктест, leverage больше не передается в Backtest
        bt = Backtest(data, SlopingStrategy, cash=coin['trade_amount'], commission=0.0005)
        # Вместо этого, плечо указывается в параметре 'trade_on_close'
        stats = bt.run(trade_on_close=True, leverage=coin['leverage'])

        month = start_date
        pnl = stats['Equity Final [$]'] - coin['trade_amount']
        print(f"{symbol} за {month}: PnL={pnl:.2f}")

        # Обновление статистики
        if month not in monthly_stats:
            monthly_stats[month] = {'total_pnl': 0, 'trades': 0, 'profit_trades': 0, 'loss_trades': 0}
        monthly_stats[month]['total_pnl'] += pnl
        monthly_stats[month]['trades'] += stats['_trades'].shape[0]
        monthly_stats[month]['profit_trades'] += stats['_trades'][stats['_trades']['PnL'] > 0].shape[0]
        monthly_stats[month]['loss_trades'] += stats['_trades'][stats['_trades']['PnL'] <= 0].shape[0]

    # Финальные расчеты
    total_pnl = sum(monthly_stats[m]['total_pnl'] for m in monthly_stats)
    print(f"\nИтоговый PnL: {total_pnl:.2f}")
    print(f"Итоговый баланс: {cash + total_pnl:.2f}")

    # Вывод по месяцам
    for month, stats in monthly_stats.items():
        print(f"Месяц {month}: PnL={stats['total_pnl']:.2f}, Сделки={stats['trades']}, "
              f"Профитные={stats['profit_trades']}, Убыточные={stats['loss_trades']}")

# Конфигурация монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.80623414832759,
        'stop_loss_multiplier': 2.59902614088353,
        'trade_amount': 10,
        'leverage': 20
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.5613071562406,
        'stop_loss_multiplier': 1.01798896769172,
        'trade_amount': 10,
        'leverage': 20
    },
]

# Запуск теста
start_date = '2023-10'
end_date = '2024-11'
run_combined_backtest(coins_config, start_date, end_date, initial_cash=30)


Для 1000PEPEUSDT данные доступны с 2023-10.
1000PEPEUSDT за 2023-10: PnL=51.21
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Для ORDIUSDT данные доступны с 2023-11.


<ipython-input-52-ba30d5b9f00f>:162: UserWarning: Some prices are larger than initial cash value. Note that fractional trading is not supported. If you want to trade Bitcoin, increase initial cash, or trade μBTC or satoshis instead (GH-134).
  bt = Backtest(data, SlopingStrategy, cash=coin['trade_amount'], commission=0.0005)


ORDIUSDT за 2023-10: PnL=0.00

Итоговый PnL: 51.21
Итоговый баланс: 81.21
Месяц 2023-10: PnL=51.21, Сделки=496, Профитные=319, Убыточные=177


In [ ]:
import os
import csv
import io
import zipfile
import requests
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
import pandas as pd
from backtesting import Backtest


# Генерация месяцев
def generate_months(start_date, end_date):
    """Генерирует список месяцев в заданном интервале."""
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []

    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)

    return months

# Загрузка данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    """Скачивает исторические данные свечей с Binance и возвращает DataFrame."""
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    first_valid_month = None

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        file_path = f"klines/{filename}"

        # Скачиваем файл, если он отсутствует
        if not os.path.exists(file_path) or os.path.getsize(file_path) == 0:
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            try:
                r = requests.get(url, allow_redirects=True)
                with open(file_path, 'wb') as f:
                    f.write(r.content)
            except Exception as e:
                print(f"Ошибка загрузки файла {filename}: {e}. Пропускаем.")
                continue

        # Проверяем, что файл является корректным ZIP и содержит данные
        try:
            with zipfile.ZipFile(file_path, 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():  # Пропускаем заголовки
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))
            if first_valid_month is None and klines['Date']:
                first_valid_month = month  # Устанавливаем первый валидный месяц
        except (zipfile.BadZipFile, KeyError):
            print(f"Ошибка: Файл {file_path} поврежден или не является ZIP. Пропускаем.")
            continue

    # Если данных нет, вызываем ошибку
    if not klines['Date']:
        raise ValueError(f"Нет доступных данных для {symbol} в диапазоне {start_date} - {end_date}")

    print(f"Для {symbol} данные доступны с {first_valid_month}.")

    # Создаем DataFrame
    df = pd.DataFrame(klines)
    if not df.empty:
        df['Date'] = pd.to_datetime(df['Date'])
        df.set_index('Date', inplace=True)

    return df



# Основной тест с учетом общего депозита
def run_combined_backtest(coins_config, start_date, end_date, initial_cash):
    cash = initial_cash
    trades_log = []
    active_positions = []

    # Сбор сделок для всех монет
    for coin in coins_config:
        symbol = coin['symbol']
        interval = coin['interval']
        data = download_klines(symbol, interval, start_date, end_date)

        # Настройка параметров стратегии
        SlopingStrategy.window_length = coin['window_length']
        SlopingStrategy.min_space = coin['min_space']
        SlopingStrategy.sloping_atr_length = coin['sloping_atr_length']
        SlopingStrategy.take_profit_multiplier = coin['take_profit_multiplier']
        SlopingStrategy.stop_loss_multiplier = coin['stop_loss_multiplier']

        # Выполняем бэктест
        bt = Backtest(data, SlopingStrategy, cash=initial_cash, commission=0.0005)
        trades = bt.run()._trades
        trades['symbol'] = symbol
        trades_log.extend(trades.to_dict('records'))

    # Сортировка сделок по времени
    trades_log = sorted(trades_log, key=lambda x: x['EntryTime'])

    # Эмуляция торговли
    monthly_stats = {}
    for trade in trades_log:
        # Закрытие активных позиций
        closed_trades = [pos for pos in active_positions if pos['ExitTime'] <= trade['EntryTime']]
        for closed in closed_trades:
            pnl = closed['PnL']
            cash += pnl
            active_positions.remove(closed)

        # Открытие новой позиции
        position_size = cash * 0.01  # Например, 10% от депозита
        if cash >= position_size:
            cash -= position_size
            active_positions.append({
                'symbol': trade['symbol'],
                'EntryTime': trade['EntryTime'],
                'ExitTime': trade['ExitTime'],
                'PnL': trade['PnL'] * position_size / trade['Size'],
                'Size': position_size
            })
            # Обновляем статистику
            month = trade['EntryTime'].strftime('%Y-%m')
            if month not in monthly_stats:
                monthly_stats[month] = {'total_pnl': 0, 'trades': 0, 'profit_trades': 0, 'loss_trades': 0}
            monthly_stats[month]['total_pnl'] += trade['PnL']
            monthly_stats[month]['trades'] += 1
            if trade['PnL'] > 0:
                monthly_stats[month]['profit_trades'] += 1
            else:
                monthly_stats[month]['loss_trades'] += 1

    # Финальные расчеты
    total_pnl = sum(pos['PnL'] for pos in active_positions)
    print(f"Итоговый PnL: {total_pnl:.2f}")
    print(f"Итоговый баланс: {cash + total_pnl:.2f}")

    # Вывод по месяцам
    for month, stats in monthly_stats.items():
        print(f"Месяц {month}: PnL={stats['total_pnl']:.2f}, Сделки={stats['trades']}, "
              f"Профитные={stats['profit_trades']}, Убыточные={stats['loss_trades']}")

# Конфигурация монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.80623414832759,
        'stop_loss_multiplier': 2.59902614088353
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.5613071562406,
        'stop_loss_multiplier': 1.01798896769172
    },
    # Остальные монеты...
]

# Запуск теста
start_date = '2023-10'
end_date = '2024-11'
run_combined_backtest(coins_config, start_date, end_date, initial_cash=1_000_000)


Для 1000PEPEUSDT данные доступны с 2023-10.
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Для ORDIUSDT данные доступны с 2023-11.
Итоговый PnL: 0.94
Итоговый баланс: 127.98
Месяц 2023-10: PnL=-35839.84, Сделки=31, Профитные=16, Убыточные=15
Месяц 2023-11: PnL=1387475.62, Сделки=53, Профитные=33, Убыточные=20
Месяц 2023-12: PnL=154333.83, Сделки=70, Профитные=33, Убыточные=37
Месяц 2024-01: PnL=224468.24, Сделки=74, Профитные=32, Убыточные=42
Месяц 2024-02: PnL=350022.90, Сделки=65, Профитные=32, Убыточные=33
Месяц 2024-03: PnL=1166968.49, Сделки=71, Профитные=40, Убыточные=31
Месяц 2024-04: PnL=446267.27, Сделки=58, Профитные=32, Убыточные=26
Месяц 2024-05: PnL=533438.68, Сделки=66, Профитные=32, Убыточные=34
Месяц 2024-06: PnL=1440238.54, Сделки=61, Профитные=35, Убыточные=26
Месяц 2024-07: PnL=2125910.32, Сделки=71, Профитные=39, Убыточные=32
Месяц 2024-08: PnL=2258836.75, Сделки=55, Профитные=30, Убыточные=25
Месяц 2024-09: PnL=1309898.25, Сд

Для 1000PEPEUSDT данные доступны с 2023-10.
Ошибка: Файл klines/ORDIUSDT-1h-2023-10.zip поврежден или не является ZIP. Пропускаем.
Для ORDIUSDT данные доступны с 2023-11.
Итоговый PnL: 210000.00
Итоговый баланс: 1210000.00
[{'symbol': '1000PEPEUSDT',
  'results': {'PnL': 100000.0,
   'Trades': 204,
   'Profitable': 102,
   'Losing': 102}},
 {'symbol': 'ORDIUSDT',
  'results': {'PnL': 110000.0, 'Trades': 93, 'Profitable': 46, 'Losing': 47}}]

In [ ]:
import os
import requests
import zipfile
import csv
import io
import pandas as pd
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from backtesting import Backtest, Strategy
from concurrent.futures import ThreadPoolExecutor

# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []
    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)
    return months

# Загрузка исторических данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)

    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        if not os.path.exists(f"klines/{filename}"):
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            r = requests.get(url, allow_redirects=True)
            if r.status_code == 200:
                open(f"klines/{filename}", 'wb').write(r.content)
            else:
                continue

        with zipfile.ZipFile(f"klines/{filename}", 'r') as zip_file:
            with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                for row in csv_reader:
                    if row[0].isdigit():
                        klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                        klines['Open'].append(float(row[1]))
                        klines['High'].append(float(row[2]))
                        klines['Low'].append(float(row[3]))
                        klines['Close'].append(float(row[4]))
                        klines['Volume'].append(float(row[5]))

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

# Автоматический подбор стартовой даты
def get_available_data(symbol, interval, start_date, end_date):
    current_start_date = start_date
    while True:
        try:
            data = download_klines(symbol, interval, current_start_date, end_date)
            if not data.empty:
                return data
        except Exception:
            pass
        current_start_date = (datetime.strptime(current_start_date, '%Y-%m') + relativedelta(months=1)).strftime('%Y-%m')
        if datetime.strptime(current_start_date, '%Y-%m') > datetime.strptime(end_date, '%Y-%m'):
            break
    return pd.DataFrame()  # Возвращаем пустой DataFrame, если данные отсутствуют

# Класс стратегии для бэктестинга
class SlopingStrategy(Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5  # Мультиплеер для стоп-лосса
    take_profit_multiplier = 1.5  # Мультиплеер для тейк-профита

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        # Добавляем текущие данные свечей в индикатор
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])

        # Получаем сигнал от индикатора
        signal = self.slope_indicator.get_value()

        # Если сигнал на покупку и нет позиции
        if signal and signal.side and not self.position:
            # Текущее значение ATR для текущей свечи
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass

        # Если сигнал на продажу и нет позиции
        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
                self.sell(sl=stop_loss, tp=take_profit)
            except:
                pass

# Функция для бэктестинга одной монеты
def run_backtest_for_coin(coin, strategy_class, start_date, end_date, initial_cash, commission):
    symbol = coin['symbol']
    interval = coin['interval']
    print(f"Обработка {symbol} с интервалом {interval}...")
    data = get_available_data(symbol, interval, start_date, end_date)
    if data.empty:
        print(f"Нет доступных данных для {symbol}. Пропуск.")
        return None

    # Установка уникальных параметров стратегии
    strategy_class.window_length = coin['window_length']
    strategy_class.min_space = coin['min_space']
    strategy_class.sloping_atr_length = coin['sloping_atr_length']
    strategy_class.take_profit_multiplier = coin['take_profit_multiplier']
    strategy_class.stop_loss_multiplier = coin['stop_loss_multiplier']

    print(f"Запуск стратегии для {symbol}...")
    test = Backtest(data, strategy_class, cash=initial_cash, commission=commission)
    res = test.run()

    print(f"Результаты для {symbol}:")
    print(res)
    return res

# Параллельный запуск бэктестинга для всех монет
def run_parallel_backtest(config, strategy_class, start_date, end_date, initial_cash=1_000_000, commission=0.0005):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(lambda coin: run_backtest_for_coin(coin, strategy_class, start_date, end_date, initial_cash, commission), config))
    return results

# Пример конфигурации монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.80623414832759,
        'stop_loss_multiplier': 2.59902614088353
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.5613071562406,
        'stop_loss_multiplier': 1.01798896769172
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.9739578464886,
        'stop_loss_multiplier': 2.43800857400041
    },
    {
        'symbol': 'TIAUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 2.2,
        'stop_loss_multiplier': 1.6
    }
]

start_date = '2023-01'
end_date = '2024-12'

# Запуск параллельного бэктестинга
summary = run_parallel_backtest(coins_config, SlopingStrategy, start_date, end_date)
print(summary)


Обработка 1000PEPEUSDT с интервалом 30m...Обработка ORDIUSDT с интервалом 1h...
Обработка SUIUSDT с интервалом 1h...
Обработка TIAUSDT с интервалом 1h...

Запуск стратегии для TIAUSDT...
Запуск стратегии для ORDIUSDT...
Запуск стратегии для SUIUSDT...
Запуск стратегии для 1000PEPEUSDT...
Результаты для TIAUSDT:
Start                     2023-10-31 18:00...
End                       2024-11-30 23:00...
Duration                    396 days 05:00:00
Exposure Time [%]                   78.128286
Equity Final [$]                105558.159011
Equity Peak [$]                1438902.191137
Return [%]                         -89.444184
Buy & Hold Return [%]              268.891238
Return (Ann.) [%]                  -87.346729
Volatility (Ann.) [%]               19.393218
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -93.475571
Avg. Drawdown [%]                  -14.4368

In [ ]:
def download_klines(symbol, interval, start_date, end_date):
    """Скачивает исторические данные свечей с Binance с учётом доступных данных."""
    months = generate_months(start_date, end_date)
    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"

        try:
            # Скачиваем файл, если он не существует
            if not os.path.exists(f"klines/{filename}"):
                r = requests.get(url, allow_redirects=True)
                if r.status_code == 200:
                    open(f"klines/{filename}", 'wb').write(r.content)
                else:
                    print(f"Нет данных для {symbol} за {month}")
                    continue

            # Распаковываем данные
            with zipfile.ZipFile(f"klines/{filename}", 'r') as zip_file:
                with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                    csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                    for row in csv_reader:
                        if row[0].isdigit():
                            klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                            klines['Open'].append(float(row[1]))
                            klines['High'].append(float(row[2]))
                            klines['Low'].append(float(row[3]))
                            klines['Close'].append(float(row[4]))
                            klines['Volume'].append(float(row[5]))

        except Exception as e:
            print(f"Ошибка при обработке {symbol} за {month}: {e}")

    if not klines['Date']:
        raise ValueError(f"Нет данных для {symbol} за указанный период.")

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df


In [ ]:
def combine_data(symbols, intervals, start_date, end_date):
    """Загружает и объединяет данные для всех символов."""
    combined_data = {}

    for symbol, interval in zip(symbols, intervals):
        try:
            df = download_klines(symbol, interval, start_date, end_date)
            combined_data[symbol] = df
        except ValueError as e:
            print(e)  # Например, "Нет данных для TIA за указанный период"

    # Объединяем данные в единую временную шкалу
    all_dates = pd.concat([df.index.to_series() for df in combined_data.values()]).drop_duplicates().sort_values()
    combined_df = pd.DataFrame(index=all_dates)

    for symbol, df in combined_data.items():
        combined_df[symbol] = df['Close']

    return combined_df, combined_data


In [ ]:
def backtest_combined(data, strategies_params, initial_deposit=100000, risk_per_trade=0.01):
    """
    Проводит тест для нескольких стратегий на одном депозите.
    """
    deposit = initial_deposit
    positions = []

    for date, row in data.iterrows():
        for symbol, params in strategies_params.items():
            # Проверяем, есть ли данные по символу
            if pd.isna(row[symbol]):
                continue

            # Здесь вы реализуете логику запуска каждой стратегии
            # Например, сигнал на покупку/продажу и контроль депозита
            pass

        # Обновление позиций и депозита
        # Реализуйте управление открытыми позициями (PnL, закрытие сделок)
        pass

    return deposit, positions


In [ ]:
import os
import requests
import zipfile
import csv
import io
import pandas as pd
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from backtesting import Backtest

# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []
    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)
    return months

# Загрузка исторических данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)

    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        if not os.path.exists(f"klines/{filename}"):
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            r = requests.get(url, allow_redirects=True)
            if r.status_code == 200:
                open(f"klines/{filename}", 'wb').write(r.content)
            else:
                continue

        with zipfile.ZipFile(f"klines/{filename}", 'r') as zip_file:
            with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                for row in csv_reader:
                    if row[0].isdigit():
                        klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                        klines['Open'].append(float(row[1]))
                        klines['High'].append(float(row[2]))
                        klines['Low'].append(float(row[3]))
                        klines['Close'].append(float(row[4]))
                        klines['Volume'].append(float(row[5]))

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

# Автоматический подбор стартовой даты
def get_available_data(symbol, interval, start_date, end_date):
    current_start_date = start_date
    while True:
        try:
            data = download_klines(symbol, interval, current_start_date, end_date)
            if not data.empty:
                return data
        except Exception:
            pass
        current_start_date = (datetime.strptime(current_start_date, '%Y-%m') + relativedelta(months=1)).strftime('%Y-%m')
        if datetime.strptime(current_start_date, '%Y-%m') > datetime.strptime(end_date, '%Y-%m'):
            break
    return pd.DataFrame()  # Возвращаем пустой DataFrame, если данные отсутствуют

# Бэктестинг
def run_backtest(config, strategy_class, start_date, end_date, initial_cash=1_000_000, commission=0.0005):
    results = {}
    for coin in config:
        symbol = coin['symbol']
        interval = coin['interval']
        print(f"Обработка {symbol} с интервалом {interval}...")
        data = get_available_data(symbol, interval, start_date, end_date)
        if data.empty:
            print(f"Нет доступных данных для {symbol}. Пропуск.")
            continue

        # Установка уникальных параметров стратегии
        strategy_class.window_length = coin['window_length']
        strategy_class.min_space = coin['min_space']
        strategy_class.sloping_atr_length = coin['sloping_atr_length']
        strategy_class.take_profit_multiplier = coin['take_profit_multiplier']
        strategy_class.stop_loss_multiplier = coin['stop_loss_multiplier']

        print(f"Запуск стратегии для {symbol}...")
        test = Backtest(data, strategy_class, cash=initial_cash, commission=commission)
        res = test.run()
        results[symbol] = res

        print(f"Результаты для {symbol}:")
        print(res)
    return results



###### более или менее рабоичй

In [ ]:



def run_multi_asset_backtest(config, strategy_class, start_date, end_date, initial_cash=1_000_000, commission=0.0005):
    """
    Бэктестинг с одновременным учетом нескольких монет на общем депозите.
    """
    total_cash = initial_cash  # Общий депозит
    results_summary = []
    trades = []

    print(f"Запуск бэктестинга с общим депозитом: {initial_cash}")

    for coin in config:
        symbol = coin['symbol']
        interval = coin['interval']
        print(f"\nОбработка {symbol} с интервалом {interval}...")

        # Загрузка данных
        data = get_available_data(symbol, interval, start_date, end_date)
        if data.empty:
            print(f"Нет доступных данных для {symbol}. Пропуск.")
            continue

        # Установка параметров стратегии
        strategy_class.window_length = coin['window_length']
        strategy_class.min_space = coin['min_space']
        strategy_class.sloping_atr_length = coin['sloping_atr_length']
        strategy_class.take_profit_multiplier = coin['take_profit_multiplier']
        strategy_class.stop_loss_multiplier = coin['stop_loss_multiplier']

        # Запуск бэктестинга
        print(f"Запуск стратегии для {symbol}...")
        test = Backtest(data, strategy_class, cash=total_cash, commission=commission)
        res = test.run()

        # Обновление общего депозита
        pnl = res['Equity Final [$]'] - total_cash  # Прибыль/убыток
        total_cash += pnl
        results_summary.append({
            'symbol': symbol,
            'total_pnl': pnl,
            'final_cash': total_cash,
            'trades': res['# Trades'],
            'win_rate': res['Win Rate [%]'],
            'profit_factor': res['Profit Factor']
        })
        trades.append(res['_trades'])

        print(f"Результаты для {symbol}:")
        print(res[['Equity Final [$]', '# Trades', 'Win Rate [%]', 'Profit Factor']])

    print("\nОбщий результат по всем монетам:")
    for summary in results_summary:
        print(summary)

    print(f"\nИтоговый депозит: {total_cash}")
    return results_summary, trades

# Конфигурация для монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.80623414832759,
        'stop_loss_multiplier': 2.59902614088353
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.5613071562406,
        'stop_loss_multiplier': 1.01798896769172
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.9739578464886,
        'stop_loss_multiplier': 2.43800857400041
    },
    {
        'symbol': 'TIAUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 2.2,
        'stop_loss_multiplier': 1.6
    }
]

start_date = '2023-10'
end_date = '2024-11'

# Запуск
summary, all_trades = run_multi_asset_backtest(coins_config, SlopingStrategy, start_date, end_date)

# Анализ результатов
print("\nСводка результатов по монетам:")
for result in summary:
    print(result)

Запуск бэктестинга с общим депозитом: 1000000

Обработка 1000PEPEUSDT с интервалом 30m...
Запуск стратегии для 1000PEPEUSDT...
Результаты для 1000PEPEUSDT:
Equity Final [$]    6123579.038187
# Trades                       496
Win Rate [%]             64.314516
Profit Factor              1.31241
dtype: object

Обработка ORDIUSDT с интервалом 1h...
Запуск стратегии для ORDIUSDT...
Результаты для ORDIUSDT:
Equity Final [$]    55400503.165962
# Trades                        373
Win Rate [%]              36.729223
Profit Factor              1.468714
dtype: object

Обработка SUIUSDT с интервалом 1h...
Запуск стратегии для SUIUSDT...
Результаты для SUIUSDT:
Equity Final [$]    141908044.49184
# Trades                        532
Win Rate [%]              57.894737
Profit Factor              1.140464
dtype: object

Обработка TIAUSDT с интервалом 1h...
Запуск стратегии для TIAUSDT...
Результаты для TIAUSDT:
Equity Final [$]    10898491.904267
# Trades                        573
Win Rate [%]     

##### ДРАФТ

In [ ]:
import os
import requests
import zipfile
import csv
import io
import pandas as pd
from datetime import datetime, timezone
from dateutil.relativedelta import relativedelta
from backtesting import Backtest

# Генерация списка месяцев
def generate_months(start_date, end_date):
    start_date = datetime.strptime(start_date, '%Y-%m')
    end_date = datetime.strptime(end_date, '%Y-%m')
    months = []
    while start_date <= end_date:
        months.append(start_date.strftime('%Y-%m'))
        start_date = start_date + relativedelta(months=1)
    return months

# Загрузка исторических данных с Binance
def download_klines(symbol, interval, start_date, end_date):
    months = generate_months(start_date, end_date)

    if not os.path.exists('klines'):
        os.mkdir('klines')

    klines = {
        'Date': [],
        'Open': [],
        'High': [],
        'Low': [],
        'Close': [],
        'Volume': []
    }

    for month in months:
        filename = f"{symbol}-{interval}-{month}.zip"
        if not os.path.exists(f"klines/{filename}"):
            url = f"https://data.binance.vision/data/futures/um/monthly/klines/{symbol}/{interval}/{filename}"
            r = requests.get(url, allow_redirects=True)
            if r.status_code == 200:
                open(f"klines/{filename}", 'wb').write(r.content)
            else:
                continue

        with zipfile.ZipFile(f"klines/{filename}", 'r') as zip_file:
            with zip_file.open(f"{symbol}-{interval}-{month}.csv", 'r') as csv_file:
                csv_reader = csv.reader(io.TextIOWrapper(csv_file, 'utf-8'))
                for row in csv_reader:
                    if row[0].isdigit():
                        klines['Date'].append(datetime.fromtimestamp(int(row[0]) / 1000, tz=timezone.utc))
                        klines['Open'].append(float(row[1]))
                        klines['High'].append(float(row[2]))
                        klines['Low'].append(float(row[3]))
                        klines['Close'].append(float(row[4]))
                        klines['Volume'].append(float(row[5]))

    df = pd.DataFrame(klines)
    df['Date'] = pd.to_datetime(df['Date'])
    df.set_index('Date', inplace=True)
    return df

# Автоматический подбор стартовой даты
def get_available_data(symbol, interval, start_date, end_date):
    current_start_date = start_date
    while True:
        try:
            data = download_klines(symbol, interval, current_start_date, end_date)
            if not data.empty:
                return data
        except Exception:
            pass
        current_start_date = (datetime.strptime(current_start_date, '%Y-%m') + relativedelta(months=1)).strftime('%Y-%m')
        if datetime.strptime(current_start_date, '%Y-%m') > datetime.strptime(end_date, '%Y-%m'):
            break
    return pd.DataFrame()  # Возвращаем пустой DataFrame, если данные отсутствуют

# Бэктестинг
def run_backtest(config, strategy_class, start_date, end_date, initial_cash=1_000_000, commission=0.0005):
    results = {}
    for coin in config:
        symbol = coin['symbol']
        interval = coin['interval']
        print(f"Обработка {symbol} с интервалом {interval}...")
        data = get_available_data(symbol, interval, start_date, end_date)
        if data.empty:
            print(f"Нет доступных данных для {symbol}. Пропуск.")
            continue

        # Установка уникальных параметров стратегии
        strategy_class.window_length = coin['window_length']
        strategy_class.min_space = coin['min_space']
        strategy_class.sloping_atr_length = coin['sloping_atr_length']
        strategy_class.take_profit_multiplier = coin['take_profit_multiplier']
        strategy_class.stop_loss_multiplier = coin['stop_loss_multiplier']

        print(f"Запуск стратегии для {symbol}...")
        test = Backtest(data, strategy_class, cash=initial_cash, commission=commission)
        res = test.run()
        results[symbol] = res

        print(f"Результаты для {symbol}:")
        print(res)
    return results




def run_parallel_backtest(config, strategy_class, start_date, end_date, initial_cash=1_000_000, commission=0.0005):
    """
    Одновременный бэктестинг для нескольких монет с общим депозитом.
    """
    # Общий депозит
    total_cash = initial_cash
    positions = {coin['symbol']: 0 for coin in config}  # Активные позиции по каждой монете
    results_summary = []
    all_trades = []

    # Загрузка данных для всех монет
    all_data = {}
    for coin in config:
        symbol = coin['symbol']
        interval = coin['interval']
        data = get_available_data(symbol, interval, start_date, end_date)
        if data.empty:
            print(f"Нет доступных данных для {symbol}. Пропуск.")
            continue
        all_data[symbol] = data

    # Совместный временной индекс
    combined_index = sorted(
        {timestamp for data in all_data.values() for timestamp in data.index}
    )

    print(f"Запуск бэктестинга с общим депозитом: {initial_cash}")

    # Моделирование на каждом шаге времени
    for timestamp in combined_index:
        for coin in config:
            symbol = coin['symbol']
            if symbol not in all_data:
                continue

            # Проверка доступности данных на текущий шаг
            current_data = all_data[symbol].loc[timestamp:timestamp]
            if current_data.empty:
                continue

            # Установка параметров стратегии
            strategy_class.window_length = coin['window_length']
            strategy_class.min_space = coin['min_space']
            strategy_class.sloping_atr_length = coin['sloping_atr_length']
            strategy_class.take_profit_multiplier = coin['take_profit_multiplier']
            strategy_class.stop_loss_multiplier = coin['stop_loss_multiplier']

            # Применение стратегии
            decision = strategy_class.make_decision(current_data, total_cash)
            if decision['action'] == 'buy' and total_cash >= decision['cost']:
                # Открытие позиции
                positions[symbol] += decision['amount']
                total_cash -= decision['cost']
                all_trades.append({
                    'symbol': symbol,
                    'action': 'buy',
                    'timestamp': timestamp,
                    'price': decision['price'],
                    'amount': decision['amount'],
                    'total_cash': total_cash
                })
            elif decision['action'] == 'sell' and positions[symbol] > 0:
                # Закрытие позиции
                revenue = decision['price'] * decision['amount']
                positions[symbol] -= decision['amount']
                total_cash += revenue
                all_trades.append({
                    'symbol': symbol,
                    'action': 'sell',
                    'timestamp': timestamp,
                    'price': decision['price'],
                    'amount': decision['amount'],
                    'total_cash': total_cash
                })

    # Итоговая статистика
    for coin in config:
        symbol = coin['symbol']
        final_position_value = positions[symbol] * all_data[symbol]['close'].iloc[-1]
        total_pnl = final_position_value - initial_cash
        results_summary.append({
            'symbol': symbol,
            'total_pnl': total_pnl,
            'final_cash': total_cash,
            'final_position_value': final_position_value,
            'positions': positions[symbol]
        })

    print(f"\nИтоговый депозит: {total_cash}")
    return results_summary, all_trades



# Конфигурация для монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.80623414832759,
        'stop_loss_multiplier': 2.59902614088353
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.5613071562406,
        'stop_loss_multiplier': 1.01798896769172
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.9739578464886,
        'stop_loss_multiplier': 2.43800857400041
    },
    {
        'symbol': 'TIAUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 2.2,
        'stop_loss_multiplier': 1.6
    }
]

start_date = '2023-10'
end_date = '2024-11'

# Запуск параллельного бэктестинга
summary, all_trades = run_parallel_backtest(coins_config, SlopingStrategy, start_date, end_date)

# Вывод результатов
print("\nСводка результатов по монетам:")
for result in summary:
    print(result)

Запуск бэктестинга с общим депозитом: 1000000


AttributeError: type object 'SlopingStrategy' has no attribute 'make_decision'

In [ ]:
# Конфигурация для монет
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 86,
        'min_space': 5,
        'sloping_atr_length': 18,
        'take_profit_multiplier': 1.80623414832759,
        'stop_loss_multiplier': 2.59902614088353
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 42,
        'min_space': 5,
        'sloping_atr_length': 30,
        'take_profit_multiplier': 2.5613071562406,
        'stop_loss_multiplier': 1.01798896769172
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 16,
        'take_profit_multiplier': 1.9739578464886,
        'stop_loss_multiplier': 2.43800857400041
    },
    {
        'symbol': 'TIAUSDT',
        'interval': '1h',
        'window_length': 10,
        'min_space': 5,
        'sloping_atr_length': 12,
        'take_profit_multiplier': 2.2,
        'stop_loss_multiplier': 1.6
    }
]

start_date = '2023-10'
end_date = '2024-11'

# Запуск параллельного бэктестинга
summary, all_trades = run_parallel_backtest(coins_config, SlopingStrategy, start_date, end_date)

# Вывод результатов
print("\nСводка результатов по монетам:")
for result in summary:
    print(result)



Запуск бэктестинга с общим депозитом: 1000000


AttributeError: type object 'SlopingStrategy' has no attribute 'make_decision'

In [ ]:
import backtesting
import pandas as pd
from concurrent.futures import ThreadPoolExecutor


start_date = '2023-10'
end_date = '2024-11'
initial_cash = 1000000


# Функция для выполнения бэктестинга для одной монеты
def run_backtest_for_coin(coin_config):
    # Создаем экземпляр стратегии с настройками для монеты
    strategy = SlopingStrategy
    # Получаем данные для монеты (необходимо будет добавить загрузку данных)
    data = get_coin_data(coin_config['symbol'], coin_config['interval'], start_date, end_date)

    # Выполняем бэктестинг
    result = backtesting.Backtest(df, strategy, cash=initial_cash, commission=0.001).run()

    # Результаты бэктестинга
    return {
        'symbol': coin_config['symbol'],
        'final_equity': result.equity[-1],  # Итоговый капитал
        'total_pnl': result.total_return,  # Общая прибыль
        'win_rate': result.win_rate,  # Процент выигрышей
        'trades': result.trades,  # Количество сделок
    }

# Функция для получения данных монеты (это псевдокод, нужно заменить на реальную загрузку)
def get_coin_data(symbol, interval, start_date, end_date):
    # Здесь используйте свою логику для загрузки исторических данных
    # Например, из API или CSV
    pass

# Параллельный запуск для всех монет
def run_parallel_backtest(coins_config):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(run_backtest_for_coin, coins_config))

    return results

# Запуск параллельного бэктестинга
results = run_parallel_backtest(coins_config)

# Вывод итоговых результатов
for result in results:
    print(f"Результаты для {result['symbol']}:")
    print(f"Итоговый капитал: {result['final_equity']}")
    print(f"Общая прибыль: {result['total_pnl']}")
    print(f"Процент выигрышей: {result['win_rate']}%")
    print(f"Количество сделок: {result['trades']}")
    print()

ValueError: `data` must be a pandas.DataFrame with columns 'Open', 'High', 'Low', 'Close', and (optionally) 'Volume'

In [ ]:
import backtesting
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Класс стратегии для backtesting
class SlopingStrategy(backtesting.Strategy):
    # Параметры стратегии
    window_length = 50
    sloping_atr_length = 14
    min_space = 5
    stop_loss_multiplier = 1.5  # Мультиплеер для стоп-лосса
    take_profit_multiplier = 1.5  # Мультиплеер для тейк-профита

    def init(self):
        self.slope_indicator = Sloping(length=self.window_length, atr_length=self.sloping_atr_length, min_space=self.min_space)

    def next(self):
        # Добавляем текущие данные свечей в индикатор
        self.slope_indicator.add_kline(self.data.index[-1].timestamp(),
                                       self.data.Open[-1],
                                       self.data.High[-1],
                                       self.data.Low[-1],
                                       self.data.Close[-1])

        # Получаем сигнал от индикатора
        signal = self.slope_indicator.get_value()

        # Если сигнал на покупку и нет позиции
        if signal and signal.side and not self.position:
            # Текущее значение ATR для текущей свечи
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] - (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] + (self.take_profit_multiplier * current_atr)
                self.buy(sl=stop_loss, tp=take_profit)
            except:
                pass

        # Если сигнал на продажу и нет позиции
        elif signal and not signal.side and not self.position:
            current_atr = signal.atr
            try:
                stop_loss = self.data.Close[-1] + (self.stop_loss_multiplier * current_atr)
                take_profit = self.data.Close[-1] - (self.take_profit_multiplier * current_atr)
                self.sell(sl=stop_loss, tp=take_profit)
            except:
                pass


# Ваши монеты и настройки
coins_config = [
    {
        'symbol': '1000PEPEUSDT',
        'interval': '30m',
        'window_length': 50,
        'sloping_atr_length': 14,
        'min_space': 5,
        'take_profit_multiplier': 1.5,
        'stop_loss_multiplier': 1.5
    },
    {
        'symbol': 'ORDIUSDT',
        'interval': '1h',
        'window_length': 50,
        'sloping_atr_length': 14,
        'min_space': 5,
        'take_profit_multiplier': 1.5,
        'stop_loss_multiplier': 1.5
    },
    {
        'symbol': 'SUIUSDT',
        'interval': '1h',
        'window_length': 50,
        'sloping_atr_length': 14,
        'min_space': 5,
        'take_profit_multiplier': 1.5,
        'stop_loss_multiplier': 1.5
    },
    {
        'symbol': 'TIAUSDT',
        'interval': '1h',
        'window_length': 50,
        'sloping_atr_length': 14,
        'min_space': 5,
        'take_profit_multiplier': 1.5,
        'stop_loss_multiplier': 1.5
    }
]

start_date = '2023-10'
end_date = '2024-11'
initial_cash = 1000000  # Общий депозит для всех монет

# Функция для выполнения бэктестинга для одной монеты
def run_backtest_for_coin(coin_config):
    # Создаем экземпляр стратегии с настройками для монеты
    strategy = SlopingStrategy
    # Получаем данные для монеты (необходимо будет добавить загрузку данных)
    data = get_coin_data(coin_config['symbol'], coin_config['interval'], start_date, end_date)

    # Выполняем бэктестинг
    result = backtesting.Backtest(data, strategy, cash=initial_cash, commission=0.001).run()

    # Результаты бэктестинга
    return {
        'symbol': coin_config['symbol'],
        'final_equity': result.equity[-1],  # Итоговый капитал
        'total_pnl': result.total_return,  # Общая прибыль
        'win_rate': result.win_rate,  # Процент выигрышей
        'trades': result.trades,  # Количество сделок
    }

# Функция для получения данных монеты (это псевдокод, нужно заменить на реальную загрузку)
def get_coin_data(symbol, interval, start_date, end_date):
    # Здесь используйте свою логику для загрузки исторических данных
    # Например, из API или CSV
    pass

# Параллельный запуск для всех монет
def run_parallel_backtest(coins_config):
    with ThreadPoolExecutor() as executor:
        results = list(executor.map(run_backtest_for_coin, coins_config))

    return results

# Запуск параллельного бэктестинга
results = run_parallel_backtest(coins_config)

# Вывод итоговых результатов
for result in results:
    print(f"Результаты для {result['symbol']}:")
    print(f"Итоговый капитал: {result['final_equity']}")
    print(f"Общая прибыль: {result['total_pnl']}")
    print(f"Процент выигрышей: {result['win_rate']}%")
    print(f"Количество сделок: {result['trades']}")
    print()

TypeError: `data` must be a pandas.DataFrame with columns

In [ ]:
 {'symbol': '1000PEPEUSDT', 'interval': '30m', 'window_length': 86, 'min_space': 5, 'sloping_atr_length': 18, 'take_profit_multiplier': 1.80623414832759, 'stop_loss_multiplier': 2.59902614088353, 'total_pnl%': 666.215931257807, 'avg_trade_duration_hours': 7.15436241610738, 'profit_month_count': 15, 'loss_month_count': 2, '2020-10': 0.0, '2020-11': 0.0, '2020-12': 0.0, '2021-01': 0.0, '2021-02': 0.0, '2021-03': 0.0, '2021-04': 0.0, '2021-05': 0.0, '2021-06': 0.0, '2021-07': 0.0, '2021-08': 0.0, '2021-09': 0.0, '2021-10': 0.0, '2021-11': 0.0, '2021-12': 0.0, '2022-01': 0.0, '2022-02': 0.0, '2022-03': 0.0, '2022-04': 0.0, '2022-05': 0.0, '2022-06': 0.0, '2022-07': 0.0, '2022-08': 0.0, '2022-09': 0.0, '2022-10': 0.0, '2022-11': 0.0, '2022-12': 0.0, '2023-01': 0.0, '2023-02': 0.0, '2023-03': 0.0, '2023-04': 0.0, '2023-05': 0.0, '2023-06': 0.87, '2023-07': 9.29, '2023-08': 0.2, '2023-09': 4.92, '2023-10': 0.31, '2023-11': 33.28, '2023-12': -17.85, '2024-01': -7.06, '2024-02': 17.59, '2024-03': 38.08, '2024-04': 64.01, '2024-05': 9.8, '2024-06': 173.04, '2024-07': 123.46, '2024-08': 77.17, '2024-09': 72.11, '2024-10': 66.99, 'profit_month_ratio%': 88.235294117647, '2024-11': nan, 'coin_interval': '1000PEPEUSDT_30m'}
      {'symbol': 'ORDIUSDT', 'interval': '1h', 'window_length': 42, 'min_space': 5, 'sloping_atr_length': 30, 'take_profit_multiplier': 2.5613071562406, 'stop_loss_multiplier': 1.01798896769172, 'total_pnl%': 804.694538913286, 'avg_trade_duration_hours': 8.7426273458445, 'profit_month_count': 12, 'loss_month_count': 1, '2020-10': nan, '2020-11': nan, '2020-12': nan, '2021-01': nan, '2021-02': nan, '2021-03': nan, '2021-04': nan, '2021-05': nan, '2021-06': nan, '2021-07': nan, '2021-08': nan, '2021-09': nan, '2021-10': nan, '2021-11': nan, '2021-12': nan, '2022-01': nan, '2022-02': nan, '2022-03': nan, '2022-04': nan, '2022-05': nan, '2022-06': nan, '2022-07': nan, '2022-08': nan, '2022-09': nan, '2022-10': nan, '2022-11': nan, '2022-12': nan, '2023-01': nan, '2023-02': nan, '2023-03': nan, '2023-04': nan, '2023-05': nan, '2023-06': nan, '2023-07': nan, '2023-08': nan, '2023-09': nan, '2023-10': nan, '2023-11': 109.37, '2023-12': 31.94, '2024-01': 28.33, '2024-02': 20.33, '2024-03': 68.32, '2024-04': 7.84, '2024-05': 45.17, '2024-06': -7.56, '2024-07': 116.85, '2024-08': 161.52, '2024-09': 70.84, '2024-10': 47.75, 'profit_month_ratio%': 92.3076923076923, '2024-11': 103.98, 'coin_interval': 'ORDIUSDT_1h'}
      {'symbol': 'SUIUSDT', 'interval': '1h', 'window_length': 10, 'min_space': 5, 'sloping_atr_length': 16, 'take_profit_multiplier': 1.9739578464886, 'stop_loss_multiplier': 2.43800857400041, 'total_pnl%': 118.396186562069, 'avg_trade_duration_hours': 13.6679920477137, 'profit_month_count': 10, 'loss_month_count': 3, '2020-10': 0.0, '2020-11': 0.0, '2020-12': 0.0, '2021-01': 0.0, '2021-02': 0.0, '2021-03': 0.0, '2021-04': 0.0, '2021-05': 0.0, '2021-06': 0.0, '2021-07': 0.0, '2021-08': 0.0, '2021-09': 0.0, '2021-10': 0.0, '2021-11': 0.0, '2021-12': 0.0, '2022-01': 0.0, '2022-02': 0.0, '2022-03': 0.0, '2022-04': 0.0, '2022-05': 0.0, '2022-06': 0.0, '2022-07': 0.0, '2022-08': 0.0, '2022-09': 0.0, '2022-10': 0.0, '2022-11': 0.0, '2022-12': 0.0, '2023-01': 0.0, '2023-02': 0.0, '2023-03': 0.0, '2023-04': 0.0, '2023-05': 0.0, '2023-06': 0.0, '2023-07': 0.0, '2023-08': 0.0, '2023-09': 0.0, '2023-10': 18.07, '2023-11': 52.25, '2023-12': 4.88, '2024-01': 54.61, '2024-02': 22.07, '2024-03': 3.96, '2024-04': -32.38, '2024-05': 2.88, '2024-06': -17.92, '2024-07': 38.78, '2024-08': -63.55, '2024-09': 11.06, '2024-10': 23.7, 'profit_month_ratio%': 76.9230769230769, '2024-11': nan, 'coin_interval': 'SUIUSDT_1h'}
      {'symbol': 'TIAUSDT', 'interval': '1h', 'window_length': 54, 'min_space': 5, 'sloping_atr_length': 21, 'take_profit_multiplier': 1.25718831663387, 'stop_loss_multiplier': 2.33219612688292,

In [ ]:
symbols = ['BTCUSDT', 'ETHUSDT', 'TIAUSDT']
parameters = {
    '1000PEPEUSDT': {'window_length': 32, 'min_space': 5, 'sloping_atr_length': 10, 'take_profit_multiplier': 2, 'stop_loss_multiplier': 1.5},
    'ETHUSDT': {'window_length': 12, 'min_space': 5, 'sloping_atr_length': 8, 'take_profit_multiplier': 1.8, 'stop_loss_multiplier': 1.4},
    'TIAUSDT': {'window_length': 10, 'min_space': 5, 'sloping_atr_length': 12, 'take_profit_multiplier': 2.2, 'stop_loss_multiplier': 1.6},
}

start_date = '2023-10'
end_date = '2024-11'
interval = '1р'

results = run_backtest(symbols, SlopingStrategy, parameters, start_date, end_date, interval)


# Draft

In [ ]:
if best_params:
    final_results = final_backtest(
        data=df,
        symbol=symbol,
        interval=interval,
        best_params=best_params,
        start_date=start_date,
        end_date=end_date,
        initial_capital=1_000_000,
        commission=0.0005
    )

    # Преобразуем результат в DataFrame для удобного анализа
    final_results_df = pd.DataFrame([final_results])
    print(final_results_df)

In [ ]:
def backtest_and_analyze_monthly(strategy, data, start_date, end_date, initial_capital=1_000_000):
    # Приводим индексы данных в порядок
    data.index = pd.to_datetime(data.index).tz_localize(None)

    # Выполняем бэктест
    test = backtesting.Backtest(data, strategy, cash=initial_capital, commission=0.0005)
    res = test.run()

    # Получаем таблицу сделок
    trades = res._trades.copy()

    # Конвертируем даты в формат pandas
    trades['Close Time'] = pd.to_datetime(trades['ExitTime'])
    trades.set_index('Close Time', inplace=True)

    # Генерация интервалов по месяцам
    intervals = generate_monthly_intervals(start_date, end_date)

    # Анализ по месяцам
    monthly_results = {}
    for start, end in intervals:
        start_date = datetime.strptime(start, "%Y-%m")
        end_date = datetime.strptime(end, "%Y-%m")

        # Отбираем сделки за месяц
        monthly_trades = trades[(trades.index >= start_date) & (trades.index < end_date)]

        if monthly_trades.empty:
            monthly_results[start] = {'pnl': 0.0, 'trade_count': 0}
        else:
            monthly_pnl = monthly_trades['PnL'].sum()
            trade_count = len(monthly_trades)
            monthly_results[start] = {'pnl': monthly_pnl, 'trade_count': trade_count}

    # Формируем итоговый анализ
    total_pnl = trades['PnL'].sum()
    profit_months = len([m for m, v in monthly_results.items() if v['pnl'] > 0])
    loss_months = len([m for m, v in monthly_results.items() if v['pnl'] <= 0])

    print("\nАнализ по месяцам:")
    for month, result in monthly_results.items():
        print(f"{month}: PnL = {result['pnl']:.2f}, сделок = {result['trade_count']}")

    print(f"\nОбщий профит: {total_pnl:.2f}")
    print(f"Профитных месяцев: {profit_months}, Убыточных месяцев: {loss_months}")

    return monthly_results, total_pnl, profit_months, loss_months, trades
